In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import NuLattice.smearedInteraction as smearI
import NuLattice.CCM.coupled_cluster as ccm
import NuLattice.references as ref
import NuLattice.FCI.few_body_diagonalization as fbd
import NuLattice.lattice as lat
import NuLattice.constants as consts
from scipy.sparse.linalg import eigsh as arpack_eigsh

In [15]:
myL= 4
bpi = 0.7
sL = 0.08
sNL = 0.08
a = 1.0 / 100.0
at = 1.0 / 150.0
c0 = -1.85e-05/ (a ** 2)
a_lat = consts.hbarc * a
kin = smearI.tKin(lat.get_lattice(myL), myL, lat.phys_unit(a_lat))
kinTest  = np.zeros([myL ** 3, myL ** 3])
for i in kin:
        ind1, sz1, tz1 = smearI.indConv(i[0], myL)
        ind2, sz2, tz2 = smearI.indConv(i[1], myL)
        if sz1 + tz1 + sz2 + tz2 == 0:
            kinTest[ind1, ind2] += i[2] 
kin2 = smearI.tKin2(myL, 2, a)
lst = []
for i in range(myL ** 3):
    for j in range(myL ** 3):
        val1 = kinTest[i][j]
        val2 = kin2[i][j] / a
        if val1 != val2:
             comp = val1 / val2
             if comp not in lst:
                  print(f'At {i},{j}:\t my val:{val1}\t their val:{val2}')
                  print(comp)
                  lst.append(comp)

At 0,1:	 my val:-7.100365847738247	 their val:-7.100365847738248
0.9999999999999999


In [16]:
tKin, inter, trMat = smearI.get_full_int(myL, bpi, c0, sL, sNL, a, at, nk=2)
holes = ref.ref_2H_gs
no2b_stuff = ccm.get_norm_ordered_ham(myL, holes, tKin, inter)

In [17]:
# Compute the deuteron
# additive quantum numbers
numpart=2 # number of nucleons
tz = 0    # twice the value of isospin projection
sz = 2    # twice the value of spin projection
my_basis = lat.get_sp_basis(myL)
nstat =  len(my_basis)

# get two-body basis as a dictionary for lookup
H2_lookup = fbd.get_many_body_states(my_basis, numpart, total_tz=tz, total_sz=sz)
print("matrix dimension:", len(H2_lookup))

# make scipy.sparse.csr_matrix for kinetic energy 
T2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, tKin, nstat)
print("kinetic energy done")

# make scipy.sparse.csr_matrix for 2-body interactions 
V2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, inter, nstat)
print("2-body interaction done")

# add all into Hamiltonian
H2_csr_mat = T2_csr_mat + V2_csr_mat

# compute lowest eigenvalue(s)
k_eig=20  # number of eigenvalues
vals, vecs = arpack_eigsh(H2_csr_mat, k=k_eig, which='SA')
print("Energies (MeV):", vals)

matrix dimension: 4096
kinetic energy done
2-body interaction done
Energies (MeV): [0.39939558 0.39939558 0.39939558 0.39939558 0.39939558 0.39939558
 0.39939558 0.39939558 0.39939558 0.39939558 0.39939558 0.39939558
 0.39939558 0.39939558 0.39939558 0.39939558 0.39939558 0.39939558
 0.39939558 0.39939558]


In [18]:
vals, vecs = arpack_eigsh(trMat, k=10, which='LA')
vals = -np.log(vals) / at
print(vals / a)

[48.28521887 48.28521887 43.08896375 24.06483535 24.06483535 24.06483535
 24.06111369 24.06111369 15.56570741 -5.28575756]


In [39]:
testL4Kin =   [[(1,1), 0.994211542426410],
   [(2,1), 0.001069717680456],
   [(3,1),-0.000211817505560],
   [(4,1), 0.001069717680456],
   [(5,1), 0.001069717680456],
   [(6,1), 0.000000575603073],
   [(7,1),-0.000000114274140],
   [(8,1), 0.000000575603073],
   [(9,1),-0.000211817505560],
  [(10,1),-0.000000114274140],
  [(11,1), 0.000000022686778],
  [(12,1),-0.000000114274140],
  [(13,1), 0.001069717680456],
  [(14,1), 0.000000575603073],
  [(15,1),-0.000000114274140],
  [(16,1), 0.000000575603073],
  [(17,1), 0.001069717680456],
  [(18,1), 0.000000575603073],
  [(19,1),-0.000000114274140],
  [(20,1), 0.000000575603073],
  [(21,1), 0.000000575603073],
  [(25,1),-0.000000114274140],
  [(29,1), 0.000000575603073],
  [(33,1),-0.000211817505560],
  [(34,1),-0.000000114274140],
  [(35,1), 0.000000022686778],
  [(36,1),-0.000000114274140],
  [(37,1),-0.000000114274140],
  [(41,1), 0.000000022686778],
  [(45,1),-0.000000114274140],
  [(49,1), 0.001069717680456],
  [(50,1), 0.000000575603073],
  [(51,1),-0.000000114274140],
  [(52,1), 0.000000575603073],
  [(53,1), 0.000000575603073],
  [(57,1),-0.000000114274140],
  [(61,1), 0.000000575603073],
   [(1,2), 0.001069717680456],
   [(2,2), 0.994211542426410],
   [(3,2), 0.001069717680456],
   [(4,2),-0.000211817505560],
   [(5,2), 0.000000575603073],
   [(6,2), 0.001069717680456],
   [(7,2), 0.000000575603073],
   [(8,2),-0.000000114274140],
   [(9,2),-0.000000114274140],
  [(10,2),-0.000211817505560],
  [(11,2),-0.000000114274140],
  [(12,2), 0.000000022686778],
  [(13,2), 0.000000575603073],
  [(14,2), 0.001069717680456],
  [(15,2), 0.000000575603073],
  [(16,2),-0.000000114274140],
  [(17,2), 0.000000575603073],
  [(18,2), 0.001069717680456],
  [(19,2), 0.000000575603073],
  [(20,2),-0.000000114274140],
  [(22,2), 0.000000575603073],
  [(26,2),-0.000000114274140],
  [(30,2), 0.000000575603073],
  [(33,2),-0.000000114274140],
  [(34,2),-0.000211817505560],
  [(35,2),-0.000000114274140],
  [(36,2), 0.000000022686778],
  [(38,2),-0.000000114274140],
  [(42,2), 0.000000022686778],
  [(46,2),-0.000000114274140],
  [(49,2), 0.000000575603073],
  [(50,2), 0.001069717680456],
  [(51,2), 0.000000575603073],
  [(52,2),-0.000000114274140],
  [(54,2), 0.000000575603073],
  [(58,2),-0.000000114274140],
  [(62,2), 0.000000575603073],
   [(1,3),-0.000211817505560],
   [(2,3), 0.001069717680456],
   [(3,3), 0.994211542426410],
   [(4,3), 0.001069717680456],
   [(5,3),-0.000000114274140],
   [(6,3), 0.000000575603073],
   [(7,3), 0.001069717680456],
   [(8,3), 0.000000575603073],
   [(9,3), 0.000000022686778],
  [(10,3),-0.000000114274140],
  [(11,3),-0.000211817505560],
  [(12,3),-0.000000114274140],
  [(13,3),-0.000000114274140],
  [(14,3), 0.000000575603073],
  [(15,3), 0.001069717680456],
  [(16,3), 0.000000575603073],
  [(17,3),-0.000000114274140],
  [(18,3), 0.000000575603073],
  [(19,3), 0.001069717680456],
  [(20,3), 0.000000575603073],
  [(23,3), 0.000000575603073],
  [(27,3),-0.000000114274140],
  [(31,3), 0.000000575603073],
  [(33,3), 0.000000022686778],
  [(34,3),-0.000000114274140],
  [(35,3),-0.000211817505560],
  [(36,3),-0.000000114274140],
  [(39,3),-0.000000114274140],
  [(43,3), 0.000000022686778],
  [(47,3),-0.000000114274140],
  [(49,3),-0.000000114274140],
  [(50,3), 0.000000575603073],
  [(51,3), 0.001069717680456],
  [(52,3), 0.000000575603073],
  [(55,3), 0.000000575603073],
  [(59,3),-0.000000114274140],
  [(63,3), 0.000000575603073],
   [(1,4), 0.001069717680456],
   [(2,4),-0.000211817505560],
   [(3,4), 0.001069717680456],
   [(4,4), 0.994211542426410],
   [(5,4), 0.000000575603073],
   [(6,4),-0.000000114274140],
   [(7,4), 0.000000575603073],
   [(8,4), 0.001069717680456],
   [(9,4),-0.000000114274140],
  [(10,4), 0.000000022686778],
  [(11,4),-0.000000114274140],
  [(12,4),-0.000211817505560],
  [(13,4), 0.000000575603073],
  [(14,4),-0.000000114274140],
  [(15,4), 0.000000575603073],
  [(16,4), 0.001069717680456],
  [(17,4), 0.000000575603073],
  [(18,4),-0.000000114274140],
  [(19,4), 0.000000575603073],
  [(20,4), 0.001069717680456],
  [(24,4), 0.000000575603073],
  [(28,4),-0.000000114274140],
  [(32,4), 0.000000575603073],
  [(33,4),-0.000000114274140],
  [(34,4), 0.000000022686778],
  [(35,4),-0.000000114274140],
  [(36,4),-0.000211817505560],
  [(40,4),-0.000000114274140],
  [(44,4), 0.000000022686778],
  [(48,4),-0.000000114274140],
  [(49,4), 0.000000575603073],
  [(50,4),-0.000000114274140],
  [(51,4), 0.000000575603073],
  [(52,4), 0.001069717680456],
  [(56,4), 0.000000575603073],
  [(60,4),-0.000000114274140],
  [(64,4), 0.000000575603073],
   [(1,5), 0.001069717680456],
   [(2,5), 0.000000575603073],
   [(3,5),-0.000000114274140],
   [(4,5), 0.000000575603073],
   [(5,5), 0.994211542426410],
   [(6,5), 0.001069717680456],
   [(7,5),-0.000211817505560],
   [(8,5), 0.001069717680456],
   [(9,5), 0.001069717680456],
  [(10,5), 0.000000575603073],
  [(11,5),-0.000000114274140],
  [(12,5), 0.000000575603073],
  [(13,5),-0.000211817505560],
  [(14,5),-0.000000114274140],
  [(15,5), 0.000000022686778],
  [(16,5),-0.000000114274140],
  [(17,5), 0.000000575603073],
  [(21,5), 0.001069717680456],
  [(22,5), 0.000000575603073],
  [(23,5),-0.000000114274140],
  [(24,5), 0.000000575603073],
  [(25,5), 0.000000575603073],
  [(29,5),-0.000000114274140],
  [(33,5),-0.000000114274140],
  [(37,5),-0.000211817505560],
  [(38,5),-0.000000114274140],
  [(39,5), 0.000000022686778],
  [(40,5),-0.000000114274140],
  [(41,5),-0.000000114274140],
  [(45,5), 0.000000022686778],
  [(49,5), 0.000000575603073],
  [(53,5), 0.001069717680456],
  [(54,5), 0.000000575603073],
  [(55,5),-0.000000114274140],
  [(56,5), 0.000000575603073],
  [(57,5), 0.000000575603073],
  [(61,5),-0.000000114274140],
   [(1,6), 0.000000575603073],
   [(2,6), 0.001069717680456],
   [(3,6), 0.000000575603073],
   [(4,6),-0.000000114274140],
   [(5,6), 0.001069717680456],
   [(6,6), 0.994211542426410],
   [(7,6), 0.001069717680456],
   [(8,6),-0.000211817505560],
   [(9,6), 0.000000575603073],
  [(10,6), 0.001069717680456],
  [(11,6), 0.000000575603073],
  [(12,6),-0.000000114274140],
  [(13,6),-0.000000114274140],
  [(14,6),-0.000211817505560],
  [(15,6),-0.000000114274140],
  [(16,6), 0.000000022686778],
  [(18,6), 0.000000575603073],
  [(21,6), 0.000000575603073],
  [(22,6), 0.001069717680456],
  [(23,6), 0.000000575603073],
  [(24,6),-0.000000114274140],
  [(26,6), 0.000000575603073],
  [(30,6),-0.000000114274140],
  [(34,6),-0.000000114274140],
  [(37,6),-0.000000114274140],
  [(38,6),-0.000211817505560],
  [(39,6),-0.000000114274140],
  [(40,6), 0.000000022686778],
  [(42,6),-0.000000114274140],
  [(46,6), 0.000000022686778],
  [(50,6), 0.000000575603073],
  [(53,6), 0.000000575603073],
  [(54,6), 0.001069717680456],
  [(55,6), 0.000000575603073],
  [(56,6),-0.000000114274140],
  [(58,6), 0.000000575603073],
  [(62,6),-0.000000114274140],
   [(1,7),-0.000000114274140],
   [(2,7), 0.000000575603073],
   [(3,7), 0.001069717680456],
   [(4,7), 0.000000575603073],
   [(5,7),-0.000211817505560],
   [(6,7), 0.001069717680456],
   [(7,7), 0.994211542426410],
   [(8,7), 0.001069717680456],
   [(9,7),-0.000000114274140],
  [(10,7), 0.000000575603073],
  [(11,7), 0.001069717680456],
  [(12,7), 0.000000575603073],
  [(13,7), 0.000000022686778],
  [(14,7),-0.000000114274140],
  [(15,7),-0.000211817505560],
  [(16,7),-0.000000114274140],
  [(19,7), 0.000000575603073],
  [(21,7),-0.000000114274140],
  [(22,7), 0.000000575603073],
  [(23,7), 0.001069717680456],
  [(24,7), 0.000000575603073],
  [(27,7), 0.000000575603073],
  [(31,7),-0.000000114274140],
  [(35,7),-0.000000114274140],
  [(37,7), 0.000000022686778],
  [(38,7),-0.000000114274140],
  [(39,7),-0.000211817505560],
  [(40,7),-0.000000114274140],
  [(43,7),-0.000000114274140],
  [(47,7), 0.000000022686778],
  [(51,7), 0.000000575603073],
  [(53,7),-0.000000114274140],
  [(54,7), 0.000000575603073],
  [(55,7), 0.001069717680456],
  [(56,7), 0.000000575603073],
  [(59,7), 0.000000575603073],
  [(63,7),-0.000000114274140],
   [(1,8), 0.000000575603073],
   [(2,8),-0.000000114274140],
   [(3,8), 0.000000575603073],
   [(4,8), 0.001069717680456],
   [(5,8), 0.001069717680456],
   [(6,8),-0.000211817505560],
   [(7,8), 0.001069717680456],
   [(8,8), 0.994211542426410],
   [(9,8), 0.000000575603073],
  [(10,8),-0.000000114274140],
  [(11,8), 0.000000575603073],
  [(12,8), 0.001069717680456],
  [(13,8),-0.000000114274140],
  [(14,8), 0.000000022686778],
  [(15,8),-0.000000114274140],
  [(16,8),-0.000211817505560],
  [(20,8), 0.000000575603073],
  [(21,8), 0.000000575603073],
  [(22,8),-0.000000114274140],
  [(23,8), 0.000000575603073],
  [(24,8), 0.001069717680456],
  [(28,8), 0.000000575603073],
  [(32,8),-0.000000114274140],
  [(36,8),-0.000000114274140],
  [(37,8),-0.000000114274140],
  [(38,8), 0.000000022686778],
  [(39,8),-0.000000114274140],
  [(40,8),-0.000211817505560],
  [(44,8),-0.000000114274140],
  [(48,8), 0.000000022686778],
  [(52,8), 0.000000575603073],
  [(53,8), 0.000000575603073],
  [(54,8),-0.000000114274140],
  [(55,8), 0.000000575603073],
  [(56,8), 0.001069717680456],
  [(60,8), 0.000000575603073],
  [(64,8),-0.000000114274140],
   [(1,9),-0.000211817505560],
   [(2,9),-0.000000114274140],
   [(3,9), 0.000000022686778],
   [(4,9),-0.000000114274140],
   [(5,9), 0.001069717680456],
   [(6,9), 0.000000575603073],
   [(7,9),-0.000000114274140],
   [(8,9), 0.000000575603073],
   [(9,9), 0.994211542426410],
  [(10,9), 0.001069717680456],
  [(11,9),-0.000211817505560],
  [(12,9), 0.001069717680456],
  [(13,9), 0.001069717680456],
  [(14,9), 0.000000575603073],
  [(15,9),-0.000000114274140],
  [(16,9), 0.000000575603073],
  [(17,9),-0.000000114274140],
  [(21,9), 0.000000575603073],
  [(25,9), 0.001069717680456],
  [(26,9), 0.000000575603073],
  [(27,9),-0.000000114274140],
  [(28,9), 0.000000575603073],
  [(29,9), 0.000000575603073],
  [(33,9), 0.000000022686778],
  [(37,9),-0.000000114274140],
  [(41,9),-0.000211817505560],
  [(42,9),-0.000000114274140],
  [(43,9), 0.000000022686778],
  [(44,9),-0.000000114274140],
  [(45,9),-0.000000114274140],
  [(49,9),-0.000000114274140],
  [(53,9), 0.000000575603073],
  [(57,9), 0.001069717680456],
  [(58,9), 0.000000575603073],
  [(59,9),-0.000000114274140],
  [(60,9), 0.000000575603073],
  [(61,9), 0.000000575603073],
   [(1,10),-0.000000114274140],
   [(2,10),-0.000211817505560],
   [(3,10),-0.000000114274140],
   [(4,10),0.000000022686778],
   [(5,10),0.000000575603073],
   [(6,10),0.001069717680456],
   [(7,10),0.000000575603073],
   [(8,10),-0.000000114274140],
   [(9,10),0.001069717680456],
  [(10,10),0.994211542426410],
  [(11,10),0.001069717680456],
  [(12,10),-0.000211817505560],
  [(13,10),0.000000575603073],
  [(14,10),0.001069717680456],
  [(15,10),0.000000575603073],
  [(16,10),-0.000000114274140],
  [(18,10),-0.000000114274140],
  [(22,10),0.000000575603073],
  [(25,10),0.000000575603073],
  [(26,10),0.001069717680456],
  [(27,10),0.000000575603073],
  [(28,10),-0.000000114274140],
  [(30,10),0.000000575603073],
  [(34,10),0.000000022686778],
  [(38,10),-0.000000114274140],
  [(41,10),-0.000000114274140],
  [(42,10),-0.000211817505560],
  [(43,10),-0.000000114274140],
  [(44,10),0.000000022686778],
  [(46,10),-0.000000114274140],
  [(50,10),-0.000000114274140],
  [(54,10),0.000000575603073],
  [(57,10),0.000000575603073],
  [(58,10),0.001069717680456],
  [(59,10),0.000000575603073],
  [(60,10),-0.000000114274140],
  [(62,10),0.000000575603073],
   [(1,11),0.000000022686778],
   [(2,11),-0.000000114274140],
   [(3,11),-0.000211817505560],
   [(4,11),-0.000000114274140],
   [(5,11),-0.000000114274140],
   [(6,11),0.000000575603073],
   [(7,11),0.001069717680456],
   [(8,11),0.000000575603073],
   [(9,11),-0.000211817505560],
  [(10,11),0.001069717680456],
  [(11,11),0.994211542426410],
  [(12,11),0.001069717680456],
  [(13,11),-0.000000114274140],
  [(14,11),0.000000575603073],
  [(15,11),0.001069717680456],
  [(16,11),0.000000575603073],
  [(19,11),-0.000000114274140],
  [(23,11),0.000000575603073],
  [(25,11),-0.000000114274140],
  [(26,11),0.000000575603073],
  [(27,11),0.001069717680456],
  [(28,11),0.000000575603073],
  [(31,11),0.000000575603073],
  [(35,11),0.000000022686778],
  [(39,11),-0.000000114274140],
  [(41,11),0.000000022686778],
  [(42,11),-0.000000114274140],
  [(43,11),-0.000211817505560],
  [(44,11),-0.000000114274140],
  [(47,11),-0.000000114274140],
  [(51,11),-0.000000114274140],
  [(55,11),0.000000575603073],
  [(57,11),-0.000000114274140],
  [(58,11),0.000000575603073],
  [(59,11),0.001069717680456],
  [(60,11),0.000000575603073],
  [(63,11),0.000000575603073],
   [(1,12),-0.000000114274140],
   [(2,12),0.000000022686778],
   [(3,12),-0.000000114274140],
   [(4,12),-0.000211817505560],
   [(5,12),0.000000575603073],
   [(6,12),-0.000000114274140],
   [(7,12),0.000000575603073],
   [(8,12),0.001069717680456],
   [(9,12),0.001069717680456],
  [(10,12),-0.000211817505560],
  [(11,12),0.001069717680456],
  [(12,12),0.994211542426410],
  [(13,12),0.000000575603073],
  [(14,12),-0.000000114274140],
  [(15,12),0.000000575603073],
  [(16,12),0.001069717680456],
  [(20,12),-0.000000114274140],
  [(24,12),0.000000575603073],
  [(25,12),0.000000575603073],
  [(26,12),-0.000000114274140],
  [(27,12),0.000000575603073],
  [(28,12),0.001069717680456],
  [(32,12),0.000000575603073],
  [(36,12),0.000000022686778],
  [(40,12),-0.000000114274140],
  [(41,12),-0.000000114274140],
  [(42,12),0.000000022686778],
  [(43,12),-0.000000114274140],
  [(44,12),-0.000211817505560],
  [(48,12),-0.000000114274140],
  [(52,12),-0.000000114274140],
  [(56,12),0.000000575603073],
  [(57,12),0.000000575603073],
  [(58,12),-0.000000114274140],
  [(59,12),0.000000575603073],
  [(60,12),0.001069717680456],
  [(64,12),0.000000575603073],
   [(1,13),0.001069717680456],
   [(2,13),0.000000575603073],
   [(3,13),-0.000000114274140],
   [(4,13),0.000000575603073],
   [(5,13),-0.000211817505560],
   [(6,13),-0.000000114274140],
   [(7,13),0.000000022686778],
   [(8,13),-0.000000114274140],
   [(9,13),0.001069717680456],
  [(10,13),0.000000575603073],
  [(11,13),-0.000000114274140],
  [(12,13),0.000000575603073],
  [(13,13),0.994211542426410],
  [(14,13),0.001069717680456],
  [(15,13),-0.000211817505560],
  [(16,13),0.001069717680456],
  [(17,13),0.000000575603073],
  [(21,13),-0.000000114274140],
  [(25,13),0.000000575603073],
  [(29,13),0.001069717680456],
  [(30,13),0.000000575603073],
  [(31,13),-0.000000114274140],
  [(32,13),0.000000575603073],
  [(33,13),-0.000000114274140],
  [(37,13),0.000000022686778],
  [(41,13),-0.000000114274140],
  [(45,13),-0.000211817505560],
  [(46,13),-0.000000114274140],
  [(47,13),0.000000022686778],
  [(48,13),-0.000000114274140],
  [(49,13),0.000000575603073],
  [(53,13),-0.000000114274140],
  [(57,13),0.000000575603073],
  [(61,13),0.001069717680456],
  [(62,13),0.000000575603073],
  [(63,13),-0.000000114274140],
  [(64,13),0.000000575603073],
   [(1,14),0.000000575603073],
   [(2,14),0.001069717680456],
   [(3,14),0.000000575603073],
   [(4,14),-0.000000114274140],
   [(5,14),-0.000000114274140],
   [(6,14),-0.000211817505560],
   [(7,14),-0.000000114274140],
   [(8,14),0.000000022686778],
   [(9,14),0.000000575603073],
  [(10,14),0.001069717680456],
  [(11,14),0.000000575603073],
  [(12,14),-0.000000114274140],
  [(13,14),0.001069717680456],
  [(14,14),0.994211542426410],
  [(15,14),0.001069717680456],
  [(16,14),-0.000211817505560],
  [(18,14),0.000000575603073],
  [(22,14),-0.000000114274140],
  [(26,14),0.000000575603073],
  [(29,14),0.000000575603073],
  [(30,14),0.001069717680456],
  [(31,14),0.000000575603073],
  [(32,14),-0.000000114274140],
  [(34,14),-0.000000114274140],
  [(38,14),0.000000022686778],
  [(42,14),-0.000000114274140],
  [(45,14),-0.000000114274140],
  [(46,14),-0.000211817505560],
  [(47,14),-0.000000114274140],
  [(48,14),0.000000022686778],
  [(50,14),0.000000575603073],
  [(54,14),-0.000000114274140],
  [(58,14),0.000000575603073],
  [(61,14),0.000000575603073],
  [(62,14),0.001069717680456],
  [(63,14),0.000000575603073],
  [(64,14),-0.000000114274140],
   [(1,15),-0.000000114274140],
   [(2,15),0.000000575603073],
   [(3,15),0.001069717680456],
   [(4,15),0.000000575603073],
   [(5,15),0.000000022686778],
   [(6,15),-0.000000114274140],
   [(7,15),-0.000211817505560],
   [(8,15),-0.000000114274140],
   [(9,15),-0.000000114274140],
  [(10,15),0.000000575603073],
  [(11,15),0.001069717680456],
  [(12,15),0.000000575603073],
  [(13,15),-0.000211817505560],
  [(14,15),0.001069717680456],
  [(15,15),0.994211542426410],
  [(16,15),0.001069717680456],
  [(19,15),0.000000575603073],
  [(23,15),-0.000000114274140],
  [(27,15),0.000000575603073],
  [(29,15),-0.000000114274140],
  [(30,15),0.000000575603073],
  [(31,15),0.001069717680456],
  [(32,15),0.000000575603073],
  [(35,15),-0.000000114274140],
  [(39,15),0.000000022686778],
  [(43,15),-0.000000114274140],
  [(45,15),0.000000022686778],
  [(46,15),-0.000000114274140],
  [(47,15),-0.000211817505560],
  [(48,15),-0.000000114274140],
  [(51,15),0.000000575603073],
  [(55,15),-0.000000114274140],
  [(59,15),0.000000575603073],
  [(61,15),-0.000000114274140],
  [(62,15),0.000000575603073],
  [(63,15),0.001069717680456],
  [(64,15),0.000000575603073],
   [(1,16),0.000000575603073],
   [(2,16),-0.000000114274140],
   [(3,16),0.000000575603073],
   [(4,16),0.001069717680456],
   [(5,16),-0.000000114274140],
   [(6,16),0.000000022686778],
   [(7,16),-0.000000114274140],
   [(8,16),-0.000211817505560],
   [(9,16),0.000000575603073],
  [(10,16),-0.000000114274140],
  [(11,16),0.000000575603073],
  [(12,16),0.001069717680456],
  [(13,16),0.001069717680456],
  [(14,16),-0.000211817505560],
  [(15,16),0.001069717680456],
  [(16,16),0.994211542426410],
  [(20,16),0.000000575603073],
  [(24,16),-0.000000114274140],
  [(28,16),0.000000575603073],
  [(29,16),0.000000575603073],
  [(30,16),-0.000000114274140],
  [(31,16),0.000000575603073],
  [(32,16),0.001069717680456],
  [(36,16),-0.000000114274140],
  [(40,16),0.000000022686778],
  [(44,16),-0.000000114274140],
  [(45,16),-0.000000114274140],
  [(46,16),0.000000022686778],
  [(47,16),-0.000000114274140],
  [(48,16),-0.000211817505560],
  [(52,16),0.000000575603073],
  [(56,16),-0.000000114274140],
  [(60,16),0.000000575603073],
  [(61,16),0.000000575603073],
  [(62,16),-0.000000114274140],
  [(63,16),0.000000575603073],
  [(64,16),0.001069717680456],
   [(1,17),0.001069717680456],
   [(2,17),0.000000575603073],
   [(3,17),-0.000000114274140],
   [(4,17),0.000000575603073],
   [(5,17),0.000000575603073],
   [(9,17),-0.000000114274140],
  [(13,17),0.000000575603073],
  [(17,17),0.994211542426410],
  [(18,17),0.001069717680456],
  [(19,17),-0.000211817505560],
  [(20,17),0.001069717680456],
  [(21,17),0.001069717680456],
  [(22,17),0.000000575603073],
  [(23,17),-0.000000114274140],
  [(24,17),0.000000575603073],
  [(25,17),-0.000211817505560],
  [(26,17),-0.000000114274140],
  [(27,17),0.000000022686778],
  [(28,17),-0.000000114274140],
  [(29,17),0.001069717680456],
  [(30,17),0.000000575603073],
  [(31,17),-0.000000114274140],
  [(32,17),0.000000575603073],
  [(33,17),0.001069717680456],
  [(34,17),0.000000575603073],
  [(35,17),-0.000000114274140],
  [(36,17),0.000000575603073],
  [(37,17),0.000000575603073],
  [(41,17),-0.000000114274140],
  [(45,17),0.000000575603073],
  [(49,17),-0.000211817505560],
  [(50,17),-0.000000114274140],
  [(51,17),0.000000022686778],
  [(52,17),-0.000000114274140],
  [(53,17),-0.000000114274140],
  [(57,17),0.000000022686778],
  [(61,17),-0.000000114274140],
   [(1,18),0.000000575603073],
   [(2,18),0.001069717680456],
   [(3,18),0.000000575603073],
   [(4,18),-0.000000114274140],
   [(6,18),0.000000575603073],
  [(10,18),-0.000000114274140],
  [(14,18),0.000000575603073],
  [(17,18),0.001069717680456],
  [(18,18),0.994211542426410],
  [(19,18),0.001069717680456],
  [(20,18),-0.000211817505560],
  [(21,18),0.000000575603073],
  [(22,18),0.001069717680456],
  [(23,18),0.000000575603073],
  [(24,18),-0.000000114274140],
  [(25,18),-0.000000114274140],
  [(26,18),-0.000211817505560],
  [(27,18),-0.000000114274140],
  [(28,18),0.000000022686778],
  [(29,18),0.000000575603073],
  [(30,18),0.001069717680456],
  [(31,18),0.000000575603073],
  [(32,18),-0.000000114274140],
  [(33,18),0.000000575603073],
  [(34,18),0.001069717680456],
  [(35,18),0.000000575603073],
  [(36,18),-0.000000114274140],
  [(38,18),0.000000575603073],
  [(42,18),-0.000000114274140],
  [(46,18),0.000000575603073],
  [(49,18),-0.000000114274140],
  [(50,18),-0.000211817505560],
  [(51,18),-0.000000114274140],
  [(52,18),0.000000022686778],
  [(54,18),-0.000000114274140],
  [(58,18),0.000000022686778],
  [(62,18),-0.000000114274140],
   [(1,19),-0.000000114274140],
   [(2,19),0.000000575603073],
   [(3,19),0.001069717680456],
   [(4,19),0.000000575603073],
   [(7,19),0.000000575603073],
  [(11,19),-0.000000114274140],
  [(15,19),0.000000575603073],
  [(17,19),-0.000211817505560],
  [(18,19),0.001069717680456],
  [(19,19),0.994211542426410],
  [(20,19),0.001069717680456],
  [(21,19),-0.000000114274140],
  [(22,19),0.000000575603073],
  [(23,19),0.001069717680456],
  [(24,19),0.000000575603073],
  [(25,19),0.000000022686778],
  [(26,19),-0.000000114274140],
  [(27,19),-0.000211817505560],
  [(28,19),-0.000000114274140],
  [(29,19),-0.000000114274140],
  [(30,19),0.000000575603073],
  [(31,19),0.001069717680456],
  [(32,19),0.000000575603073],
  [(33,19),-0.000000114274140],
  [(34,19),0.000000575603073],
  [(35,19),0.001069717680456],
  [(36,19),0.000000575603073],
  [(39,19),0.000000575603073],
  [(43,19),-0.000000114274140],
  [(47,19),0.000000575603073],
  [(49,19),0.000000022686778],
  [(50,19),-0.000000114274140],
  [(51,19),-0.000211817505560],
  [(52,19),-0.000000114274140],
  [(55,19),-0.000000114274140],
  [(59,19),0.000000022686778],
  [(63,19),-0.000000114274140],
   [(1,20),0.000000575603073],
   [(2,20),-0.000000114274140],
   [(3,20),0.000000575603073],
   [(4,20),0.001069717680456],
   [(8,20),0.000000575603073],
  [(12,20),-0.000000114274140],
  [(16,20),0.000000575603073],
  [(17,20),0.001069717680456],
  [(18,20),-0.000211817505560],
  [(19,20),0.001069717680456],
  [(20,20),0.994211542426410],
  [(21,20),0.000000575603073],
  [(22,20),-0.000000114274140],
  [(23,20),0.000000575603073],
  [(24,20),0.001069717680456],
  [(25,20),-0.000000114274140],
  [(26,20),0.000000022686778],
  [(27,20),-0.000000114274140],
  [(28,20),-0.000211817505560],
  [(29,20),0.000000575603073],
  [(30,20),-0.000000114274140],
  [(31,20),0.000000575603073],
  [(32,20),0.001069717680456],
  [(33,20),0.000000575603073],
  [(34,20),-0.000000114274140],
  [(35,20),0.000000575603073],
  [(36,20),0.001069717680456],
  [(40,20),0.000000575603073],
  [(44,20),-0.000000114274140],
  [(48,20),0.000000575603073],
  [(49,20),-0.000000114274140],
  [(50,20),0.000000022686778],
  [(51,20),-0.000000114274140],
  [(52,20),-0.000211817505560],
  [(56,20),-0.000000114274140],
  [(60,20),0.000000022686778],
  [(64,20),-0.000000114274140],
   [(1,21),0.000000575603073],
   [(5,21),0.001069717680456],
   [(6,21),0.000000575603073],
   [(7,21),-0.000000114274140],
   [(8,21),0.000000575603073],
   [(9,21),0.000000575603073],
  [(13,21),-0.000000114274140],
  [(17,21),0.001069717680456],
  [(18,21),0.000000575603073],
  [(19,21),-0.000000114274140],
  [(20,21),0.000000575603073],
  [(21,21),0.994211542426410],
  [(22,21),0.001069717680456],
  [(23,21),-0.000211817505560],
  [(24,21),0.001069717680456],
  [(25,21),0.001069717680456],
  [(26,21),0.000000575603073],
  [(27,21),-0.000000114274140],
  [(28,21),0.000000575603073],
  [(29,21),-0.000211817505560],
  [(30,21),-0.000000114274140],
  [(31,21),0.000000022686778],
  [(32,21),-0.000000114274140],
  [(33,21),0.000000575603073],
  [(37,21),0.001069717680456],
  [(38,21),0.000000575603073],
  [(39,21),-0.000000114274140],
  [(40,21),0.000000575603073],
  [(41,21),0.000000575603073],
  [(45,21),-0.000000114274140],
  [(49,21),-0.000000114274140],
  [(53,21),-0.000211817505560],
  [(54,21),-0.000000114274140],
  [(55,21),0.000000022686778],
  [(56,21),-0.000000114274140],
  [(57,21),-0.000000114274140],
  [(61,21),0.000000022686778],
   [(2,22),0.000000575603073],
   [(5,22),0.000000575603073],
   [(6,22),0.001069717680456],
   [(7,22),0.000000575603073],
   [(8,22),-0.000000114274140],
  [(10,22),0.000000575603073],
  [(14,22),-0.000000114274140],
  [(17,22),0.000000575603073],
  [(18,22),0.001069717680456],
  [(19,22),0.000000575603073],
  [(20,22),-0.000000114274140],
  [(21,22),0.001069717680456],
  [(22,22),0.994211542426410],
  [(23,22),0.001069717680456],
  [(24,22),-0.000211817505560],
  [(25,22),0.000000575603073],
  [(26,22),0.001069717680456],
  [(27,22),0.000000575603073],
  [(28,22),-0.000000114274140],
  [(29,22),-0.000000114274140],
  [(30,22),-0.000211817505560],
  [(31,22),-0.000000114274140],
  [(32,22),0.000000022686778],
  [(34,22),0.000000575603073],
  [(37,22),0.000000575603073],
  [(38,22),0.001069717680456],
  [(39,22),0.000000575603073],
  [(40,22),-0.000000114274140],
  [(42,22),0.000000575603073],
  [(46,22),-0.000000114274140],
  [(50,22),-0.000000114274140],
  [(53,22),-0.000000114274140],
  [(54,22),-0.000211817505560],
  [(55,22),-0.000000114274140],
  [(56,22),0.000000022686778],
  [(58,22),-0.000000114274140],
  [(62,22),0.000000022686778],
   [(3,23),0.000000575603073],
   [(5,23),-0.000000114274140],
   [(6,23),0.000000575603073],
   [(7,23),0.001069717680456],
   [(8,23),0.000000575603073],
  [(11,23),0.000000575603073],
  [(15,23),-0.000000114274140],
  [(17,23),-0.000000114274140],
  [(18,23),0.000000575603073],
  [(19,23),0.001069717680456],
  [(20,23),0.000000575603073],
  [(21,23),-0.000211817505560],
  [(22,23),0.001069717680456],
  [(23,23),0.994211542426410],
  [(24,23),0.001069717680456],
  [(25,23),-0.000000114274140],
  [(26,23),0.000000575603073],
  [(27,23),0.001069717680456],
  [(28,23),0.000000575603073],
  [(29,23),0.000000022686778],
  [(30,23),-0.000000114274140],
  [(31,23),-0.000211817505560],
  [(32,23),-0.000000114274140],
  [(35,23),0.000000575603073],
  [(37,23),-0.000000114274140],
  [(38,23),0.000000575603073],
  [(39,23),0.001069717680456],
  [(40,23),0.000000575603073],
  [(43,23),0.000000575603073],
  [(47,23),-0.000000114274140],
  [(51,23),-0.000000114274140],
  [(53,23),0.000000022686778],
  [(54,23),-0.000000114274140],
  [(55,23),-0.000211817505560],
  [(56,23),-0.000000114274140],
  [(59,23),-0.000000114274140],
  [(63,23),0.000000022686778],
   [(4,24),0.000000575603073],
   [(5,24),0.000000575603073],
   [(6,24),-0.000000114274140],
   [(7,24),0.000000575603073],
   [(8,24),0.001069717680456],
  [(12,24),0.000000575603073],
  [(16,24),-0.000000114274140],
  [(17,24),0.000000575603073],
  [(18,24),-0.000000114274140],
  [(19,24),0.000000575603073],
  [(20,24),0.001069717680456],
  [(21,24),0.001069717680456],
  [(22,24),-0.000211817505560],
  [(23,24),0.001069717680456],
  [(24,24),0.994211542426410],
  [(25,24),0.000000575603073],
  [(26,24),-0.000000114274140],
  [(27,24),0.000000575603073],
  [(28,24),0.001069717680456],
  [(29,24),-0.000000114274140],
  [(30,24),0.000000022686778],
  [(31,24),-0.000000114274140],
  [(32,24),-0.000211817505560],
  [(36,24),0.000000575603073],
  [(37,24),0.000000575603073],
  [(38,24),-0.000000114274140],
  [(39,24),0.000000575603073],
  [(40,24),0.001069717680456],
  [(44,24),0.000000575603073],
  [(48,24),-0.000000114274140],
  [(52,24),-0.000000114274140],
  [(53,24),-0.000000114274140],
  [(54,24),0.000000022686778],
  [(55,24),-0.000000114274140],
  [(56,24),-0.000211817505560],
  [(60,24),-0.000000114274140],
  [(64,24),0.000000022686778],
   [(1,25),-0.000000114274140],
   [(5,25),0.000000575603073],
   [(9,25),0.001069717680456],
  [(10,25),0.000000575603073],
  [(11,25),-0.000000114274140],
  [(12,25),0.000000575603073],
  [(13,25),0.000000575603073],
  [(17,25),-0.000211817505560],
  [(18,25),-0.000000114274140],
  [(19,25),0.000000022686778],
  [(20,25),-0.000000114274140],
  [(21,25),0.001069717680456],
  [(22,25),0.000000575603073],
  [(23,25),-0.000000114274140],
  [(24,25),0.000000575603073],
  [(25,25),0.994211542426410],
  [(26,25),0.001069717680456],
  [(27,25),-0.000211817505560],
  [(28,25),0.001069717680456],
  [(29,25),0.001069717680456],
  [(30,25),0.000000575603073],
  [(31,25),-0.000000114274140],
  [(32,25),0.000000575603073],
  [(33,25),-0.000000114274140],
  [(37,25),0.000000575603073],
  [(41,25),0.001069717680456],
  [(42,25),0.000000575603073],
  [(43,25),-0.000000114274140],
  [(44,25),0.000000575603073],
  [(45,25),0.000000575603073],
  [(49,25),0.000000022686778],
  [(53,25),-0.000000114274140],
  [(57,25),-0.000211817505560],
  [(58,25),-0.000000114274140],
  [(59,25),0.000000022686778],
  [(60,25),-0.000000114274140],
  [(61,25),-0.000000114274140],
   [(2,26),-0.000000114274140],
   [(6,26),0.000000575603073],
   [(9,26),0.000000575603073],
  [(10,26),0.001069717680456],
  [(11,26),0.000000575603073],
  [(12,26),-0.000000114274140],
  [(14,26),0.000000575603073],
  [(17,26),-0.000000114274140],
  [(18,26),-0.000211817505560],
  [(19,26),-0.000000114274140],
  [(20,26),0.000000022686778],
  [(21,26),0.000000575603073],
  [(22,26),0.001069717680456],
  [(23,26),0.000000575603073],
  [(24,26),-0.000000114274140],
  [(25,26),0.001069717680456],
  [(26,26),0.994211542426410],
  [(27,26),0.001069717680456],
  [(28,26),-0.000211817505560],
  [(29,26),0.000000575603073],
  [(30,26),0.001069717680456],
  [(31,26),0.000000575603073],
  [(32,26),-0.000000114274140],
  [(34,26),-0.000000114274140],
  [(38,26),0.000000575603073],
  [(41,26),0.000000575603073],
  [(42,26),0.001069717680456],
  [(43,26),0.000000575603073],
  [(44,26),-0.000000114274140],
  [(46,26),0.000000575603073],
  [(50,26),0.000000022686778],
  [(54,26),-0.000000114274140],
  [(57,26),-0.000000114274140],
  [(58,26),-0.000211817505560],
  [(59,26),-0.000000114274140],
  [(60,26),0.000000022686778],
  [(62,26),-0.000000114274140],
   [(3,27),-0.000000114274140],
   [(7,27),0.000000575603073],
   [(9,27),-0.000000114274140],
  [(10,27),0.000000575603073],
  [(11,27),0.001069717680456],
  [(12,27),0.000000575603073],
  [(15,27),0.000000575603073],
  [(17,27),0.000000022686778],
  [(18,27),-0.000000114274140],
  [(19,27),-0.000211817505560],
  [(20,27),-0.000000114274140],
  [(21,27),-0.000000114274140],
  [(22,27),0.000000575603073],
  [(23,27),0.001069717680456],
  [(24,27),0.000000575603073],
  [(25,27),-0.000211817505560],
  [(26,27),0.001069717680456],
  [(27,27),0.994211542426410],
  [(28,27),0.001069717680456],
  [(29,27),-0.000000114274140],
  [(30,27),0.000000575603073],
  [(31,27),0.001069717680456],
  [(32,27),0.000000575603073],
  [(35,27),-0.000000114274140],
  [(39,27),0.000000575603073],
  [(41,27),-0.000000114274140],
  [(42,27),0.000000575603073],
  [(43,27),0.001069717680456],
  [(44,27),0.000000575603073],
  [(47,27),0.000000575603073],
  [(51,27),0.000000022686778],
  [(55,27),-0.000000114274140],
  [(57,27),0.000000022686778],
  [(58,27),-0.000000114274140],
  [(59,27),-0.000211817505560],
  [(60,27),-0.000000114274140],
  [(63,27),-0.000000114274140],
   [(4,28),-0.000000114274140],
   [(8,28),0.000000575603073],
   [(9,28),0.000000575603073],
  [(10,28),-0.000000114274140],
  [(11,28),0.000000575603073],
  [(12,28),0.001069717680456],
  [(16,28),0.000000575603073],
  [(17,28),-0.000000114274140],
  [(18,28),0.000000022686778],
  [(19,28),-0.000000114274140],
  [(20,28),-0.000211817505560],
  [(21,28),0.000000575603073],
  [(22,28),-0.000000114274140],
  [(23,28),0.000000575603073],
  [(24,28),0.001069717680456],
  [(25,28),0.001069717680456],
  [(26,28),-0.000211817505560],
  [(27,28),0.001069717680456],
  [(28,28),0.994211542426410],
  [(29,28),0.000000575603073],
  [(30,28),-0.000000114274140],
  [(31,28),0.000000575603073],
  [(32,28),0.001069717680456],
  [(36,28),-0.000000114274140],
  [(40,28),0.000000575603073],
  [(41,28),0.000000575603073],
  [(42,28),-0.000000114274140],
  [(43,28),0.000000575603073],
  [(44,28),0.001069717680456],
  [(48,28),0.000000575603073],
  [(52,28),0.000000022686778],
  [(56,28),-0.000000114274140],
  [(57,28),-0.000000114274140],
  [(58,28),0.000000022686778],
  [(59,28),-0.000000114274140],
  [(60,28),-0.000211817505560],
  [(64,28),-0.000000114274140],
   [(1,29),0.000000575603073],
   [(5,29),-0.000000114274140],
   [(9,29),0.000000575603073],
  [(13,29),0.001069717680456],
  [(14,29),0.000000575603073],
  [(15,29),-0.000000114274140],
  [(16,29),0.000000575603073],
  [(17,29),0.001069717680456],
  [(18,29),0.000000575603073],
  [(19,29),-0.000000114274140],
  [(20,29),0.000000575603073],
  [(21,29),-0.000211817505560],
  [(22,29),-0.000000114274140],
  [(23,29),0.000000022686778],
  [(24,29),-0.000000114274140],
  [(25,29),0.001069717680456],
  [(26,29),0.000000575603073],
  [(27,29),-0.000000114274140],
  [(28,29),0.000000575603073],
  [(29,29),0.994211542426410],
  [(30,29),0.001069717680456],
  [(31,29),-0.000211817505560],
  [(32,29),0.001069717680456],
  [(33,29),0.000000575603073],
  [(37,29),-0.000000114274140],
  [(41,29),0.000000575603073],
  [(45,29),0.001069717680456],
  [(46,29),0.000000575603073],
  [(47,29),-0.000000114274140],
  [(48,29),0.000000575603073],
  [(49,29),-0.000000114274140],
  [(53,29),0.000000022686778],
  [(57,29),-0.000000114274140],
  [(61,29),-0.000211817505560],
  [(62,29),-0.000000114274140],
  [(63,29),0.000000022686778],
  [(64,29),-0.000000114274140],
   [(2,30),0.000000575603073],
   [(6,30),-0.000000114274140],
  [(10,30),0.000000575603073],
  [(13,30),0.000000575603073],
  [(14,30),0.001069717680456],
  [(15,30),0.000000575603073],
  [(16,30),-0.000000114274140],
  [(17,30),0.000000575603073],
  [(18,30),0.001069717680456],
  [(19,30),0.000000575603073],
  [(20,30),-0.000000114274140],
  [(21,30),-0.000000114274140],
  [(22,30),-0.000211817505560],
  [(23,30),-0.000000114274140],
  [(24,30),0.000000022686778],
  [(25,30),0.000000575603073],
  [(26,30),0.001069717680456],
  [(27,30),0.000000575603073],
  [(28,30),-0.000000114274140],
  [(29,30),0.001069717680456],
  [(30,30),0.994211542426410],
  [(31,30),0.001069717680456],
  [(32,30),-0.000211817505560],
  [(34,30),0.000000575603073],
  [(38,30),-0.000000114274140],
  [(42,30),0.000000575603073],
  [(45,30),0.000000575603073],
  [(46,30),0.001069717680456],
  [(47,30),0.000000575603073],
  [(48,30),-0.000000114274140],
  [(50,30),-0.000000114274140],
  [(54,30),0.000000022686778],
  [(58,30),-0.000000114274140],
  [(61,30),-0.000000114274140],
  [(62,30),-0.000211817505560],
  [(63,30),-0.000000114274140],
  [(64,30),0.000000022686778],
   [(3,31),0.000000575603073],
   [(7,31),-0.000000114274140],
  [(11,31),0.000000575603073],
  [(13,31),-0.000000114274140],
  [(14,31),0.000000575603073],
  [(15,31),0.001069717680456],
  [(16,31),0.000000575603073],
  [(17,31),-0.000000114274140],
  [(18,31),0.000000575603073],
  [(19,31),0.001069717680456],
  [(20,31),0.000000575603073],
  [(21,31),0.000000022686778],
  [(22,31),-0.000000114274140],
  [(23,31),-0.000211817505560],
  [(24,31),-0.000000114274140],
  [(25,31),-0.000000114274140],
  [(26,31),0.000000575603073],
  [(27,31),0.001069717680456],
  [(28,31),0.000000575603073],
  [(29,31),-0.000211817505560],
  [(30,31),0.001069717680456],
  [(31,31),0.994211542426410],
  [(32,31),0.001069717680456],
  [(35,31),0.000000575603073],
  [(39,31),-0.000000114274140],
  [(43,31),0.000000575603073],
  [(45,31),-0.000000114274140],
  [(46,31),0.000000575603073],
  [(47,31),0.001069717680456],
  [(48,31),0.000000575603073],
  [(51,31),-0.000000114274140],
  [(55,31),0.000000022686778],
  [(59,31),-0.000000114274140],
  [(61,31),0.000000022686778],
  [(62,31),-0.000000114274140],
  [(63,31),-0.000211817505560],
  [(64,31),-0.000000114274140],
   [(4,32),0.000000575603073],
   [(8,32),-0.000000114274140],
  [(12,32),0.000000575603073],
  [(13,32),0.000000575603073],
  [(14,32),-0.000000114274140],
  [(15,32),0.000000575603073],
  [(16,32),0.001069717680456],
  [(17,32),0.000000575603073],
  [(18,32),-0.000000114274140],
  [(19,32),0.000000575603073],
  [(20,32),0.001069717680456],
  [(21,32),-0.000000114274140],
  [(22,32),0.000000022686778],
  [(23,32),-0.000000114274140],
  [(24,32),-0.000211817505560],
  [(25,32),0.000000575603073],
  [(26,32),-0.000000114274140],
  [(27,32),0.000000575603073],
  [(28,32),0.001069717680456],
  [(29,32),0.001069717680456],
  [(30,32),-0.000211817505560],
  [(31,32),0.001069717680456],
  [(32,32),0.994211542426410],
  [(36,32),0.000000575603073],
  [(40,32),-0.000000114274140],
  [(44,32),0.000000575603073],
  [(45,32),0.000000575603073],
  [(46,32),-0.000000114274140],
  [(47,32),0.000000575603073],
  [(48,32),0.001069717680456],
  [(52,32),-0.000000114274140],
  [(56,32),0.000000022686778],
  [(60,32),-0.000000114274140],
  [(61,32),-0.000000114274140],
  [(62,32),0.000000022686778],
  [(63,32),-0.000000114274140],
  [(64,32),-0.000211817505560],
   [(1,33),-0.000211817505560],
   [(2,33),-0.000000114274140],
   [(3,33),0.000000022686778],
   [(4,33),-0.000000114274140],
   [(5,33),-0.000000114274140],
   [(9,33),0.000000022686778],
  [(13,33),-0.000000114274140],
  [(17,33),0.001069717680456],
  [(18,33),0.000000575603073],
  [(19,33),-0.000000114274140],
  [(20,33),0.000000575603073],
  [(21,33),0.000000575603073],
  [(25,33),-0.000000114274140],
  [(29,33),0.000000575603073],
  [(33,33),0.994211542426410],
  [(34,33),0.001069717680456],
  [(35,33),-0.000211817505560],
  [(36,33),0.001069717680456],
  [(37,33),0.001069717680456],
  [(38,33),0.000000575603073],
  [(39,33),-0.000000114274140],
  [(40,33),0.000000575603073],
  [(41,33),-0.000211817505560],
  [(42,33),-0.000000114274140],
  [(43,33),0.000000022686778],
  [(44,33),-0.000000114274140],
  [(45,33),0.001069717680456],
  [(46,33),0.000000575603073],
  [(47,33),-0.000000114274140],
  [(48,33),0.000000575603073],
  [(49,33),0.001069717680456],
  [(50,33),0.000000575603073],
  [(51,33),-0.000000114274140],
  [(52,33),0.000000575603073],
  [(53,33),0.000000575603073],
  [(57,33),-0.000000114274140],
  [(61,33),0.000000575603073],
   [(1,34),-0.000000114274140],
   [(2,34),-0.000211817505560],
   [(3,34),-0.000000114274140],
   [(4,34),0.000000022686778],
   [(6,34),-0.000000114274140],
  [(10,34),0.000000022686778],
  [(14,34),-0.000000114274140],
  [(17,34),0.000000575603073],
  [(18,34),0.001069717680456],
  [(19,34),0.000000575603073],
  [(20,34),-0.000000114274140],
  [(22,34),0.000000575603073],
  [(26,34),-0.000000114274140],
  [(30,34),0.000000575603073],
  [(33,34),0.001069717680456],
  [(34,34),0.994211542426410],
  [(35,34),0.001069717680456],
  [(36,34),-0.000211817505560],
  [(37,34),0.000000575603073],
  [(38,34),0.001069717680456],
  [(39,34),0.000000575603073],
  [(40,34),-0.000000114274140],
  [(41,34),-0.000000114274140],
  [(42,34),-0.000211817505560],
  [(43,34),-0.000000114274140],
  [(44,34),0.000000022686778],
  [(45,34),0.000000575603073],
  [(46,34),0.001069717680456],
  [(47,34),0.000000575603073],
  [(48,34),-0.000000114274140],
  [(49,34),0.000000575603073],
  [(50,34),0.001069717680456],
  [(51,34),0.000000575603073],
  [(52,34),-0.000000114274140],
  [(54,34),0.000000575603073],
  [(58,34),-0.000000114274140],
  [(62,34),0.000000575603073],
   [(1,35),0.000000022686778],
   [(2,35),-0.000000114274140],
   [(3,35),-0.000211817505560],
   [(4,35),-0.000000114274140],
   [(7,35),-0.000000114274140],
  [(11,35),0.000000022686778],
  [(15,35),-0.000000114274140],
  [(17,35),-0.000000114274140],
  [(18,35),0.000000575603073],
  [(19,35),0.001069717680456],
  [(20,35),0.000000575603073],
  [(23,35),0.000000575603073],
  [(27,35),-0.000000114274140],
  [(31,35),0.000000575603073],
  [(33,35),-0.000211817505560],
  [(34,35),0.001069717680456],
  [(35,35),0.994211542426410],
  [(36,35),0.001069717680456],
  [(37,35),-0.000000114274140],
  [(38,35),0.000000575603073],
  [(39,35),0.001069717680456],
  [(40,35),0.000000575603073],
  [(41,35),0.000000022686778],
  [(42,35),-0.000000114274140],
  [(43,35),-0.000211817505560],
  [(44,35),-0.000000114274140],
  [(45,35),-0.000000114274140],
  [(46,35),0.000000575603073],
  [(47,35),0.001069717680456],
  [(48,35),0.000000575603073],
  [(49,35),-0.000000114274140],
  [(50,35),0.000000575603073],
  [(51,35),0.001069717680456],
  [(52,35),0.000000575603073],
  [(55,35),0.000000575603073],
  [(59,35),-0.000000114274140],
  [(63,35),0.000000575603073],
   [(1,36),-0.000000114274140],
   [(2,36),0.000000022686778],
   [(3,36),-0.000000114274140],
   [(4,36),-0.000211817505560],
   [(8,36),-0.000000114274140],
  [(12,36),0.000000022686778],
  [(16,36),-0.000000114274140],
  [(17,36),0.000000575603073],
  [(18,36),-0.000000114274140],
  [(19,36),0.000000575603073],
  [(20,36),0.001069717680456],
  [(24,36),0.000000575603073],
  [(28,36),-0.000000114274140],
  [(32,36),0.000000575603073],
  [(33,36),0.001069717680456],
  [(34,36),-0.000211817505560],
  [(35,36),0.001069717680456],
  [(36,36),0.994211542426410],
  [(37,36),0.000000575603073],
  [(38,36),-0.000000114274140],
  [(39,36),0.000000575603073],
  [(40,36),0.001069717680456],
  [(41,36),-0.000000114274140],
  [(42,36),0.000000022686778],
  [(43,36),-0.000000114274140],
  [(44,36),-0.000211817505560],
  [(45,36),0.000000575603073],
  [(46,36),-0.000000114274140],
  [(47,36),0.000000575603073],
  [(48,36),0.001069717680456],
  [(49,36),0.000000575603073],
  [(50,36),-0.000000114274140],
  [(51,36),0.000000575603073],
  [(52,36),0.001069717680456],
  [(56,36),0.000000575603073],
  [(60,36),-0.000000114274140],
  [(64,36),0.000000575603073],
   [(1,37),-0.000000114274140],
   [(5,37),-0.000211817505560],
   [(6,37),-0.000000114274140],
   [(7,37),0.000000022686778],
   [(8,37),-0.000000114274140],
   [(9,37),-0.000000114274140],
  [(13,37),0.000000022686778],
  [(17,37),0.000000575603073],
  [(21,37),0.001069717680456],
  [(22,37),0.000000575603073],
  [(23,37),-0.000000114274140],
  [(24,37),0.000000575603073],
  [(25,37),0.000000575603073],
  [(29,37),-0.000000114274140],
  [(33,37),0.001069717680456],
  [(34,37),0.000000575603073],
  [(35,37),-0.000000114274140],
  [(36,37),0.000000575603073],
  [(37,37),0.994211542426410],
  [(38,37),0.001069717680456],
  [(39,37),-0.000211817505560],
  [(40,37),0.001069717680456],
  [(41,37),0.001069717680456],
  [(42,37),0.000000575603073],
  [(43,37),-0.000000114274140],
  [(44,37),0.000000575603073],
  [(45,37),-0.000211817505560],
  [(46,37),-0.000000114274140],
  [(47,37),0.000000022686778],
  [(48,37),-0.000000114274140],
  [(49,37),0.000000575603073],
  [(53,37),0.001069717680456],
  [(54,37),0.000000575603073],
  [(55,37),-0.000000114274140],
  [(56,37),0.000000575603073],
  [(57,37),0.000000575603073],
  [(61,37),-0.000000114274140],
   [(2,38),-0.000000114274140],
   [(5,38),-0.000000114274140],
   [(6,38),-0.000211817505560],
   [(7,38),-0.000000114274140],
   [(8,38),0.000000022686778],
  [(10,38),-0.000000114274140],
  [(14,38),0.000000022686778],
  [(18,38),0.000000575603073],
  [(21,38),0.000000575603073],
  [(22,38),0.001069717680456],
  [(23,38),0.000000575603073],
  [(24,38),-0.000000114274140],
  [(26,38),0.000000575603073],
  [(30,38),-0.000000114274140],
  [(33,38),0.000000575603073],
  [(34,38),0.001069717680456],
  [(35,38),0.000000575603073],
  [(36,38),-0.000000114274140],
  [(37,38),0.001069717680456],
  [(38,38),0.994211542426410],
  [(39,38),0.001069717680456],
  [(40,38),-0.000211817505560],
  [(41,38),0.000000575603073],
  [(42,38),0.001069717680456],
  [(43,38),0.000000575603073],
  [(44,38),-0.000000114274140],
  [(45,38),-0.000000114274140],
  [(46,38),-0.000211817505560],
  [(47,38),-0.000000114274140],
  [(48,38),0.000000022686778],
  [(50,38),0.000000575603073],
  [(53,38),0.000000575603073],
  [(54,38),0.001069717680456],
  [(55,38),0.000000575603073],
  [(56,38),-0.000000114274140],
  [(58,38),0.000000575603073],
  [(62,38),-0.000000114274140],
   [(3,39),-0.000000114274140],
   [(5,39),0.000000022686778],
   [(6,39),-0.000000114274140],
   [(7,39),-0.000211817505560],
   [(8,39),-0.000000114274140],
  [(11,39),-0.000000114274140],
  [(15,39),0.000000022686778],
  [(19,39),0.000000575603073],
  [(21,39),-0.000000114274140],
  [(22,39),0.000000575603073],
  [(23,39),0.001069717680456],
  [(24,39),0.000000575603073],
  [(27,39),0.000000575603073],
  [(31,39),-0.000000114274140],
  [(33,39),-0.000000114274140],
  [(34,39),0.000000575603073],
  [(35,39),0.001069717680456],
  [(36,39),0.000000575603073],
  [(37,39),-0.000211817505560],
  [(38,39),0.001069717680456],
  [(39,39),0.994211542426410],
  [(40,39),0.001069717680456],
  [(41,39),-0.000000114274140],
  [(42,39),0.000000575603073],
  [(43,39),0.001069717680456],
  [(44,39),0.000000575603073],
  [(45,39),0.000000022686778],
  [(46,39),-0.000000114274140],
  [(47,39),-0.000211817505560],
  [(48,39),-0.000000114274140],
  [(51,39),0.000000575603073],
  [(53,39),-0.000000114274140],
  [(54,39),0.000000575603073],
  [(55,39),0.001069717680456],
  [(56,39),0.000000575603073],
  [(59,39),0.000000575603073],
  [(63,39),-0.000000114274140],
   [(4,40),-0.000000114274140],
   [(5,40),-0.000000114274140],
   [(6,40),0.000000022686778],
   [(7,40),-0.000000114274140],
   [(8,40),-0.000211817505560],
  [(12,40),-0.000000114274140],
  [(16,40),0.000000022686778],
  [(20,40),0.000000575603073],
  [(21,40),0.000000575603073],
  [(22,40),-0.000000114274140],
  [(23,40),0.000000575603073],
  [(24,40),0.001069717680456],
  [(28,40),0.000000575603073],
  [(32,40),-0.000000114274140],
  [(33,40),0.000000575603073],
  [(34,40),-0.000000114274140],
  [(35,40),0.000000575603073],
  [(36,40),0.001069717680456],
  [(37,40),0.001069717680456],
  [(38,40),-0.000211817505560],
  [(39,40),0.001069717680456],
  [(40,40),0.994211542426410],
  [(41,40),0.000000575603073],
  [(42,40),-0.000000114274140],
  [(43,40),0.000000575603073],
  [(44,40),0.001069717680456],
  [(45,40),-0.000000114274140],
  [(46,40),0.000000022686778],
  [(47,40),-0.000000114274140],
  [(48,40),-0.000211817505560],
  [(52,40),0.000000575603073],
  [(53,40),0.000000575603073],
  [(54,40),-0.000000114274140],
  [(55,40),0.000000575603073],
  [(56,40),0.001069717680456],
  [(60,40),0.000000575603073],
  [(64,40),-0.000000114274140],
   [(1,41),0.000000022686778],
   [(5,41),-0.000000114274140],
   [(9,41),-0.000211817505560],
  [(10,41),-0.000000114274140],
  [(11,41),0.000000022686778],
  [(12,41),-0.000000114274140],
  [(13,41),-0.000000114274140],
  [(17,41),-0.000000114274140],
  [(21,41),0.000000575603073],
  [(25,41),0.001069717680456],
  [(26,41),0.000000575603073],
  [(27,41),-0.000000114274140],
  [(28,41),0.000000575603073],
  [(29,41),0.000000575603073],
  [(33,41),-0.000211817505560],
  [(34,41),-0.000000114274140],
  [(35,41),0.000000022686778],
  [(36,41),-0.000000114274140],
  [(37,41),0.001069717680456],
  [(38,41),0.000000575603073],
  [(39,41),-0.000000114274140],
  [(40,41),0.000000575603073],
  [(41,41),0.994211542426410],
  [(42,41),0.001069717680456],
  [(43,41),-0.000211817505560],
  [(44,41),0.001069717680456],
  [(45,41),0.001069717680456],
  [(46,41),0.000000575603073],
  [(47,41),-0.000000114274140],
  [(48,41),0.000000575603073],
  [(49,41),-0.000000114274140],
  [(53,41),0.000000575603073],
  [(57,41),0.001069717680456],
  [(58,41),0.000000575603073],
  [(59,41),-0.000000114274140],
  [(60,41),0.000000575603073],
  [(61,41),0.000000575603073],
   [(2,42),0.000000022686778],
   [(6,42),-0.000000114274140],
   [(9,42),-0.000000114274140],
  [(10,42),-0.000211817505560],
  [(11,42),-0.000000114274140],
  [(12,42),0.000000022686778],
  [(14,42),-0.000000114274140],
  [(18,42),-0.000000114274140],
  [(22,42),0.000000575603073],
  [(25,42),0.000000575603073],
  [(26,42),0.001069717680456],
  [(27,42),0.000000575603073],
  [(28,42),-0.000000114274140],
  [(30,42),0.000000575603073],
  [(33,42),-0.000000114274140],
  [(34,42),-0.000211817505560],
  [(35,42),-0.000000114274140],
  [(36,42),0.000000022686778],
  [(37,42),0.000000575603073],
  [(38,42),0.001069717680456],
  [(39,42),0.000000575603073],
  [(40,42),-0.000000114274140],
  [(41,42),0.001069717680456],
  [(42,42),0.994211542426410],
  [(43,42),0.001069717680456],
  [(44,42),-0.000211817505560],
  [(45,42),0.000000575603073],
  [(46,42),0.001069717680456],
  [(47,42),0.000000575603073],
  [(48,42),-0.000000114274140],
  [(50,42),-0.000000114274140],
  [(54,42),0.000000575603073],
  [(57,42),0.000000575603073],
  [(58,42),0.001069717680456],
  [(59,42),0.000000575603073],
  [(60,42),-0.000000114274140],
  [(62,42),0.000000575603073],
   [(3,43),0.000000022686778],
   [(7,43),-0.000000114274140],
   [(9,43),0.000000022686778],
  [(10,43),-0.000000114274140],
  [(11,43),-0.000211817505560],
  [(12,43),-0.000000114274140],
  [(15,43),-0.000000114274140],
  [(19,43),-0.000000114274140],
  [(23,43),0.000000575603073],
  [(25,43),-0.000000114274140],
  [(26,43),0.000000575603073],
  [(27,43),0.001069717680456],
  [(28,43),0.000000575603073],
  [(31,43),0.000000575603073],
  [(33,43),0.000000022686778],
  [(34,43),-0.000000114274140],
  [(35,43),-0.000211817505560],
  [(36,43),-0.000000114274140],
  [(37,43),-0.000000114274140],
  [(38,43),0.000000575603073],
  [(39,43),0.001069717680456],
  [(40,43),0.000000575603073],
  [(41,43),-0.000211817505560],
  [(42,43),0.001069717680456],
  [(43,43),0.994211542426410],
  [(44,43),0.001069717680456],
  [(45,43),-0.000000114274140],
  [(46,43),0.000000575603073],
  [(47,43),0.001069717680456],
  [(48,43),0.000000575603073],
  [(51,43),-0.000000114274140],
  [(55,43),0.000000575603073],
  [(57,43),-0.000000114274140],
  [(58,43),0.000000575603073],
  [(59,43),0.001069717680456],
  [(60,43),0.000000575603073],
  [(63,43),0.000000575603073],
   [(4,44),0.000000022686778],
   [(8,44),-0.000000114274140],
   [(9,44),-0.000000114274140],
  [(10,44),0.000000022686778],
  [(11,44),-0.000000114274140],
  [(12,44),-0.000211817505560],
  [(16,44),-0.000000114274140],
  [(20,44),-0.000000114274140],
  [(24,44),0.000000575603073],
  [(25,44),0.000000575603073],
  [(26,44),-0.000000114274140],
  [(27,44),0.000000575603073],
  [(28,44),0.001069717680456],
  [(32,44),0.000000575603073],
  [(33,44),-0.000000114274140],
  [(34,44),0.000000022686778],
  [(35,44),-0.000000114274140],
  [(36,44),-0.000211817505560],
  [(37,44),0.000000575603073],
  [(38,44),-0.000000114274140],
  [(39,44),0.000000575603073],
  [(40,44),0.001069717680456],
  [(41,44),0.001069717680456],
  [(42,44),-0.000211817505560],
  [(43,44),0.001069717680456],
  [(44,44),0.994211542426410],
  [(45,44),0.000000575603073],
  [(46,44),-0.000000114274140],
  [(47,44),0.000000575603073],
  [(48,44),0.001069717680456],
  [(52,44),-0.000000114274140],
  [(56,44),0.000000575603073],
  [(57,44),0.000000575603073],
  [(58,44),-0.000000114274140],
  [(59,44),0.000000575603073],
  [(60,44),0.001069717680456],
  [(64,44),0.000000575603073],
   [(1,45),-0.000000114274140],
   [(5,45),0.000000022686778],
   [(9,45),-0.000000114274140],
  [(13,45),-0.000211817505560],
  [(14,45),-0.000000114274140],
  [(15,45),0.000000022686778],
  [(16,45),-0.000000114274140],
  [(17,45),0.000000575603073],
  [(21,45),-0.000000114274140],
  [(25,45),0.000000575603073],
  [(29,45),0.001069717680456],
  [(30,45),0.000000575603073],
  [(31,45),-0.000000114274140],
  [(32,45),0.000000575603073],
  [(33,45),0.001069717680456],
  [(34,45),0.000000575603073],
  [(35,45),-0.000000114274140],
  [(36,45),0.000000575603073],
  [(37,45),-0.000211817505560],
  [(38,45),-0.000000114274140],
  [(39,45),0.000000022686778],
  [(40,45),-0.000000114274140],
  [(41,45),0.001069717680456],
  [(42,45),0.000000575603073],
  [(43,45),-0.000000114274140],
  [(44,45),0.000000575603073],
  [(45,45),0.994211542426410],
  [(46,45),0.001069717680456],
  [(47,45),-0.000211817505560],
  [(48,45),0.001069717680456],
  [(49,45),0.000000575603073],
  [(53,45),-0.000000114274140],
  [(57,45),0.000000575603073],
  [(61,45),0.001069717680456],
  [(62,45),0.000000575603073],
  [(63,45),-0.000000114274140],
  [(64,45),0.000000575603073],
   [(2,46),-0.000000114274140],
   [(6,46),0.000000022686778],
  [(10,46),-0.000000114274140],
  [(13,46),-0.000000114274140],
  [(14,46),-0.000211817505560],
  [(15,46),-0.000000114274140],
  [(16,46),0.000000022686778],
  [(18,46),0.000000575603073],
  [(22,46),-0.000000114274140],
  [(26,46),0.000000575603073],
  [(29,46),0.000000575603073],
  [(30,46),0.001069717680456],
  [(31,46),0.000000575603073],
  [(32,46),-0.000000114274140],
  [(33,46),0.000000575603073],
  [(34,46),0.001069717680456],
  [(35,46),0.000000575603073],
  [(36,46),-0.000000114274140],
  [(37,46),-0.000000114274140],
  [(38,46),-0.000211817505560],
  [(39,46),-0.000000114274140],
  [(40,46),0.000000022686778],
  [(41,46),0.000000575603073],
  [(42,46),0.001069717680456],
  [(43,46),0.000000575603073],
  [(44,46),-0.000000114274140],
  [(45,46),0.001069717680456],
  [(46,46),0.994211542426410],
  [(47,46),0.001069717680456],
  [(48,46),-0.000211817505560],
  [(50,46),0.000000575603073],
  [(54,46),-0.000000114274140],
  [(58,46),0.000000575603073],
  [(61,46),0.000000575603073],
  [(62,46),0.001069717680456],
  [(63,46),0.000000575603073],
  [(64,46),-0.000000114274140],
   [(3,47),-0.000000114274140],
   [(7,47),0.000000022686778],
  [(11,47),-0.000000114274140],
  [(13,47),0.000000022686778],
  [(14,47),-0.000000114274140],
  [(15,47),-0.000211817505560],
  [(16,47),-0.000000114274140],
  [(19,47),0.000000575603073],
  [(23,47),-0.000000114274140],
  [(27,47),0.000000575603073],
  [(29,47),-0.000000114274140],
  [(30,47),0.000000575603073],
  [(31,47),0.001069717680456],
  [(32,47),0.000000575603073],
  [(33,47),-0.000000114274140],
  [(34,47),0.000000575603073],
  [(35,47),0.001069717680456],
  [(36,47),0.000000575603073],
  [(37,47),0.000000022686778],
  [(38,47),-0.000000114274140],
  [(39,47),-0.000211817505560],
  [(40,47),-0.000000114274140],
  [(41,47),-0.000000114274140],
  [(42,47),0.000000575603073],
  [(43,47),0.001069717680456],
  [(44,47),0.000000575603073],
  [(45,47),-0.000211817505560],
  [(46,47),0.001069717680456],
  [(47,47),0.994211542426410],
  [(48,47),0.001069717680456],
  [(51,47),0.000000575603073],
  [(55,47),-0.000000114274140],
  [(59,47),0.000000575603073],
  [(61,47),-0.000000114274140],
  [(62,47),0.000000575603073],
  [(63,47),0.001069717680456],
  [(64,47),0.000000575603073],
   [(4,48),-0.000000114274140],
   [(8,48),0.000000022686778],
  [(12,48),-0.000000114274140],
  [(13,48),-0.000000114274140],
  [(14,48),0.000000022686778],
  [(15,48),-0.000000114274140],
  [(16,48),-0.000211817505560],
  [(20,48),0.000000575603073],
  [(24,48),-0.000000114274140],
  [(28,48),0.000000575603073],
  [(29,48),0.000000575603073],
  [(30,48),-0.000000114274140],
  [(31,48),0.000000575603073],
  [(32,48),0.001069717680456],
  [(33,48),0.000000575603073],
  [(34,48),-0.000000114274140],
  [(35,48),0.000000575603073],
  [(36,48),0.001069717680456],
  [(37,48),-0.000000114274140],
  [(38,48),0.000000022686778],
  [(39,48),-0.000000114274140],
  [(40,48),-0.000211817505560],
  [(41,48),0.000000575603073],
  [(42,48),-0.000000114274140],
  [(43,48),0.000000575603073],
  [(44,48),0.001069717680456],
  [(45,48),0.001069717680456],
  [(46,48),-0.000211817505560],
  [(47,48),0.001069717680456],
  [(48,48),0.994211542426410],
  [(52,48),0.000000575603073],
  [(56,48),-0.000000114274140],
  [(60,48),0.000000575603073],
  [(61,48),0.000000575603073],
  [(62,48),-0.000000114274140],
  [(63,48),0.000000575603073],
  [(64,48),0.001069717680456],
   [(1,49),0.001069717680456],
   [(2,49),0.000000575603073],
   [(3,49),-0.000000114274140],
   [(4,49),0.000000575603073],
   [(5,49),0.000000575603073],
   [(9,49),-0.000000114274140],
  [(13,49),0.000000575603073],
  [(17,49),-0.000211817505560],
  [(18,49),-0.000000114274140],
  [(19,49),0.000000022686778],
  [(20,49),-0.000000114274140],
  [(21,49),-0.000000114274140],
  [(25,49),0.000000022686778],
  [(29,49),-0.000000114274140],
  [(33,49),0.001069717680456],
  [(34,49),0.000000575603073],
  [(35,49),-0.000000114274140],
  [(36,49),0.000000575603073],
  [(37,49),0.000000575603073],
  [(41,49),-0.000000114274140],
  [(45,49),0.000000575603073],
  [(49,49),0.994211542426410],
  [(50,49),0.001069717680456],
  [(51,49),-0.000211817505560],
  [(52,49),0.001069717680456],
  [(53,49),0.001069717680456],
  [(54,49),0.000000575603073],
  [(55,49),-0.000000114274140],
  [(56,49),0.000000575603073],
  [(57,49),-0.000211817505560],
  [(58,49),-0.000000114274140],
  [(59,49),0.000000022686778],
  [(60,49),-0.000000114274140],
  [(61,49),0.001069717680456],
  [(62,49),0.000000575603073],
  [(63,49),-0.000000114274140],
  [(64,49),0.000000575603073],
   [(1,50),0.000000575603073],
   [(2,50),0.001069717680456],
   [(3,50),0.000000575603073],
   [(4,50),-0.000000114274140],
   [(6,50),0.000000575603073],
  [(10,50),-0.000000114274140],
  [(14,50),0.000000575603073],
  [(17,50),-0.000000114274140],
  [(18,50),-0.000211817505560],
  [(19,50),-0.000000114274140],
  [(20,50),0.000000022686778],
  [(22,50),-0.000000114274140],
  [(26,50),0.000000022686778],
  [(30,50),-0.000000114274140],
  [(33,50),0.000000575603073],
  [(34,50),0.001069717680456],
  [(35,50),0.000000575603073],
  [(36,50),-0.000000114274140],
  [(38,50),0.000000575603073],
  [(42,50),-0.000000114274140],
  [(46,50),0.000000575603073],
  [(49,50),0.001069717680456],
  [(50,50),0.994211542426410],
  [(51,50),0.001069717680456],
  [(52,50),-0.000211817505560],
  [(53,50),0.000000575603073],
  [(54,50),0.001069717680456],
  [(55,50),0.000000575603073],
  [(56,50),-0.000000114274140],
  [(57,50),-0.000000114274140],
  [(58,50),-0.000211817505560],
  [(59,50),-0.000000114274140],
  [(60,50),0.000000022686778],
  [(61,50),0.000000575603073],
  [(62,50),0.001069717680456],
  [(63,50),0.000000575603073],
  [(64,50),-0.000000114274140],
   [(1,51),-0.000000114274140],
   [(2,51),0.000000575603073],
   [(3,51),0.001069717680456],
   [(4,51),0.000000575603073],
   [(7,51),0.000000575603073],
  [(11,51),-0.000000114274140],
  [(15,51),0.000000575603073],
  [(17,51),0.000000022686778],
  [(18,51),-0.000000114274140],
  [(19,51),-0.000211817505560],
  [(20,51),-0.000000114274140],
  [(23,51),-0.000000114274140],
  [(27,51),0.000000022686778],
  [(31,51),-0.000000114274140],
  [(33,51),-0.000000114274140],
  [(34,51),0.000000575603073],
  [(35,51),0.001069717680456],
  [(36,51),0.000000575603073],
  [(39,51),0.000000575603073],
  [(43,51),-0.000000114274140],
  [(47,51),0.000000575603073],
  [(49,51),-0.000211817505560],
  [(50,51),0.001069717680456],
  [(51,51),0.994211542426410],
  [(52,51),0.001069717680456],
  [(53,51),-0.000000114274140],
  [(54,51),0.000000575603073],
  [(55,51),0.001069717680456],
  [(56,51),0.000000575603073],
  [(57,51),0.000000022686778],
  [(58,51),-0.000000114274140],
  [(59,51),-0.000211817505560],
  [(60,51),-0.000000114274140],
  [(61,51),-0.000000114274140],
  [(62,51),0.000000575603073],
  [(63,51),0.001069717680456],
  [(64,51),0.000000575603073],
   [(1,52),0.000000575603073],
   [(2,52),-0.000000114274140],
   [(3,52),0.000000575603073],
   [(4,52),0.001069717680456],
   [(8,52),0.000000575603073],
  [(12,52),-0.000000114274140],
  [(16,52),0.000000575603073],
  [(17,52),-0.000000114274140],
  [(18,52),0.000000022686778],
  [(19,52),-0.000000114274140],
  [(20,52),-0.000211817505560],
  [(24,52),-0.000000114274140],
  [(28,52),0.000000022686778],
  [(32,52),-0.000000114274140],
  [(33,52),0.000000575603073],
  [(34,52),-0.000000114274140],
  [(35,52),0.000000575603073],
  [(36,52),0.001069717680456],
  [(40,52),0.000000575603073],
  [(44,52),-0.000000114274140],
  [(48,52),0.000000575603073],
  [(49,52),0.001069717680456],
  [(50,52),-0.000211817505560],
  [(51,52),0.001069717680456],
  [(52,52),0.994211542426410],
  [(53,52),0.000000575603073],
  [(54,52),-0.000000114274140],
  [(55,52),0.000000575603073],
  [(56,52),0.001069717680456],
  [(57,52),-0.000000114274140],
  [(58,52),0.000000022686778],
  [(59,52),-0.000000114274140],
  [(60,52),-0.000211817505560],
  [(61,52),0.000000575603073],
  [(62,52),-0.000000114274140],
  [(63,52),0.000000575603073],
  [(64,52),0.001069717680456],
   [(1,53),0.000000575603073],
   [(5,53),0.001069717680456],
   [(6,53),0.000000575603073],
   [(7,53),-0.000000114274140],
   [(8,53),0.000000575603073],
   [(9,53),0.000000575603073],
  [(13,53),-0.000000114274140],
  [(17,53),-0.000000114274140],
  [(21,53),-0.000211817505560],
  [(22,53),-0.000000114274140],
  [(23,53),0.000000022686778],
  [(24,53),-0.000000114274140],
  [(25,53),-0.000000114274140],
  [(29,53),0.000000022686778],
  [(33,53),0.000000575603073],
  [(37,53),0.001069717680456],
  [(38,53),0.000000575603073],
  [(39,53),-0.000000114274140],
  [(40,53),0.000000575603073],
  [(41,53),0.000000575603073],
  [(45,53),-0.000000114274140],
  [(49,53),0.001069717680456],
  [(50,53),0.000000575603073],
  [(51,53),-0.000000114274140],
  [(52,53),0.000000575603073],
  [(53,53),0.994211542426410],
  [(54,53),0.001069717680456],
  [(55,53),-0.000211817505560],
  [(56,53),0.001069717680456],
  [(57,53),0.001069717680456],
  [(58,53),0.000000575603073],
  [(59,53),-0.000000114274140],
  [(60,53),0.000000575603073],
  [(61,53),-0.000211817505560],
  [(62,53),-0.000000114274140],
  [(63,53),0.000000022686778],
  [(64,53),-0.000000114274140],
   [(2,54),0.000000575603073],
   [(5,54),0.000000575603073],
   [(6,54),0.001069717680456],
   [(7,54),0.000000575603073],
   [(8,54),-0.000000114274140],
  [(10,54),0.000000575603073],
  [(14,54),-0.000000114274140],
  [(18,54),-0.000000114274140],
  [(21,54),-0.000000114274140],
  [(22,54),-0.000211817505560],
  [(23,54),-0.000000114274140],
  [(24,54),0.000000022686778],
  [(26,54),-0.000000114274140],
  [(30,54),0.000000022686778],
  [(34,54),0.000000575603073],
  [(37,54),0.000000575603073],
  [(38,54),0.001069717680456],
  [(39,54),0.000000575603073],
  [(40,54),-0.000000114274140],
  [(42,54),0.000000575603073],
  [(46,54),-0.000000114274140],
  [(49,54),0.000000575603073],
  [(50,54),0.001069717680456],
  [(51,54),0.000000575603073],
  [(52,54),-0.000000114274140],
  [(53,54),0.001069717680456],
  [(54,54),0.994211542426410],
  [(55,54),0.001069717680456],
  [(56,54),-0.000211817505560],
  [(57,54),0.000000575603073],
  [(58,54),0.001069717680456],
  [(59,54),0.000000575603073],
  [(60,54),-0.000000114274140],
  [(61,54),-0.000000114274140],
  [(62,54),-0.000211817505560],
  [(63,54),-0.000000114274140],
  [(64,54),0.000000022686778],
   [(3,55),0.000000575603073],
   [(5,55),-0.000000114274140],
   [(6,55),0.000000575603073],
   [(7,55),0.001069717680456],
   [(8,55),0.000000575603073],
  [(11,55),0.000000575603073],
  [(15,55),-0.000000114274140],
  [(19,55),-0.000000114274140],
  [(21,55),0.000000022686778],
  [(22,55),-0.000000114274140],
  [(23,55),-0.000211817505560],
  [(24,55),-0.000000114274140],
  [(27,55),-0.000000114274140],
  [(31,55),0.000000022686778],
  [(35,55),0.000000575603073],
  [(37,55),-0.000000114274140],
  [(38,55),0.000000575603073],
  [(39,55),0.001069717680456],
  [(40,55),0.000000575603073],
  [(43,55),0.000000575603073],
  [(47,55),-0.000000114274140],
  [(49,55),-0.000000114274140],
  [(50,55),0.000000575603073],
  [(51,55),0.001069717680456],
  [(52,55),0.000000575603073],
  [(53,55),-0.000211817505560],
  [(54,55),0.001069717680456],
  [(55,55),0.994211542426410],
  [(56,55),0.001069717680456],
  [(57,55),-0.000000114274140],
  [(58,55),0.000000575603073],
  [(59,55),0.001069717680456],
  [(60,55),0.000000575603073],
  [(61,55),0.000000022686778],
  [(62,55),-0.000000114274140],
  [(63,55),-0.000211817505560],
  [(64,55),-0.000000114274140],
   [(4,56),0.000000575603073],
   [(5,56),0.000000575603073],
   [(6,56),-0.000000114274140],
   [(7,56),0.000000575603073],
   [(8,56),0.001069717680456],
  [(12,56),0.000000575603073],
  [(16,56),-0.000000114274140],
  [(20,56),-0.000000114274140],
  [(21,56),-0.000000114274140],
  [(22,56),0.000000022686778],
  [(23,56),-0.000000114274140],
  [(24,56),-0.000211817505560],
  [(28,56),-0.000000114274140],
  [(32,56),0.000000022686778],
  [(36,56),0.000000575603073],
  [(37,56),0.000000575603073],
  [(38,56),-0.000000114274140],
  [(39,56),0.000000575603073],
  [(40,56),0.001069717680456],
  [(44,56),0.000000575603073],
  [(48,56),-0.000000114274140],
  [(49,56),0.000000575603073],
  [(50,56),-0.000000114274140],
  [(51,56),0.000000575603073],
  [(52,56),0.001069717680456],
  [(53,56),0.001069717680456],
  [(54,56),-0.000211817505560],
  [(55,56),0.001069717680456],
  [(56,56),0.994211542426410],
  [(57,56),0.000000575603073],
  [(58,56),-0.000000114274140],
  [(59,56),0.000000575603073],
  [(60,56),0.001069717680456],
  [(61,56),-0.000000114274140],
  [(62,56),0.000000022686778],
  [(63,56),-0.000000114274140],
  [(64,56),-0.000211817505560],
   [(1,57),-0.000000114274140],
   [(5,57),0.000000575603073],
   [(9,57),0.001069717680456],
  [(10,57),0.000000575603073],
  [(11,57),-0.000000114274140],
  [(12,57),0.000000575603073],
  [(13,57),0.000000575603073],
  [(17,57),0.000000022686778],
  [(21,57),-0.000000114274140],
  [(25,57),-0.000211817505560],
  [(26,57),-0.000000114274140],
  [(27,57),0.000000022686778],
  [(28,57),-0.000000114274140],
  [(29,57),-0.000000114274140],
  [(33,57),-0.000000114274140],
  [(37,57),0.000000575603073],
  [(41,57),0.001069717680456],
  [(42,57),0.000000575603073],
  [(43,57),-0.000000114274140],
  [(44,57),0.000000575603073],
  [(45,57),0.000000575603073],
  [(49,57),-0.000211817505560],
  [(50,57),-0.000000114274140],
  [(51,57),0.000000022686778],
  [(52,57),-0.000000114274140],
  [(53,57),0.001069717680456],
  [(54,57),0.000000575603073],
  [(55,57),-0.000000114274140],
  [(56,57),0.000000575603073],
  [(57,57),0.994211542426410],
  [(58,57),0.001069717680456],
  [(59,57),-0.000211817505560],
  [(60,57),0.001069717680456],
  [(61,57),0.001069717680456],
  [(62,57),0.000000575603073],
  [(63,57),-0.000000114274140],
  [(64,57),0.000000575603073],
   [(2,58),-0.000000114274140],
   [(6,58),0.000000575603073],
   [(9,58),0.000000575603073],
  [(10,58),0.001069717680456],
  [(11,58),0.000000575603073],
  [(12,58),-0.000000114274140],
  [(14,58),0.000000575603073],
  [(18,58),0.000000022686778],
  [(22,58),-0.000000114274140],
  [(25,58),-0.000000114274140],
  [(26,58),-0.000211817505560],
  [(27,58),-0.000000114274140],
  [(28,58),0.000000022686778],
  [(30,58),-0.000000114274140],
  [(34,58),-0.000000114274140],
  [(38,58),0.000000575603073],
  [(41,58),0.000000575603073],
  [(42,58),0.001069717680456],
  [(43,58),0.000000575603073],
  [(44,58),-0.000000114274140],
  [(46,58),0.000000575603073],
  [(49,58),-0.000000114274140],
  [(50,58),-0.000211817505560],
  [(51,58),-0.000000114274140],
  [(52,58),0.000000022686778],
  [(53,58),0.000000575603073],
  [(54,58),0.001069717680456],
  [(55,58),0.000000575603073],
  [(56,58),-0.000000114274140],
  [(57,58),0.001069717680456],
  [(58,58),0.994211542426410],
  [(59,58),0.001069717680456],
  [(60,58),-0.000211817505560],
  [(61,58),0.000000575603073],
  [(62,58),0.001069717680456],
  [(63,58),0.000000575603073],
  [(64,58),-0.000000114274140],
   [(3,59),-0.000000114274140],
   [(7,59),0.000000575603073],
   [(9,59),-0.000000114274140],
  [(10,59),0.000000575603073],
  [(11,59),0.001069717680456],
  [(12,59),0.000000575603073],
  [(15,59),0.000000575603073],
  [(19,59),0.000000022686778],
  [(23,59),-0.000000114274140],
  [(25,59),0.000000022686778],
  [(26,59),-0.000000114274140],
  [(27,59),-0.000211817505560],
  [(28,59),-0.000000114274140],
  [(31,59),-0.000000114274140],
  [(35,59),-0.000000114274140],
  [(39,59),0.000000575603073],
  [(41,59),-0.000000114274140],
  [(42,59),0.000000575603073],
  [(43,59),0.001069717680456],
  [(44,59),0.000000575603073],
  [(47,59),0.000000575603073],
  [(49,59),0.000000022686778],
  [(50,59),-0.000000114274140],
  [(51,59),-0.000211817505560],
  [(52,59),-0.000000114274140],
  [(53,59),-0.000000114274140],
  [(54,59),0.000000575603073],
  [(55,59),0.001069717680456],
  [(56,59),0.000000575603073],
  [(57,59),-0.000211817505560],
  [(58,59),0.001069717680456],
  [(59,59),0.994211542426410],
  [(60,59),0.001069717680456],
  [(61,59),-0.000000114274140],
  [(62,59),0.000000575603073],
  [(63,59),0.001069717680456],
  [(64,59),0.000000575603073],
   [(4,60),-0.000000114274140],
   [(8,60),0.000000575603073],
   [(9,60),0.000000575603073],
  [(10,60),-0.000000114274140],
  [(11,60),0.000000575603073],
  [(12,60),0.001069717680456],
  [(16,60),0.000000575603073],
  [(20,60),0.000000022686778],
  [(24,60),-0.000000114274140],
  [(25,60),-0.000000114274140],
  [(26,60),0.000000022686778],
  [(27,60),-0.000000114274140],
  [(28,60),-0.000211817505560],
  [(32,60),-0.000000114274140],
  [(36,60),-0.000000114274140],
  [(40,60),0.000000575603073],
  [(41,60),0.000000575603073],
  [(42,60),-0.000000114274140],
  [(43,60),0.000000575603073],
  [(44,60),0.001069717680456],
  [(48,60),0.000000575603073],
  [(49,60),-0.000000114274140],
  [(50,60),0.000000022686778],
  [(51,60),-0.000000114274140],
  [(52,60),-0.000211817505560],
  [(53,60),0.000000575603073],
  [(54,60),-0.000000114274140],
  [(55,60),0.000000575603073],
  [(56,60),0.001069717680456],
  [(57,60),0.001069717680456],
  [(58,60),-0.000211817505560],
  [(59,60),0.001069717680456],
  [(60,60),0.994211542426410],
  [(61,60),0.000000575603073],
  [(62,60),-0.000000114274140],
  [(63,60),0.000000575603073],
  [(64,60),0.001069717680456],
   [(1,61),0.000000575603073],
   [(5,61),-0.000000114274140],
   [(9,61),0.000000575603073],
  [(13,61),0.001069717680456],
  [(14,61),0.000000575603073],
  [(15,61),-0.000000114274140],
  [(16,61),0.000000575603073],
  [(17,61),-0.000000114274140],
  [(21,61),0.000000022686778],
  [(25,61),-0.000000114274140],
  [(29,61),-0.000211817505560],
  [(30,61),-0.000000114274140],
  [(31,61),0.000000022686778],
  [(32,61),-0.000000114274140],
  [(33,61),0.000000575603073],
  [(37,61),-0.000000114274140],
  [(41,61),0.000000575603073],
  [(45,61),0.001069717680456],
  [(46,61),0.000000575603073],
  [(47,61),-0.000000114274140],
  [(48,61),0.000000575603073],
  [(49,61),0.001069717680456],
  [(50,61),0.000000575603073],
  [(51,61),-0.000000114274140],
  [(52,61),0.000000575603073],
  [(53,61),-0.000211817505560],
  [(54,61),-0.000000114274140],
  [(55,61),0.000000022686778],
  [(56,61),-0.000000114274140],
  [(57,61),0.001069717680456],
  [(58,61),0.000000575603073],
  [(59,61),-0.000000114274140],
  [(60,61),0.000000575603073],
  [(61,61),0.994211542426410],
  [(62,61),0.001069717680456],
  [(63,61),-0.000211817505560],
  [(64,61),0.001069717680456],
   [(2,62),0.000000575603073],
   [(6,62),-0.000000114274140],
  [(10,62),0.000000575603073],
  [(13,62),0.000000575603073],
  [(14,62),0.001069717680456],
  [(15,62),0.000000575603073],
  [(16,62),-0.000000114274140],
  [(18,62),-0.000000114274140],
  [(22,62),0.000000022686778],
  [(26,62),-0.000000114274140],
  [(29,62),-0.000000114274140],
  [(30,62),-0.000211817505560],
  [(31,62),-0.000000114274140],
  [(32,62),0.000000022686778],
  [(34,62),0.000000575603073],
  [(38,62),-0.000000114274140],
  [(42,62),0.000000575603073],
  [(45,62),0.000000575603073],
  [(46,62),0.001069717680456],
  [(47,62),0.000000575603073],
  [(48,62),-0.000000114274140],
  [(49,62),0.000000575603073],
  [(50,62),0.001069717680456],
  [(51,62),0.000000575603073],
  [(52,62),-0.000000114274140],
  [(53,62),-0.000000114274140],
  [(54,62),-0.000211817505560],
  [(55,62),-0.000000114274140],
  [(56,62),0.000000022686778],
  [(57,62),0.000000575603073],
  [(58,62),0.001069717680456],
  [(59,62),0.000000575603073],
  [(60,62),-0.000000114274140],
  [(61,62),0.001069717680456],
  [(62,62),0.994211542426410],
  [(63,62),0.001069717680456],
  [(64,62),-0.000211817505560],
   [(3,63),0.000000575603073],
   [(7,63),-0.000000114274140],
  [(11,63),0.000000575603073],
  [(13,63),-0.000000114274140],
  [(14,63),0.000000575603073],
  [(15,63),0.001069717680456],
  [(16,63),0.000000575603073],
  [(19,63),-0.000000114274140],
  [(23,63),0.000000022686778],
  [(27,63),-0.000000114274140],
  [(29,63),0.000000022686778],
  [(30,63),-0.000000114274140],
  [(31,63),-0.000211817505560],
  [(32,63),-0.000000114274140],
  [(35,63),0.000000575603073],
  [(39,63),-0.000000114274140],
  [(43,63),0.000000575603073],
  [(45,63),-0.000000114274140],
  [(46,63),0.000000575603073],
  [(47,63),0.001069717680456],
  [(48,63),0.000000575603073],
  [(49,63),-0.000000114274140],
  [(50,63),0.000000575603073],
  [(51,63),0.001069717680456],
  [(52,63),0.000000575603073],
  [(53,63),0.000000022686778],
  [(54,63),-0.000000114274140],
  [(55,63),-0.000211817505560],
  [(56,63),-0.000000114274140],
  [(57,63),-0.000000114274140],
  [(58,63),0.000000575603073],
  [(59,63),0.001069717680456],
  [(60,63),0.000000575603073],
  [(61,63),-0.000211817505560],
  [(62,63),0.001069717680456],
  [(63,63),0.994211542426410],
  [(64,63),0.001069717680456],
   [(4,64),0.000000575603073],
   [(8,64),-0.000000114274140],
  [(12,64),0.000000575603073],
  [(13,64),0.000000575603073],
  [(14,64),-0.000000114274140],
  [(15,64),0.000000575603073],
  [(16,64),0.001069717680456],
  [(20,64),-0.000000114274140],
  [(24,64),0.000000022686778],
  [(28,64),-0.000000114274140],
  [(29,64),-0.000000114274140],
  [(30,64),0.000000022686778],
  [(31,64),-0.000000114274140],
  [(32,64),-0.000211817505560],
  [(36,64),0.000000575603073],
  [(40,64),-0.000000114274140],
  [(44,64),0.000000575603073],
  [(45,64),0.000000575603073],
  [(46,64),-0.000000114274140],
  [(47,64),0.000000575603073],
  [(48,64),0.001069717680456],
  [(49,64),0.000000575603073],
  [(50,64),-0.000000114274140],
  [(51,64),0.000000575603073],
  [(52,64),0.001069717680456],
  [(53,64),-0.000000114274140],
  [(54,64),0.000000022686778],
  [(55,64),-0.000000114274140],
  [(56,64),-0.000211817505560],
  [(57,64),0.000000575603073],
  [(58,64),-0.000000114274140],
  [(59,64),0.000000575603073],
  [(60,64),0.001069717680456],
  [(61,64),0.001069717680456],
  [(62,64),-0.000211817505560],
  [(63,64),0.001069717680456],
  [(64,64),0.994211542426410]]

In [43]:
testL4KinPion = [[(1,1),   0.994198340820593],
   [(2,1),   0.001069717680456],
   [(3,1),  -0.000211817505560],
   [(4,1),   0.001069717680456],
   [(5,1),   0.001069717680456],
   [(6,1),   0.000000575603073],
   [(7,1),  -0.000000114274140],
   [(8,1),   0.000000575603073],
   [(9,1),  -0.000211817505560],
  [(10,1),  -0.000000114274140],
  [(11,1),   0.000000022686778],
  [(12,1),  -0.000000114274140],
  [(13,1),   0.001069717680456],
  [(14,1),   0.000000575603073],
  [(15,1),  -0.000000114274140],
  [(16,1),   0.000000575603073],
  [(17,1),   0.001069717680456],
  [(18,1),   0.000000575603073],
  [(19,1),  -0.000000114274140],
  [(20,1),   0.000000575603073],
  [(21,1),   0.000000575603073],
  [(25,1),  -0.000000114274140],
  [(29,1),   0.000000575603073],
  [(33,1),  -0.000211817505560],
  [(34,1),  -0.000000114274140],
  [(35,1),   0.000000022686778],
  [(36,1),  -0.000000114274140],
  [(37,1),  -0.000000114274140],
  [(41,1),   0.000000022686778],
  [(45,1),  -0.000000114274140],
  [(49,1),   0.001069717680456],
  [(50,1),   0.000000575603073],
  [(51,1),  -0.000000114274140],
  [(52,1),   0.000000575603073],
  [(53,1),   0.000000575603073],
  [(57,1),  -0.000000114274140],
  [(61,1),   0.000000575603073],
   [(1,2),   0.001069717680456],
   [(2,2),   0.994204425324595],
   [(3,2),   0.001069717680456],
   [(4,2),  -0.000211817505560],
   [(5,2),   0.000000575603073],
   [(6,2),   0.001069717680456],
   [(7,2),   0.000000575603073],
   [(8,2),  -0.000000114274140],
   [(9,2),  -0.000000114274140],
  [(10,2),  -0.000211817505560],
  [(11,2),  -0.000000114274140],
  [(12,2),   0.000000022686778],
  [(13,2),   0.000000575603073],
  [(14,2),   0.001069717680456],
  [(15,2),   0.000000575603073],
  [(16,2),  -0.000000114274140],
  [(17,2),   0.000000575603073],
  [(18,2),   0.001069717680456],
  [(19,2),   0.000000575603073],
  [(20,2),  -0.000000114274140],
  [(22,2),   0.000000575603073],
  [(26,2),  -0.000000114274140],
  [(30,2),   0.000000575603073],
  [(33,2),  -0.000000114274140],
  [(34,2),  -0.000211817505560],
  [(35,2),  -0.000000114274140],
  [(36,2),   0.000000022686778],
  [(38,2),  -0.000000114274140],
  [(42,2),   0.000000022686778],
  [(46,2),  -0.000000114274140],
  [(49,2),   0.000000575603073],
  [(50,2),   0.001069717680456],
  [(51,2),   0.000000575603073],
  [(52,2),  -0.000000114274140],
  [(54,2),   0.000000575603073],
  [(58,2),  -0.000000114274140],
  [(62,2),   0.000000575603073],
   [(1,3),  -0.000211817505560],
   [(2,3),   0.001069717680456],
   [(3,3),   0.994210420403448],
   [(4,3),   0.001069717680456],
   [(5,3),  -0.000000114274140],
   [(6,3),   0.000000575603073],
   [(7,3),   0.001069717680456],
   [(8,3),   0.000000575603073],
   [(9,3),   0.000000022686778],
  [(10,3),  -0.000000114274140],
  [(11,3),  -0.000211817505560],
  [(12,3),  -0.000000114274140],
  [(13,3),  -0.000000114274140],
  [(14,3),   0.000000575603073],
  [(15,3),   0.001069717680456],
  [(16,3),   0.000000575603073],
  [(17,3),  -0.000000114274140],
  [(18,3),   0.000000575603073],
  [(19,3),   0.001069717680456],
  [(20,3),   0.000000575603073],
  [(23,3),   0.000000575603073],
  [(27,3),  -0.000000114274140],
  [(31,3),   0.000000575603073],
  [(33,3),   0.000000022686778],
  [(34,3),  -0.000000114274140],
  [(35,3),  -0.000211817505560],
  [(36,3),  -0.000000114274140],
  [(39,3),  -0.000000114274140],
  [(43,3),   0.000000022686778],
  [(47,3),  -0.000000114274140],
  [(49,3),  -0.000000114274140],
  [(50,3),   0.000000575603073],
  [(51,3),   0.001069717680456],
  [(52,3),   0.000000575603073],
  [(55,3),   0.000000575603073],
  [(59,3),  -0.000000114274140],
  [(63,3),   0.000000575603073],
   [(1,4),   0.001069717680456],
   [(2,4),  -0.000211817505560],
   [(3,4),   0.001069717680456],
   [(4,4),   0.994204425324595],
   [(5,4),   0.000000575603073],
   [(6,4),  -0.000000114274140],
   [(7,4),   0.000000575603073],
   [(8,4),   0.001069717680456],
   [(9,4),  -0.000000114274140],
  [(10,4),   0.000000022686778],
  [(11,4),  -0.000000114274140],
  [(12,4),  -0.000211817505560],
  [(13,4),   0.000000575603073],
  [(14,4),  -0.000000114274140],
  [(15,4),   0.000000575603073],
  [(16,4),   0.001069717680456],
  [(17,4),   0.000000575603073],
  [(18,4),  -0.000000114274140],
  [(19,4),   0.000000575603073],
  [(20,4),   0.001069717680456],
  [(24,4),   0.000000575603073],
  [(28,4),  -0.000000114274140],
  [(32,4),   0.000000575603073],
  [(33,4),  -0.000000114274140],
  [(34,4),   0.000000022686778],
  [(35,4),  -0.000000114274140],
  [(36,4),  -0.000211817505560],
  [(40,4),  -0.000000114274140],
  [(44,4),   0.000000022686778],
  [(48,4),  -0.000000114274140],
  [(49,4),   0.000000575603073],
  [(50,4),  -0.000000114274140],
  [(51,4),   0.000000575603073],
  [(52,4),   0.001069717680456],
  [(56,4),   0.000000575603073],
  [(60,4),  -0.000000114274140],
  [(64,4),   0.000000575603073],
   [(1,5),   0.001069717680456],
   [(2,5),   0.000000575603073],
   [(3,5),  -0.000000114274140],
   [(4,5),   0.000000575603073],
   [(5,5),   0.994204425324595],
   [(6,5),   0.001069717680456],
   [(7,5),  -0.000211817505560],
   [(8,5),   0.001069717680456],
   [(9,5),   0.001069717680456],
  [(10,5),   0.000000575603073],
  [(11,5),  -0.000000114274140],
  [(12,5),   0.000000575603073],
  [(13,5),  -0.000211817505560],
  [(14,5),  -0.000000114274140],
  [(15,5),   0.000000022686778],
  [(16,5),  -0.000000114274140],
  [(17,5),   0.000000575603073],
  [(21,5),   0.001069717680456],
  [(22,5),   0.000000575603073],
  [(23,5),  -0.000000114274140],
  [(24,5),   0.000000575603073],
  [(25,5),   0.000000575603073],
  [(29,5),  -0.000000114274140],
  [(33,5),  -0.000000114274140],
  [(37,5),  -0.000211817505560],
  [(38,5),  -0.000000114274140],
  [(39,5),   0.000000022686778],
  [(40,5),  -0.000000114274140],
  [(41,5),  -0.000000114274140],
  [(45,5),   0.000000022686778],
  [(49,5),   0.000000575603073],
  [(53,5),   0.001069717680456],
  [(54,5),   0.000000575603073],
  [(55,5),  -0.000000114274140],
  [(56,5),   0.000000575603073],
  [(57,5),   0.000000575603073],
  [(61,5),  -0.000000114274140],
   [(1,6),   0.000000575603073],
   [(2,6),   0.001069717680456],
   [(3,6),   0.000000575603073],
   [(4,6),  -0.000000114274140],
   [(5,6),   0.001069717680456],
   [(6,6),   0.994208664156144],
   [(7,6),   0.001069717680456],
   [(8,6),  -0.000211817505560],
   [(9,6),   0.000000575603073],
  [(10,6),   0.001069717680456],
  [(11,6),   0.000000575603073],
  [(12,6),  -0.000000114274140],
  [(13,6),  -0.000000114274140],
  [(14,6),  -0.000211817505560],
  [(15,6),  -0.000000114274140],
  [(16,6),   0.000000022686778],
  [(18,6),   0.000000575603073],
  [(21,6),   0.000000575603073],
  [(22,6),   0.001069717680456],
  [(23,6),   0.000000575603073],
  [(24,6),  -0.000000114274140],
  [(26,6),   0.000000575603073],
  [(30,6),  -0.000000114274140],
  [(34,6),  -0.000000114274140],
  [(37,6),  -0.000000114274140],
  [(38,6),  -0.000211817505560],
  [(39,6),  -0.000000114274140],
  [(40,6),   0.000000022686778],
  [(42,6),  -0.000000114274140],
  [(46,6),   0.000000022686778],
  [(50,6),   0.000000575603073],
  [(53,6),   0.000000575603073],
  [(54,6),   0.001069717680456],
  [(55,6),   0.000000575603073],
  [(56,6),  -0.000000114274140],
  [(58,6),   0.000000575603073],
  [(62,6),  -0.000000114274140],
   [(1,7),  -0.000000114274140],
   [(2,7),   0.000000575603073],
   [(3,7),   0.001069717680456],
   [(4,7),   0.000000575603073],
   [(5,7),  -0.000211817505560],
   [(6,7),   0.001069717680456],
   [(7,7),   0.994212837645899],
   [(8,7),   0.001069717680456],
   [(9,7),  -0.000000114274140],
  [(10,7),   0.000000575603073],
  [(11,7),   0.001069717680456],
  [(12,7),   0.000000575603073],
  [(13,7),   0.000000022686778],
  [(14,7),  -0.000000114274140],
  [(15,7),  -0.000211817505560],
  [(16,7),  -0.000000114274140],
  [(19,7),   0.000000575603073],
  [(21,7),  -0.000000114274140],
  [(22,7),   0.000000575603073],
  [(23,7),   0.001069717680456],
  [(24,7),   0.000000575603073],
  [(27,7),   0.000000575603073],
  [(31,7),  -0.000000114274140],
  [(35,7),  -0.000000114274140],
  [(37,7),   0.000000022686778],
  [(38,7),  -0.000000114274140],
  [(39,7),  -0.000211817505560],
  [(40,7),  -0.000000114274140],
  [(43,7),  -0.000000114274140],
  [(47,7),   0.000000022686778],
  [(51,7),   0.000000575603073],
  [(53,7),  -0.000000114274140],
  [(54,7),   0.000000575603073],
  [(55,7),   0.001069717680456],
  [(56,7),   0.000000575603073],
  [(59,7),   0.000000575603073],
  [(63,7),  -0.000000114274140],
   [(1,8),   0.000000575603073],
   [(2,8),  -0.000000114274140],
   [(3,8),   0.000000575603073],
   [(4,8),   0.001069717680456],
   [(5,8),   0.001069717680456],
   [(6,8),  -0.000211817505560],
   [(7,8),   0.001069717680456],
   [(8,8),   0.994208664156144],
   [(9,8),   0.000000575603073],
  [(10,8),  -0.000000114274140],
  [(11,8),   0.000000575603073],
  [(12,8),   0.001069717680456],
  [(13,8),  -0.000000114274140],
  [(14,8),   0.000000022686778],
  [(15,8),  -0.000000114274140],
  [(16,8),  -0.000211817505560],
  [(20,8),   0.000000575603073],
  [(21,8),   0.000000575603073],
  [(22,8),  -0.000000114274140],
  [(23,8),   0.000000575603073],
  [(24,8),   0.001069717680456],
  [(28,8),   0.000000575603073],
  [(32,8),  -0.000000114274140],
  [(36,8),  -0.000000114274140],
  [(37,8),  -0.000000114274140],
  [(38,8),   0.000000022686778],
  [(39,8),  -0.000000114274140],
  [(40,8),  -0.000211817505560],
  [(44,8),  -0.000000114274140],
  [(48,8),   0.000000022686778],
  [(52,8),   0.000000575603073],
  [(53,8),   0.000000575603073],
  [(54,8),  -0.000000114274140],
  [(55,8),   0.000000575603073],
  [(56,8),   0.001069717680456],
  [(60,8),   0.000000575603073],
  [(64,8),  -0.000000114274140],
   [(1,9),  -0.000211817505560],
   [(2,9),  -0.000000114274140],
   [(3,9),   0.000000022686778],
   [(4,9),  -0.000000114274140],
   [(5,9),   0.001069717680456],
   [(6,9),   0.000000575603073],
   [(7,9),  -0.000000114274140],
   [(8,9),   0.000000575603073],
   [(9,9),   0.994210420403448],
  [(10,9),   0.001069717680456],
  [(11,9),  -0.000211817505560],
  [(12,9),   0.001069717680456],
  [(13,9),   0.001069717680456],
  [(14,9),   0.000000575603073],
  [(15,9),  -0.000000114274140],
  [(16,9),   0.000000575603073],
  [(17,9),  -0.000000114274140],
  [(21,9),   0.000000575603073],
  [(25,9),   0.001069717680456],
  [(26,9),   0.000000575603073],
  [(27,9),  -0.000000114274140],
  [(28,9),   0.000000575603073],
  [(29,9),   0.000000575603073],
  [(33,9),   0.000000022686778],
  [(37,9),  -0.000000114274140],
  [(41,9),  -0.000211817505560],
  [(42,9),  -0.000000114274140],
  [(43,9),   0.000000022686778],
  [(44,9),  -0.000000114274140],
  [(45,9),  -0.000000114274140],
  [(49,9),  -0.000000114274140],
  [(53,9),   0.000000575603073],
  [(57,9),   0.001069717680456],
  [(58,9),   0.000000575603073],
  [(59,9),  -0.000000114274140],
  [(60,9),   0.000000575603073],
  [(61,9),   0.000000575603073],
   [(1,10), -0.000000114274140],
   [(2,10), -0.000211817505560],
   [(3,10), -0.000000114274140],
   [(4,10),  0.000000022686778],
   [(5,10),  0.000000575603073],
   [(6,10),  0.001069717680456],
   [(7,10),  0.000000575603073],
   [(8,10), -0.000000114274140],
   [(9,10),  0.001069717680456],
  [(10,10),  0.994212837645899],
  [(11,10),  0.001069717680456],
  [(12,10), -0.000211817505560],
  [(13,10),  0.000000575603073],
  [(14,10),  0.001069717680456],
  [(15,10),  0.000000575603073],
  [(16,10), -0.000000114274140],
  [(18,10), -0.000000114274140],
  [(22,10),  0.000000575603073],
  [(25,10),  0.000000575603073],
  [(26,10),  0.001069717680456],
  [(27,10),  0.000000575603073],
  [(28,10), -0.000000114274140],
  [(30,10),  0.000000575603073],
  [(34,10),  0.000000022686778],
  [(38,10), -0.000000114274140],
  [(41,10), -0.000000114274140],
  [(42,10), -0.000211817505560],
  [(43,10), -0.000000114274140],
  [(44,10),  0.000000022686778],
  [(46,10), -0.000000114274140],
  [(50,10), -0.000000114274140],
  [(54,10),  0.000000575603073],
  [(57,10),  0.000000575603073],
  [(58,10),  0.001069717680456],
  [(59,10),  0.000000575603073],
  [(60,10), -0.000000114274140],
  [(62,10),  0.000000575603073],
   [(1,11),  0.000000022686778],
   [(2,11), -0.000000114274140],
   [(3,11), -0.000211817505560],
   [(4,11), -0.000000114274140],
   [(5,11), -0.000000114274140],
   [(6,11),  0.000000575603073],
   [(7,11),  0.001069717680456],
   [(8,11),  0.000000575603073],
   [(9,11), -0.000211817505560],
  [(10,11),  0.001069717680456],
  [(11,11),  0.994215213348107],
  [(12,11),  0.001069717680456],
  [(13,11), -0.000000114274140],
  [(14,11),  0.000000575603073],
  [(15,11),  0.001069717680456],
  [(16,11),  0.000000575603073],
  [(19,11), -0.000000114274140],
  [(23,11),  0.000000575603073],
  [(25,11), -0.000000114274140],
  [(26,11),  0.000000575603073],
  [(27,11),  0.001069717680456],
  [(28,11),  0.000000575603073],
  [(31,11),  0.000000575603073],
  [(35,11),  0.000000022686778],
  [(39,11), -0.000000114274140],
  [(41,11),  0.000000022686778],
  [(42,11), -0.000000114274140],
  [(43,11), -0.000211817505560],
  [(44,11), -0.000000114274140],
  [(47,11), -0.000000114274140],
  [(51,11), -0.000000114274140],
  [(55,11),  0.000000575603073],
  [(57,11), -0.000000114274140],
  [(58,11),  0.000000575603073],
  [(59,11),  0.001069717680456],
  [(60,11),  0.000000575603073],
  [(63,11),  0.000000575603073],
   [(1,12), -0.000000114274140],
   [(2,12),  0.000000022686778],
   [(3,12), -0.000000114274140],
   [(4,12), -0.000211817505560],
   [(5,12),  0.000000575603073],
   [(6,12), -0.000000114274140],
   [(7,12),  0.000000575603073],
   [(8,12),  0.001069717680456],
   [(9,12),  0.001069717680456],
  [(10,12), -0.000211817505560],
  [(11,12),  0.001069717680456],
  [(12,12),  0.994212837645899],
  [(13,12),  0.000000575603073],
  [(14,12), -0.000000114274140],
  [(15,12),  0.000000575603073],
  [(16,12),  0.001069717680456],
  [(20,12), -0.000000114274140],
  [(24,12),  0.000000575603073],
  [(25,12),  0.000000575603073],
  [(26,12), -0.000000114274140],
  [(27,12),  0.000000575603073],
  [(28,12),  0.001069717680456],
  [(32,12),  0.000000575603073],
  [(36,12),  0.000000022686778],
  [(40,12), -0.000000114274140],
  [(41,12), -0.000000114274140],
  [(42,12),  0.000000022686778],
  [(43,12), -0.000000114274140],
  [(44,12), -0.000211817505560],
  [(48,12), -0.000000114274140],
  [(52,12), -0.000000114274140],
  [(56,12),  0.000000575603073],
  [(57,12),  0.000000575603073],
  [(58,12), -0.000000114274140],
  [(59,12),  0.000000575603073],
  [(60,12),  0.001069717680456],
  [(64,12),  0.000000575603073],
   [(1,13),  0.001069717680456],
   [(2,13),  0.000000575603073],
   [(3,13), -0.000000114274140],
   [(4,13),  0.000000575603073],
   [(5,13), -0.000211817505560],
   [(6,13), -0.000000114274140],
   [(7,13),  0.000000022686778],
   [(8,13), -0.000000114274140],
   [(9,13),  0.001069717680456],
  [(10,13),  0.000000575603073],
  [(11,13), -0.000000114274140],
  [(12,13),  0.000000575603073],
  [(13,13),  0.994204425324595],
  [(14,13),  0.001069717680456],
  [(15,13), -0.000211817505560],
  [(16,13),  0.001069717680456],
  [(17,13),  0.000000575603073],
  [(21,13), -0.000000114274140],
  [(25,13),  0.000000575603073],
  [(29,13),  0.001069717680456],
  [(30,13),  0.000000575603073],
  [(31,13), -0.000000114274140],
  [(32,13),  0.000000575603073],
  [(33,13), -0.000000114274140],
  [(37,13),  0.000000022686778],
  [(41,13), -0.000000114274140],
  [(45,13), -0.000211817505560],
  [(46,13), -0.000000114274140],
  [(47,13),  0.000000022686778],
  [(48,13), -0.000000114274140],
  [(49,13),  0.000000575603073],
  [(53,13), -0.000000114274140],
  [(57,13),  0.000000575603073],
  [(61,13),  0.001069717680456],
  [(62,13),  0.000000575603073],
  [(63,13), -0.000000114274140],
  [(64,13),  0.000000575603073],
   [(1,14),  0.000000575603073],
   [(2,14),  0.001069717680456],
   [(3,14),  0.000000575603073],
   [(4,14), -0.000000114274140],
   [(5,14), -0.000000114274140],
   [(6,14), -0.000211817505560],
   [(7,14), -0.000000114274140],
   [(8,14),  0.000000022686778],
   [(9,14),  0.000000575603073],
  [(10,14),  0.001069717680456],
  [(11,14),  0.000000575603073],
  [(12,14), -0.000000114274140],
  [(13,14),  0.001069717680456],
  [(14,14),  0.994208664156144],
  [(15,14),  0.001069717680456],
  [(16,14), -0.000211817505560],
  [(18,14),  0.000000575603073],
  [(22,14), -0.000000114274140],
  [(26,14),  0.000000575603073],
  [(29,14),  0.000000575603073],
  [(30,14),  0.001069717680456],
  [(31,14),  0.000000575603073],
  [(32,14), -0.000000114274140],
  [(34,14), -0.000000114274140],
  [(38,14),  0.000000022686778],
  [(42,14), -0.000000114274140],
  [(45,14), -0.000000114274140],
  [(46,14), -0.000211817505560],
  [(47,14), -0.000000114274140],
  [(48,14),  0.000000022686778],
  [(50,14),  0.000000575603073],
  [(54,14), -0.000000114274140],
  [(58,14),  0.000000575603073],
  [(61,14),  0.000000575603073],
  [(62,14),  0.001069717680456],
  [(63,14),  0.000000575603073],
  [(64,14), -0.000000114274140],
   [(1,15), -0.000000114274140],
   [(2,15),  0.000000575603073],
   [(3,15),  0.001069717680456],
   [(4,15),  0.000000575603073],
   [(5,15),  0.000000022686778],
   [(6,15), -0.000000114274140],
   [(7,15), -0.000211817505560],
   [(8,15), -0.000000114274140],
   [(9,15), -0.000000114274140],
  [(10,15),  0.000000575603073],
  [(11,15),  0.001069717680456],
  [(12,15),  0.000000575603073],
  [(13,15), -0.000211817505560],
  [(14,15),  0.001069717680456],
  [(15,15),  0.994212837645899],
  [(16,15),  0.001069717680456],
  [(19,15),  0.000000575603073],
  [(23,15), -0.000000114274140],
  [(27,15),  0.000000575603073],
  [(29,15), -0.000000114274140],
  [(30,15),  0.000000575603073],
  [(31,15),  0.001069717680456],
  [(32,15),  0.000000575603073],
  [(35,15), -0.000000114274140],
  [(39,15),  0.000000022686778],
  [(43,15), -0.000000114274140],
  [(45,15),  0.000000022686778],
  [(46,15), -0.000000114274140],
  [(47,15), -0.000211817505560],
  [(48,15), -0.000000114274140],
  [(51,15),  0.000000575603073],
  [(55,15), -0.000000114274140],
  [(59,15),  0.000000575603073],
  [(61,15), -0.000000114274140],
  [(62,15),  0.000000575603073],
  [(63,15),  0.001069717680456],
  [(64,15),  0.000000575603073],
   [(1,16),  0.000000575603073],
   [(2,16), -0.000000114274140],
   [(3,16),  0.000000575603073],
   [(4,16),  0.001069717680456],
   [(5,16), -0.000000114274140],
   [(6,16),  0.000000022686778],
   [(7,16), -0.000000114274140],
   [(8,16), -0.000211817505560],
   [(9,16),  0.000000575603073],
  [(10,16), -0.000000114274140],
  [(11,16),  0.000000575603073],
  [(12,16),  0.001069717680456],
  [(13,16),  0.001069717680456],
  [(14,16), -0.000211817505560],
  [(15,16),  0.001069717680456],
  [(16,16),  0.994208664156144],
  [(20,16),  0.000000575603073],
  [(24,16), -0.000000114274140],
  [(28,16),  0.000000575603073],
  [(29,16),  0.000000575603073],
  [(30,16), -0.000000114274140],
  [(31,16),  0.000000575603073],
  [(32,16),  0.001069717680456],
  [(36,16), -0.000000114274140],
  [(40,16),  0.000000022686778],
  [(44,16), -0.000000114274140],
  [(45,16), -0.000000114274140],
  [(46,16),  0.000000022686778],
  [(47,16), -0.000000114274140],
  [(48,16), -0.000211817505560],
  [(52,16),  0.000000575603073],
  [(56,16), -0.000000114274140],
  [(60,16),  0.000000575603073],
  [(61,16),  0.000000575603073],
  [(62,16), -0.000000114274140],
  [(63,16),  0.000000575603073],
  [(64,16),  0.001069717680456],
   [(1,17),  0.001069717680456],
   [(2,17),  0.000000575603073],
   [(3,17), -0.000000114274140],
   [(4,17),  0.000000575603073],
   [(5,17),  0.000000575603073],
   [(9,17), -0.000000114274140],
  [(13,17),  0.000000575603073],
  [(17,17),  0.994204425324595],
  [(18,17),  0.001069717680456],
  [(19,17), -0.000211817505560],
  [(20,17),  0.001069717680456],
  [(21,17),  0.001069717680456],
  [(22,17),  0.000000575603073],
  [(23,17), -0.000000114274140],
  [(24,17),  0.000000575603073],
  [(25,17), -0.000211817505560],
  [(26,17), -0.000000114274140],
  [(27,17),  0.000000022686778],
  [(28,17), -0.000000114274140],
  [(29,17),  0.001069717680456],
  [(30,17),  0.000000575603073],
  [(31,17), -0.000000114274140],
  [(32,17),  0.000000575603073],
  [(33,17),  0.001069717680456],
  [(34,17),  0.000000575603073],
  [(35,17), -0.000000114274140],
  [(36,17),  0.000000575603073],
  [(37,17),  0.000000575603073],
  [(41,17), -0.000000114274140],
  [(45,17),  0.000000575603073],
  [(49,17), -0.000211817505560],
  [(50,17), -0.000000114274140],
  [(51,17),  0.000000022686778],
  [(52,17), -0.000000114274140],
  [(53,17), -0.000000114274140],
  [(57,17),  0.000000022686778],
  [(61,17), -0.000000114274140],
   [(1,18),  0.000000575603073],
   [(2,18),  0.001069717680456],
   [(3,18),  0.000000575603073],
   [(4,18), -0.000000114274140],
   [(6,18),  0.000000575603073],
  [(10,18), -0.000000114274140],
  [(14,18),  0.000000575603073],
  [(17,18),  0.001069717680456],
  [(18,18),  0.994208664156144],
  [(19,18),  0.001069717680456],
  [(20,18), -0.000211817505560],
  [(21,18),  0.000000575603073],
  [(22,18),  0.001069717680456],
  [(23,18),  0.000000575603073],
  [(24,18), -0.000000114274140],
  [(25,18), -0.000000114274140],
  [(26,18), -0.000211817505560],
  [(27,18), -0.000000114274140],
  [(28,18),  0.000000022686778],
  [(29,18),  0.000000575603073],
  [(30,18),  0.001069717680456],
  [(31,18),  0.000000575603073],
  [(32,18), -0.000000114274140],
  [(33,18),  0.000000575603073],
  [(34,18),  0.001069717680456],
  [(35,18),  0.000000575603073],
  [(36,18), -0.000000114274140],
  [(38,18),  0.000000575603073],
  [(42,18), -0.000000114274140],
  [(46,18),  0.000000575603073],
  [(49,18), -0.000000114274140],
  [(50,18), -0.000211817505560],
  [(51,18), -0.000000114274140],
  [(52,18),  0.000000022686778],
  [(54,18), -0.000000114274140],
  [(58,18),  0.000000022686778],
  [(62,18), -0.000000114274140],
   [(1,19), -0.000000114274140],
   [(2,19),  0.000000575603073],
   [(3,19),  0.001069717680456],
   [(4,19),  0.000000575603073],
   [(7,19),  0.000000575603073],
  [(11,19), -0.000000114274140],
  [(15,19),  0.000000575603073],
  [(17,19), -0.000211817505560],
  [(18,19),  0.001069717680456],
  [(19,19),  0.994212837645899],
  [(20,19),  0.001069717680456],
  [(21,19), -0.000000114274140],
  [(22,19),  0.000000575603073],
  [(23,19),  0.001069717680456],
  [(24,19),  0.000000575603073],
  [(25,19),  0.000000022686778],
  [(26,19), -0.000000114274140],
  [(27,19), -0.000211817505560],
  [(28,19), -0.000000114274140],
  [(29,19), -0.000000114274140],
  [(30,19),  0.000000575603073],
  [(31,19),  0.001069717680456],
  [(32,19),  0.000000575603073],
  [(33,19), -0.000000114274140],
  [(34,19),  0.000000575603073],
  [(35,19),  0.001069717680456],
  [(36,19),  0.000000575603073],
  [(39,19),  0.000000575603073],
  [(43,19), -0.000000114274140],
  [(47,19),  0.000000575603073],
  [(49,19),  0.000000022686778],
  [(50,19), -0.000000114274140],
  [(51,19), -0.000211817505560],
  [(52,19), -0.000000114274140],
  [(55,19), -0.000000114274140],
  [(59,19),  0.000000022686778],
  [(63,19), -0.000000114274140],
   [(1,20),  0.000000575603073],
   [(2,20), -0.000000114274140],
   [(3,20),  0.000000575603073],
   [(4,20),  0.001069717680456],
   [(8,20),  0.000000575603073],
  [(12,20), -0.000000114274140],
  [(16,20),  0.000000575603073],
  [(17,20),  0.001069717680456],
  [(18,20), -0.000211817505560],
  [(19,20),  0.001069717680456],
  [(20,20),  0.994208664156144],
  [(21,20),  0.000000575603073],
  [(22,20), -0.000000114274140],
  [(23,20),  0.000000575603073],
  [(24,20),  0.001069717680456],
  [(25,20), -0.000000114274140],
  [(26,20),  0.000000022686778],
  [(27,20), -0.000000114274140],
  [(28,20), -0.000211817505560],
  [(29,20),  0.000000575603073],
  [(30,20), -0.000000114274140],
  [(31,20),  0.000000575603073],
  [(32,20),  0.001069717680456],
  [(33,20),  0.000000575603073],
  [(34,20), -0.000000114274140],
  [(35,20),  0.000000575603073],
  [(36,20),  0.001069717680456],
  [(40,20),  0.000000575603073],
  [(44,20), -0.000000114274140],
  [(48,20),  0.000000575603073],
  [(49,20), -0.000000114274140],
  [(50,20),  0.000000022686778],
  [(51,20), -0.000000114274140],
  [(52,20), -0.000211817505560],
  [(56,20), -0.000000114274140],
  [(60,20),  0.000000022686778],
  [(64,20), -0.000000114274140],
   [(1,21),  0.000000575603073],
   [(5,21),  0.001069717680456],
   [(6,21),  0.000000575603073],
   [(7,21), -0.000000114274140],
   [(8,21),  0.000000575603073],
   [(9,21),  0.000000575603073],
  [(13,21), -0.000000114274140],
  [(17,21),  0.001069717680456],
  [(18,21),  0.000000575603073],
  [(19,21), -0.000000114274140],
  [(20,21),  0.000000575603073],
  [(21,21),  0.994208664156144],
  [(22,21),  0.001069717680456],
  [(23,21), -0.000211817505560],
  [(24,21),  0.001069717680456],
  [(25,21),  0.001069717680456],
  [(26,21),  0.000000575603073],
  [(27,21), -0.000000114274140],
  [(28,21),  0.000000575603073],
  [(29,21), -0.000211817505560],
  [(30,21), -0.000000114274140],
  [(31,21),  0.000000022686778],
  [(32,21), -0.000000114274140],
  [(33,21),  0.000000575603073],
  [(37,21),  0.001069717680456],
  [(38,21),  0.000000575603073],
  [(39,21), -0.000000114274140],
  [(40,21),  0.000000575603073],
  [(41,21),  0.000000575603073],
  [(45,21), -0.000000114274140],
  [(49,21), -0.000000114274140],
  [(53,21), -0.000211817505560],
  [(54,21), -0.000000114274140],
  [(55,21),  0.000000022686778],
  [(56,21), -0.000000114274140],
  [(57,21), -0.000000114274140],
  [(61,21),  0.000000022686778],
   [(2,22),  0.000000575603073],
   [(5,22),  0.000000575603073],
   [(6,22),  0.001069717680456],
   [(7,22),  0.000000575603073],
   [(8,22), -0.000000114274140],
  [(10,22),  0.000000575603073],
  [(14,22), -0.000000114274140],
  [(17,22),  0.000000575603073],
  [(18,22),  0.001069717680456],
  [(19,22),  0.000000575603073],
  [(20,22), -0.000000114274140],
  [(21,22),  0.001069717680456],
  [(22,22),  0.994211578247139],
  [(23,22),  0.001069717680456],
  [(24,22), -0.000211817505560],
  [(25,22),  0.000000575603073],
  [(26,22),  0.001069717680456],
  [(27,22),  0.000000575603073],
  [(28,22), -0.000000114274140],
  [(29,22), -0.000000114274140],
  [(30,22), -0.000211817505560],
  [(31,22), -0.000000114274140],
  [(32,22),  0.000000022686778],
  [(34,22),  0.000000575603073],
  [(37,22),  0.000000575603073],
  [(38,22),  0.001069717680456],
  [(39,22),  0.000000575603073],
  [(40,22), -0.000000114274140],
  [(42,22),  0.000000575603073],
  [(46,22), -0.000000114274140],
  [(50,22), -0.000000114274140],
  [(53,22), -0.000000114274140],
  [(54,22), -0.000211817505560],
  [(55,22), -0.000000114274140],
  [(56,22),  0.000000022686778],
  [(58,22), -0.000000114274140],
  [(62,22),  0.000000022686778],
   [(3,23),  0.000000575603073],
   [(5,23), -0.000000114274140],
   [(6,23),  0.000000575603073],
   [(7,23),  0.001069717680456],
   [(8,23),  0.000000575603073],
  [(11,23),  0.000000575603073],
  [(15,23), -0.000000114274140],
  [(17,23), -0.000000114274140],
  [(18,23),  0.000000575603073],
  [(19,23),  0.001069717680456],
  [(20,23),  0.000000575603073],
  [(21,23), -0.000211817505560],
  [(22,23),  0.001069717680456],
  [(23,23),  0.994214444628929],
  [(24,23),  0.001069717680456],
  [(25,23), -0.000000114274140],
  [(26,23),  0.000000575603073],
  [(27,23),  0.001069717680456],
  [(28,23),  0.000000575603073],
  [(29,23),  0.000000022686778],
  [(30,23), -0.000000114274140],
  [(31,23), -0.000211817505560],
  [(32,23), -0.000000114274140],
  [(35,23),  0.000000575603073],
  [(37,23), -0.000000114274140],
  [(38,23),  0.000000575603073],
  [(39,23),  0.001069717680456],
  [(40,23),  0.000000575603073],
  [(43,23),  0.000000575603073],
  [(47,23), -0.000000114274140],
  [(51,23), -0.000000114274140],
  [(53,23),  0.000000022686778],
  [(54,23), -0.000000114274140],
  [(55,23), -0.000211817505560],
  [(56,23), -0.000000114274140],
  [(59,23), -0.000000114274140],
  [(63,23),  0.000000022686778],
   [(4,24),  0.000000575603073],
   [(5,24),  0.000000575603073],
   [(6,24), -0.000000114274140],
   [(7,24),  0.000000575603073],
   [(8,24),  0.001069717680456],
  [(12,24),  0.000000575603073],
  [(16,24), -0.000000114274140],
  [(17,24),  0.000000575603073],
  [(18,24), -0.000000114274140],
  [(19,24),  0.000000575603073],
  [(20,24),  0.001069717680456],
  [(21,24),  0.001069717680456],
  [(22,24), -0.000211817505560],
  [(23,24),  0.001069717680456],
  [(24,24),  0.994211578247139],
  [(25,24),  0.000000575603073],
  [(26,24), -0.000000114274140],
  [(27,24),  0.000000575603073],
  [(28,24),  0.001069717680456],
  [(29,24), -0.000000114274140],
  [(30,24),  0.000000022686778],
  [(31,24), -0.000000114274140],
  [(32,24), -0.000211817505560],
  [(36,24),  0.000000575603073],
  [(37,24),  0.000000575603073],
  [(38,24), -0.000000114274140],
  [(39,24),  0.000000575603073],
  [(40,24),  0.001069717680456],
  [(44,24),  0.000000575603073],
  [(48,24), -0.000000114274140],
  [(52,24), -0.000000114274140],
  [(53,24), -0.000000114274140],
  [(54,24),  0.000000022686778],
  [(55,24), -0.000000114274140],
  [(56,24), -0.000211817505560],
  [(60,24), -0.000000114274140],
  [(64,24),  0.000000022686778],
   [(1,25), -0.000000114274140],
   [(5,25),  0.000000575603073],
   [(9,25),  0.001069717680456],
  [(10,25),  0.000000575603073],
  [(11,25), -0.000000114274140],
  [(12,25),  0.000000575603073],
  [(13,25),  0.000000575603073],
  [(17,25), -0.000211817505560],
  [(18,25), -0.000000114274140],
  [(19,25),  0.000000022686778],
  [(20,25), -0.000000114274140],
  [(21,25),  0.001069717680456],
  [(22,25),  0.000000575603073],
  [(23,25), -0.000000114274140],
  [(24,25),  0.000000575603073],
  [(25,25),  0.994212837645899],
  [(26,25),  0.001069717680456],
  [(27,25), -0.000211817505560],
  [(28,25),  0.001069717680456],
  [(29,25),  0.001069717680456],
  [(30,25),  0.000000575603073],
  [(31,25), -0.000000114274140],
  [(32,25),  0.000000575603073],
  [(33,25), -0.000000114274140],
  [(37,25),  0.000000575603073],
  [(41,25),  0.001069717680456],
  [(42,25),  0.000000575603073],
  [(43,25), -0.000000114274140],
  [(44,25),  0.000000575603073],
  [(45,25),  0.000000575603073],
  [(49,25),  0.000000022686778],
  [(53,25), -0.000000114274140],
  [(57,25), -0.000211817505560],
  [(58,25), -0.000000114274140],
  [(59,25),  0.000000022686778],
  [(60,25), -0.000000114274140],
  [(61,25), -0.000000114274140],
   [(2,26), -0.000000114274140],
   [(6,26),  0.000000575603073],
   [(9,26),  0.000000575603073],
  [(10,26),  0.001069717680456],
  [(11,26),  0.000000575603073],
  [(12,26), -0.000000114274140],
  [(14,26),  0.000000575603073],
  [(17,26), -0.000000114274140],
  [(18,26), -0.000211817505560],
  [(19,26), -0.000000114274140],
  [(20,26),  0.000000022686778],
  [(21,26),  0.000000575603073],
  [(22,26),  0.001069717680456],
  [(23,26),  0.000000575603073],
  [(24,26), -0.000000114274140],
  [(25,26),  0.001069717680456],
  [(26,26),  0.994214444628929],
  [(27,26),  0.001069717680456],
  [(28,26), -0.000211817505560],
  [(29,26),  0.000000575603073],
  [(30,26),  0.001069717680456],
  [(31,26),  0.000000575603073],
  [(32,26), -0.000000114274140],
  [(34,26), -0.000000114274140],
  [(38,26),  0.000000575603073],
  [(41,26),  0.000000575603073],
  [(42,26),  0.001069717680456],
  [(43,26),  0.000000575603073],
  [(44,26), -0.000000114274140],
  [(46,26),  0.000000575603073],
  [(50,26),  0.000000022686778],
  [(54,26), -0.000000114274140],
  [(57,26), -0.000000114274140],
  [(58,26), -0.000211817505560],
  [(59,26), -0.000000114274140],
  [(60,26),  0.000000022686778],
  [(62,26), -0.000000114274140],
   [(3,27), -0.000000114274140],
   [(7,27),  0.000000575603073],
   [(9,27), -0.000000114274140],
  [(10,27),  0.000000575603073],
  [(11,27),  0.001069717680456],
  [(12,27),  0.000000575603073],
  [(15,27),  0.000000575603073],
  [(17,27),  0.000000022686778],
  [(18,27), -0.000000114274140],
  [(19,27), -0.000211817505560],
  [(20,27), -0.000000114274140],
  [(21,27), -0.000000114274140],
  [(22,27),  0.000000575603073],
  [(23,27),  0.001069717680456],
  [(24,27),  0.000000575603073],
  [(25,27), -0.000211817505560],
  [(26,27),  0.001069717680456],
  [(27,27),  0.994216021328346],
  [(28,27),  0.001069717680456],
  [(29,27), -0.000000114274140],
  [(30,27),  0.000000575603073],
  [(31,27),  0.001069717680456],
  [(32,27),  0.000000575603073],
  [(35,27), -0.000000114274140],
  [(39,27),  0.000000575603073],
  [(41,27), -0.000000114274140],
  [(42,27),  0.000000575603073],
  [(43,27),  0.001069717680456],
  [(44,27),  0.000000575603073],
  [(47,27),  0.000000575603073],
  [(51,27),  0.000000022686778],
  [(55,27), -0.000000114274140],
  [(57,27),  0.000000022686778],
  [(58,27), -0.000000114274140],
  [(59,27), -0.000211817505560],
  [(60,27), -0.000000114274140],
  [(63,27), -0.000000114274140],
   [(4,28), -0.000000114274140],
   [(8,28),  0.000000575603073],
   [(9,28),  0.000000575603073],
  [(10,28), -0.000000114274140],
  [(11,28),  0.000000575603073],
  [(12,28),  0.001069717680456],
  [(16,28),  0.000000575603073],
  [(17,28), -0.000000114274140],
  [(18,28),  0.000000022686778],
  [(19,28), -0.000000114274140],
  [(20,28), -0.000211817505560],
  [(21,28),  0.000000575603073],
  [(22,28), -0.000000114274140],
  [(23,28),  0.000000575603073],
  [(24,28),  0.001069717680456],
  [(25,28),  0.001069717680456],
  [(26,28), -0.000211817505560],
  [(27,28),  0.001069717680456],
  [(28,28),  0.994214444628929],
  [(29,28),  0.000000575603073],
  [(30,28), -0.000000114274140],
  [(31,28),  0.000000575603073],
  [(32,28),  0.001069717680456],
  [(36,28), -0.000000114274140],
  [(40,28),  0.000000575603073],
  [(41,28),  0.000000575603073],
  [(42,28), -0.000000114274140],
  [(43,28),  0.000000575603073],
  [(44,28),  0.001069717680456],
  [(48,28),  0.000000575603073],
  [(52,28),  0.000000022686778],
  [(56,28), -0.000000114274140],
  [(57,28), -0.000000114274140],
  [(58,28),  0.000000022686778],
  [(59,28), -0.000000114274140],
  [(60,28), -0.000211817505560],
  [(64,28), -0.000000114274140],
   [(1,29),  0.000000575603073],
   [(5,29), -0.000000114274140],
   [(9,29),  0.000000575603073],
  [(13,29),  0.001069717680456],
  [(14,29),  0.000000575603073],
  [(15,29), -0.000000114274140],
  [(16,29),  0.000000575603073],
  [(17,29),  0.001069717680456],
  [(18,29),  0.000000575603073],
  [(19,29), -0.000000114274140],
  [(20,29),  0.000000575603073],
  [(21,29), -0.000211817505560],
  [(22,29), -0.000000114274140],
  [(23,29),  0.000000022686778],
  [(24,29), -0.000000114274140],
  [(25,29),  0.001069717680456],
  [(26,29),  0.000000575603073],
  [(27,29), -0.000000114274140],
  [(28,29),  0.000000575603073],
  [(29,29),  0.994208664156144],
  [(30,29),  0.001069717680456],
  [(31,29), -0.000211817505560],
  [(32,29),  0.001069717680456],
  [(33,29),  0.000000575603073],
  [(37,29), -0.000000114274140],
  [(41,29),  0.000000575603073],
  [(45,29),  0.001069717680456],
  [(46,29),  0.000000575603073],
  [(47,29), -0.000000114274140],
  [(48,29),  0.000000575603073],
  [(49,29), -0.000000114274140],
  [(53,29),  0.000000022686778],
  [(57,29), -0.000000114274140],
  [(61,29), -0.000211817505560],
  [(62,29), -0.000000114274140],
  [(63,29),  0.000000022686778],
  [(64,29), -0.000000114274140],
   [(2,30),  0.000000575603073],
   [(6,30), -0.000000114274140],
  [(10,30),  0.000000575603073],
  [(13,30),  0.000000575603073],
  [(14,30),  0.001069717680456],
  [(15,30),  0.000000575603073],
  [(16,30), -0.000000114274140],
  [(17,30),  0.000000575603073],
  [(18,30),  0.001069717680456],
  [(19,30),  0.000000575603073],
  [(20,30), -0.000000114274140],
  [(21,30), -0.000000114274140],
  [(22,30), -0.000211817505560],
  [(23,30), -0.000000114274140],
  [(24,30),  0.000000022686778],
  [(25,30),  0.000000575603073],
  [(26,30),  0.001069717680456],
  [(27,30),  0.000000575603073],
  [(28,30), -0.000000114274140],
  [(29,30),  0.001069717680456],
  [(30,30),  0.994211578247139],
  [(31,30),  0.001069717680456],
  [(32,30), -0.000211817505560],
  [(34,30),  0.000000575603073],
  [(38,30), -0.000000114274140],
  [(42,30),  0.000000575603073],
  [(45,30),  0.000000575603073],
  [(46,30),  0.001069717680456],
  [(47,30),  0.000000575603073],
  [(48,30), -0.000000114274140],
  [(50,30), -0.000000114274140],
  [(54,30),  0.000000022686778],
  [(58,30), -0.000000114274140],
  [(61,30), -0.000000114274140],
  [(62,30), -0.000211817505560],
  [(63,30), -0.000000114274140],
  [(64,30),  0.000000022686778],
   [(3,31),  0.000000575603073],
   [(7,31), -0.000000114274140],
  [(11,31),  0.000000575603073],
  [(13,31), -0.000000114274140],
  [(14,31),  0.000000575603073],
  [(15,31),  0.001069717680456],
  [(16,31),  0.000000575603073],
  [(17,31), -0.000000114274140],
  [(18,31),  0.000000575603073],
  [(19,31),  0.001069717680456],
  [(20,31),  0.000000575603073],
  [(21,31),  0.000000022686778],
  [(22,31), -0.000000114274140],
  [(23,31), -0.000211817505560],
  [(24,31), -0.000000114274140],
  [(25,31), -0.000000114274140],
  [(26,31),  0.000000575603073],
  [(27,31),  0.001069717680456],
  [(28,31),  0.000000575603073],
  [(29,31), -0.000211817505560],
  [(30,31),  0.001069717680456],
  [(31,31),  0.994214444628929],
  [(32,31),  0.001069717680456],
  [(35,31),  0.000000575603073],
  [(39,31), -0.000000114274140],
  [(43,31),  0.000000575603073],
  [(45,31), -0.000000114274140],
  [(46,31),  0.000000575603073],
  [(47,31),  0.001069717680456],
  [(48,31),  0.000000575603073],
  [(51,31), -0.000000114274140],
  [(55,31),  0.000000022686778],
  [(59,31), -0.000000114274140],
  [(61,31),  0.000000022686778],
  [(62,31), -0.000000114274140],
  [(63,31), -0.000211817505560],
  [(64,31), -0.000000114274140],
   [(4,32),  0.000000575603073],
   [(8,32), -0.000000114274140],
  [(12,32),  0.000000575603073],
  [(13,32),  0.000000575603073],
  [(14,32), -0.000000114274140],
  [(15,32),  0.000000575603073],
  [(16,32),  0.001069717680456],
  [(17,32),  0.000000575603073],
  [(18,32), -0.000000114274140],
  [(19,32),  0.000000575603073],
  [(20,32),  0.001069717680456],
  [(21,32), -0.000000114274140],
  [(22,32),  0.000000022686778],
  [(23,32), -0.000000114274140],
  [(24,32), -0.000211817505560],
  [(25,32),  0.000000575603073],
  [(26,32), -0.000000114274140],
  [(27,32),  0.000000575603073],
  [(28,32),  0.001069717680456],
  [(29,32),  0.001069717680456],
  [(30,32), -0.000211817505560],
  [(31,32),  0.001069717680456],
  [(32,32),  0.994211578247139],
  [(36,32),  0.000000575603073],
  [(40,32), -0.000000114274140],
  [(44,32),  0.000000575603073],
  [(45,32),  0.000000575603073],
  [(46,32), -0.000000114274140],
  [(47,32),  0.000000575603073],
  [(48,32),  0.001069717680456],
  [(52,32), -0.000000114274140],
  [(56,32),  0.000000022686778],
  [(60,32), -0.000000114274140],
  [(61,32), -0.000000114274140],
  [(62,32),  0.000000022686778],
  [(63,32), -0.000000114274140],
  [(64,32), -0.000211817505560],
   [(1,33), -0.000211817505560],
   [(2,33), -0.000000114274140],
   [(3,33),  0.000000022686778],
   [(4,33), -0.000000114274140],
   [(5,33), -0.000000114274140],
   [(9,33),  0.000000022686778],
  [(13,33), -0.000000114274140],
  [(17,33),  0.001069717680456],
  [(18,33),  0.000000575603073],
  [(19,33), -0.000000114274140],
  [(20,33),  0.000000575603073],
  [(21,33),  0.000000575603073],
  [(25,33), -0.000000114274140],
  [(29,33),  0.000000575603073],
  [(33,33),  0.994210420403448],
  [(34,33),  0.001069717680456],
  [(35,33), -0.000211817505560],
  [(36,33),  0.001069717680456],
  [(37,33),  0.001069717680456],
  [(38,33),  0.000000575603073],
  [(39,33), -0.000000114274140],
  [(40,33),  0.000000575603073],
  [(41,33), -0.000211817505560],
  [(42,33), -0.000000114274140],
  [(43,33),  0.000000022686778],
  [(44,33), -0.000000114274140],
  [(45,33),  0.001069717680456],
  [(46,33),  0.000000575603073],
  [(47,33), -0.000000114274140],
  [(48,33),  0.000000575603073],
  [(49,33),  0.001069717680456],
  [(50,33),  0.000000575603073],
  [(51,33), -0.000000114274140],
  [(52,33),  0.000000575603073],
  [(53,33),  0.000000575603073],
  [(57,33), -0.000000114274140],
  [(61,33),  0.000000575603073],
   [(1,34), -0.000000114274140],
   [(2,34), -0.000211817505560],
   [(3,34), -0.000000114274140],
   [(4,34),  0.000000022686778],
   [(6,34), -0.000000114274140],
  [(10,34),  0.000000022686778],
  [(14,34), -0.000000114274140],
  [(17,34),  0.000000575603073],
  [(18,34),  0.001069717680456],
  [(19,34),  0.000000575603073],
  [(20,34), -0.000000114274140],
  [(22,34),  0.000000575603073],
  [(26,34), -0.000000114274140],
  [(30,34),  0.000000575603073],
  [(33,34),  0.001069717680456],
  [(34,34),  0.994212837645899],
  [(35,34),  0.001069717680456],
  [(36,34), -0.000211817505560],
  [(37,34),  0.000000575603073],
  [(38,34),  0.001069717680456],
  [(39,34),  0.000000575603073],
  [(40,34), -0.000000114274140],
  [(41,34), -0.000000114274140],
  [(42,34), -0.000211817505560],
  [(43,34), -0.000000114274140],
  [(44,34),  0.000000022686778],
  [(45,34),  0.000000575603073],
  [(46,34),  0.001069717680456],
  [(47,34),  0.000000575603073],
  [(48,34), -0.000000114274140],
  [(49,34),  0.000000575603073],
  [(50,34),  0.001069717680456],
  [(51,34),  0.000000575603073],
  [(52,34), -0.000000114274140],
  [(54,34),  0.000000575603073],
  [(58,34), -0.000000114274140],
  [(62,34),  0.000000575603073],
   [(1,35),  0.000000022686778],
   [(2,35), -0.000000114274140],
   [(3,35), -0.000211817505560],
   [(4,35), -0.000000114274140],
   [(7,35), -0.000000114274140],
  [(11,35),  0.000000022686778],
  [(15,35), -0.000000114274140],
  [(17,35), -0.000000114274140],
  [(18,35),  0.000000575603073],
  [(19,35),  0.001069717680456],
  [(20,35),  0.000000575603073],
  [(23,35),  0.000000575603073],
  [(27,35), -0.000000114274140],
  [(31,35),  0.000000575603073],
  [(33,35), -0.000211817505560],
  [(34,35),  0.001069717680456],
  [(35,35),  0.994215213348107],
  [(36,35),  0.001069717680456],
  [(37,35), -0.000000114274140],
  [(38,35),  0.000000575603073],
  [(39,35),  0.001069717680456],
  [(40,35),  0.000000575603073],
  [(41,35),  0.000000022686778],
  [(42,35), -0.000000114274140],
  [(43,35), -0.000211817505560],
  [(44,35), -0.000000114274140],
  [(45,35), -0.000000114274140],
  [(46,35),  0.000000575603073],
  [(47,35),  0.001069717680456],
  [(48,35),  0.000000575603073],
  [(49,35), -0.000000114274140],
  [(50,35),  0.000000575603073],
  [(51,35),  0.001069717680456],
  [(52,35),  0.000000575603073],
  [(55,35),  0.000000575603073],
  [(59,35), -0.000000114274140],
  [(63,35),  0.000000575603073],
   [(1,36), -0.000000114274140],
   [(2,36),  0.000000022686778],
   [(3,36), -0.000000114274140],
   [(4,36), -0.000211817505560],
   [(8,36), -0.000000114274140],
  [(12,36),  0.000000022686778],
  [(16,36), -0.000000114274140],
  [(17,36),  0.000000575603073],
  [(18,36), -0.000000114274140],
  [(19,36),  0.000000575603073],
  [(20,36),  0.001069717680456],
  [(24,36),  0.000000575603073],
  [(28,36), -0.000000114274140],
  [(32,36),  0.000000575603073],
  [(33,36),  0.001069717680456],
  [(34,36), -0.000211817505560],
  [(35,36),  0.001069717680456],
  [(36,36),  0.994212837645899],
  [(37,36),  0.000000575603073],
  [(38,36), -0.000000114274140],
  [(39,36),  0.000000575603073],
  [(40,36),  0.001069717680456],
  [(41,36), -0.000000114274140],
  [(42,36),  0.000000022686778],
  [(43,36), -0.000000114274140],
  [(44,36), -0.000211817505560],
  [(45,36),  0.000000575603073],
  [(46,36), -0.000000114274140],
  [(47,36),  0.000000575603073],
  [(48,36),  0.001069717680456],
  [(49,36),  0.000000575603073],
  [(50,36), -0.000000114274140],
  [(51,36),  0.000000575603073],
  [(52,36),  0.001069717680456],
  [(56,36),  0.000000575603073],
  [(60,36), -0.000000114274140],
  [(64,36),  0.000000575603073],
   [(1,37), -0.000000114274140],
   [(5,37), -0.000211817505560],
   [(6,37), -0.000000114274140],
   [(7,37),  0.000000022686778],
   [(8,37), -0.000000114274140],
   [(9,37), -0.000000114274140],
  [(13,37),  0.000000022686778],
  [(17,37),  0.000000575603073],
  [(21,37),  0.001069717680456],
  [(22,37),  0.000000575603073],
  [(23,37), -0.000000114274140],
  [(24,37),  0.000000575603073],
  [(25,37),  0.000000575603073],
  [(29,37), -0.000000114274140],
  [(33,37),  0.001069717680456],
  [(34,37),  0.000000575603073],
  [(35,37), -0.000000114274140],
  [(36,37),  0.000000575603073],
  [(37,37),  0.994212837645899],
  [(38,37),  0.001069717680456],
  [(39,37), -0.000211817505560],
  [(40,37),  0.001069717680456],
  [(41,37),  0.001069717680456],
  [(42,37),  0.000000575603073],
  [(43,37), -0.000000114274140],
  [(44,37),  0.000000575603073],
  [(45,37), -0.000211817505560],
  [(46,37), -0.000000114274140],
  [(47,37),  0.000000022686778],
  [(48,37), -0.000000114274140],
  [(49,37),  0.000000575603073],
  [(53,37),  0.001069717680456],
  [(54,37),  0.000000575603073],
  [(55,37), -0.000000114274140],
  [(56,37),  0.000000575603073],
  [(57,37),  0.000000575603073],
  [(61,37), -0.000000114274140],
   [(2,38), -0.000000114274140],
   [(5,38), -0.000000114274140],
   [(6,38), -0.000211817505560],
   [(7,38), -0.000000114274140],
   [(8,38),  0.000000022686778],
  [(10,38), -0.000000114274140],
  [(14,38),  0.000000022686778],
  [(18,38),  0.000000575603073],
  [(21,38),  0.000000575603073],
  [(22,38),  0.001069717680456],
  [(23,38),  0.000000575603073],
  [(24,38), -0.000000114274140],
  [(26,38),  0.000000575603073],
  [(30,38), -0.000000114274140],
  [(33,38),  0.000000575603073],
  [(34,38),  0.001069717680456],
  [(35,38),  0.000000575603073],
  [(36,38), -0.000000114274140],
  [(37,38),  0.001069717680456],
  [(38,38),  0.994214444628929],
  [(39,38),  0.001069717680456],
  [(40,38), -0.000211817505560],
  [(41,38),  0.000000575603073],
  [(42,38),  0.001069717680456],
  [(43,38),  0.000000575603073],
  [(44,38), -0.000000114274140],
  [(45,38), -0.000000114274140],
  [(46,38), -0.000211817505560],
  [(47,38), -0.000000114274140],
  [(48,38),  0.000000022686778],
  [(50,38),  0.000000575603073],
  [(53,38),  0.000000575603073],
  [(54,38),  0.001069717680456],
  [(55,38),  0.000000575603073],
  [(56,38), -0.000000114274140],
  [(58,38),  0.000000575603073],
  [(62,38), -0.000000114274140],
   [(3,39), -0.000000114274140],
   [(5,39),  0.000000022686778],
   [(6,39), -0.000000114274140],
   [(7,39), -0.000211817505560],
   [(8,39), -0.000000114274140],
  [(11,39), -0.000000114274140],
  [(15,39),  0.000000022686778],
  [(19,39),  0.000000575603073],
  [(21,39), -0.000000114274140],
  [(22,39),  0.000000575603073],
  [(23,39),  0.001069717680456],
  [(24,39),  0.000000575603073],
  [(27,39),  0.000000575603073],
  [(31,39), -0.000000114274140],
  [(33,39), -0.000000114274140],
  [(34,39),  0.000000575603073],
  [(35,39),  0.001069717680456],
  [(36,39),  0.000000575603073],
  [(37,39), -0.000211817505560],
  [(38,39),  0.001069717680456],
  [(39,39),  0.994216021328346],
  [(40,39),  0.001069717680456],
  [(41,39), -0.000000114274140],
  [(42,39),  0.000000575603073],
  [(43,39),  0.001069717680456],
  [(44,39),  0.000000575603073],
  [(45,39),  0.000000022686778],
  [(46,39), -0.000000114274140],
  [(47,39), -0.000211817505560],
  [(48,39), -0.000000114274140],
  [(51,39),  0.000000575603073],
  [(53,39), -0.000000114274140],
  [(54,39),  0.000000575603073],
  [(55,39),  0.001069717680456],
  [(56,39),  0.000000575603073],
  [(59,39),  0.000000575603073],
  [(63,39), -0.000000114274140],
   [(4,40), -0.000000114274140],
   [(5,40), -0.000000114274140],
   [(6,40),  0.000000022686778],
   [(7,40), -0.000000114274140],
   [(8,40), -0.000211817505560],
  [(12,40), -0.000000114274140],
  [(16,40),  0.000000022686778],
  [(20,40),  0.000000575603073],
  [(21,40),  0.000000575603073],
  [(22,40), -0.000000114274140],
  [(23,40),  0.000000575603073],
  [(24,40),  0.001069717680456],
  [(28,40),  0.000000575603073],
  [(32,40), -0.000000114274140],
  [(33,40),  0.000000575603073],
  [(34,40), -0.000000114274140],
  [(35,40),  0.000000575603073],
  [(36,40),  0.001069717680456],
  [(37,40),  0.001069717680456],
  [(38,40), -0.000211817505560],
  [(39,40),  0.001069717680456],
  [(40,40),  0.994214444628929],
  [(41,40),  0.000000575603073],
  [(42,40), -0.000000114274140],
  [(43,40),  0.000000575603073],
  [(44,40),  0.001069717680456],
  [(45,40), -0.000000114274140],
  [(46,40),  0.000000022686778],
  [(47,40), -0.000000114274140],
  [(48,40), -0.000211817505560],
  [(52,40),  0.000000575603073],
  [(53,40),  0.000000575603073],
  [(54,40), -0.000000114274140],
  [(55,40),  0.000000575603073],
  [(56,40),  0.001069717680456],
  [(60,40),  0.000000575603073],
  [(64,40), -0.000000114274140],
   [(1,41),  0.000000022686778],
   [(5,41), -0.000000114274140],
   [(9,41), -0.000211817505560],
  [(10,41), -0.000000114274140],
  [(11,41),  0.000000022686778],
  [(12,41), -0.000000114274140],
  [(13,41), -0.000000114274140],
  [(17,41), -0.000000114274140],
  [(21,41),  0.000000575603073],
  [(25,41),  0.001069717680456],
  [(26,41),  0.000000575603073],
  [(27,41), -0.000000114274140],
  [(28,41),  0.000000575603073],
  [(29,41),  0.000000575603073],
  [(33,41), -0.000211817505560],
  [(34,41), -0.000000114274140],
  [(35,41),  0.000000022686778],
  [(36,41), -0.000000114274140],
  [(37,41),  0.001069717680456],
  [(38,41),  0.000000575603073],
  [(39,41), -0.000000114274140],
  [(40,41),  0.000000575603073],
  [(41,41),  0.994215213348107],
  [(42,41),  0.001069717680456],
  [(43,41), -0.000211817505560],
  [(44,41),  0.001069717680456],
  [(45,41),  0.001069717680456],
  [(46,41),  0.000000575603073],
  [(47,41), -0.000000114274140],
  [(48,41),  0.000000575603073],
  [(49,41), -0.000000114274140],
  [(53,41),  0.000000575603073],
  [(57,41),  0.001069717680456],
  [(58,41),  0.000000575603073],
  [(59,41), -0.000000114274140],
  [(60,41),  0.000000575603073],
  [(61,41),  0.000000575603073],
   [(2,42),  0.000000022686778],
   [(6,42), -0.000000114274140],
   [(9,42), -0.000000114274140],
  [(10,42), -0.000211817505560],
  [(11,42), -0.000000114274140],
  [(12,42),  0.000000022686778],
  [(14,42), -0.000000114274140],
  [(18,42), -0.000000114274140],
  [(22,42),  0.000000575603073],
  [(25,42),  0.000000575603073],
  [(26,42),  0.001069717680456],
  [(27,42),  0.000000575603073],
  [(28,42), -0.000000114274140],
  [(30,42),  0.000000575603073],
  [(33,42), -0.000000114274140],
  [(34,42), -0.000211817505560],
  [(35,42), -0.000000114274140],
  [(36,42),  0.000000022686778],
  [(37,42),  0.000000575603073],
  [(38,42),  0.001069717680456],
  [(39,42),  0.000000575603073],
  [(40,42), -0.000000114274140],
  [(41,42),  0.001069717680456],
  [(42,42),  0.994216021328346],
  [(43,42),  0.001069717680456],
  [(44,42), -0.000211817505560],
  [(45,42),  0.000000575603073],
  [(46,42),  0.001069717680456],
  [(47,42),  0.000000575603073],
  [(48,42), -0.000000114274140],
  [(50,42), -0.000000114274140],
  [(54,42),  0.000000575603073],
  [(57,42),  0.000000575603073],
  [(58,42),  0.001069717680456],
  [(59,42),  0.000000575603073],
  [(60,42), -0.000000114274140],
  [(62,42),  0.000000575603073],
   [(3,43),  0.000000022686778],
   [(7,43), -0.000000114274140],
   [(9,43),  0.000000022686778],
  [(10,43), -0.000000114274140],
  [(11,43), -0.000211817505560],
  [(12,43), -0.000000114274140],
  [(15,43), -0.000000114274140],
  [(19,43), -0.000000114274140],
  [(23,43),  0.000000575603073],
  [(25,43), -0.000000114274140],
  [(26,43),  0.000000575603073],
  [(27,43),  0.001069717680456],
  [(28,43),  0.000000575603073],
  [(31,43),  0.000000575603073],
  [(33,43),  0.000000022686778],
  [(34,43), -0.000000114274140],
  [(35,43), -0.000211817505560],
  [(36,43), -0.000000114274140],
  [(37,43), -0.000000114274140],
  [(38,43),  0.000000575603073],
  [(39,43),  0.001069717680456],
  [(40,43),  0.000000575603073],
  [(41,43), -0.000211817505560],
  [(42,43),  0.001069717680456],
  [(43,43),  0.994216810148551],
  [(44,43),  0.001069717680456],
  [(45,43), -0.000000114274140],
  [(46,43),  0.000000575603073],
  [(47,43),  0.001069717680456],
  [(48,43),  0.000000575603073],
  [(51,43), -0.000000114274140],
  [(55,43),  0.000000575603073],
  [(57,43), -0.000000114274140],
  [(58,43),  0.000000575603073],
  [(59,43),  0.001069717680456],
  [(60,43),  0.000000575603073],
  [(63,43),  0.000000575603073],
   [(4,44),  0.000000022686778],
   [(8,44), -0.000000114274140],
   [(9,44), -0.000000114274140],
  [(10,44),  0.000000022686778],
  [(11,44), -0.000000114274140],
  [(12,44), -0.000211817505560],
  [(16,44), -0.000000114274140],
  [(20,44), -0.000000114274140],
  [(24,44),  0.000000575603073],
  [(25,44),  0.000000575603073],
  [(26,44), -0.000000114274140],
  [(27,44),  0.000000575603073],
  [(28,44),  0.001069717680456],
  [(32,44),  0.000000575603073],
  [(33,44), -0.000000114274140],
  [(34,44),  0.000000022686778],
  [(35,44), -0.000000114274140],
  [(36,44), -0.000211817505560],
  [(37,44),  0.000000575603073],
  [(38,44), -0.000000114274140],
  [(39,44),  0.000000575603073],
  [(40,44),  0.001069717680456],
  [(41,44),  0.001069717680456],
  [(42,44), -0.000211817505560],
  [(43,44),  0.001069717680456],
  [(44,44),  0.994216021328346],
  [(45,44),  0.000000575603073],
  [(46,44), -0.000000114274140],
  [(47,44),  0.000000575603073],
  [(48,44),  0.001069717680456],
  [(52,44), -0.000000114274140],
  [(56,44),  0.000000575603073],
  [(57,44),  0.000000575603073],
  [(58,44), -0.000000114274140],
  [(59,44),  0.000000575603073],
  [(60,44),  0.001069717680456],
  [(64,44),  0.000000575603073],
   [(1,45), -0.000000114274140],
   [(5,45),  0.000000022686778],
   [(9,45), -0.000000114274140],
  [(13,45), -0.000211817505560],
  [(14,45), -0.000000114274140],
  [(15,45),  0.000000022686778],
  [(16,45), -0.000000114274140],
  [(17,45),  0.000000575603073],
  [(21,45), -0.000000114274140],
  [(25,45),  0.000000575603073],
  [(29,45),  0.001069717680456],
  [(30,45),  0.000000575603073],
  [(31,45), -0.000000114274140],
  [(32,45),  0.000000575603073],
  [(33,45),  0.001069717680456],
  [(34,45),  0.000000575603073],
  [(35,45), -0.000000114274140],
  [(36,45),  0.000000575603073],
  [(37,45), -0.000211817505560],
  [(38,45), -0.000000114274140],
  [(39,45),  0.000000022686778],
  [(40,45), -0.000000114274140],
  [(41,45),  0.001069717680456],
  [(42,45),  0.000000575603073],
  [(43,45), -0.000000114274140],
  [(44,45),  0.000000575603073],
  [(45,45),  0.994212837645899],
  [(46,45),  0.001069717680456],
  [(47,45), -0.000211817505560],
  [(48,45),  0.001069717680456],
  [(49,45),  0.000000575603073],
  [(53,45), -0.000000114274140],
  [(57,45),  0.000000575603073],
  [(61,45),  0.001069717680456],
  [(62,45),  0.000000575603073],
  [(63,45), -0.000000114274140],
  [(64,45),  0.000000575603073],
   [(2,46), -0.000000114274140],
   [(6,46),  0.000000022686778],
  [(10,46), -0.000000114274140],
  [(13,46), -0.000000114274140],
  [(14,46), -0.000211817505560],
  [(15,46), -0.000000114274140],
  [(16,46),  0.000000022686778],
  [(18,46),  0.000000575603073],
  [(22,46), -0.000000114274140],
  [(26,46),  0.000000575603073],
  [(29,46),  0.000000575603073],
  [(30,46),  0.001069717680456],
  [(31,46),  0.000000575603073],
  [(32,46), -0.000000114274140],
  [(33,46),  0.000000575603073],
  [(34,46),  0.001069717680456],
  [(35,46),  0.000000575603073],
  [(36,46), -0.000000114274140],
  [(37,46), -0.000000114274140],
  [(38,46), -0.000211817505560],
  [(39,46), -0.000000114274140],
  [(40,46),  0.000000022686778],
  [(41,46),  0.000000575603073],
  [(42,46),  0.001069717680456],
  [(43,46),  0.000000575603073],
  [(44,46), -0.000000114274140],
  [(45,46),  0.001069717680456],
  [(46,46),  0.994214444628929],
  [(47,46),  0.001069717680456],
  [(48,46), -0.000211817505560],
  [(50,46),  0.000000575603073],
  [(54,46), -0.000000114274140],
  [(58,46),  0.000000575603073],
  [(61,46),  0.000000575603073],
  [(62,46),  0.001069717680456],
  [(63,46),  0.000000575603073],
  [(64,46), -0.000000114274140],
   [(3,47), -0.000000114274140],
   [(7,47),  0.000000022686778],
  [(11,47), -0.000000114274140],
  [(13,47),  0.000000022686778],
  [(14,47), -0.000000114274140],
  [(15,47), -0.000211817505560],
  [(16,47), -0.000000114274140],
  [(19,47),  0.000000575603073],
  [(23,47), -0.000000114274140],
  [(27,47),  0.000000575603073],
  [(29,47), -0.000000114274140],
  [(30,47),  0.000000575603073],
  [(31,47),  0.001069717680456],
  [(32,47),  0.000000575603073],
  [(33,47), -0.000000114274140],
  [(34,47),  0.000000575603073],
  [(35,47),  0.001069717680456],
  [(36,47),  0.000000575603073],
  [(37,47),  0.000000022686778],
  [(38,47), -0.000000114274140],
  [(39,47), -0.000211817505560],
  [(40,47), -0.000000114274140],
  [(41,47), -0.000000114274140],
  [(42,47),  0.000000575603073],
  [(43,47),  0.001069717680456],
  [(44,47),  0.000000575603073],
  [(45,47), -0.000211817505560],
  [(46,47),  0.001069717680456],
  [(47,47),  0.994216021328346],
  [(48,47),  0.001069717680456],
  [(51,47),  0.000000575603073],
  [(55,47), -0.000000114274140],
  [(59,47),  0.000000575603073],
  [(61,47), -0.000000114274140],
  [(62,47),  0.000000575603073],
  [(63,47),  0.001069717680456],
  [(64,47),  0.000000575603073],
   [(4,48), -0.000000114274140],
   [(8,48),  0.000000022686778],
  [(12,48), -0.000000114274140],
  [(13,48), -0.000000114274140],
  [(14,48),  0.000000022686778],
  [(15,48), -0.000000114274140],
  [(16,48), -0.000211817505560],
  [(20,48),  0.000000575603073],
  [(24,48), -0.000000114274140],
  [(28,48),  0.000000575603073],
  [(29,48),  0.000000575603073],
  [(30,48), -0.000000114274140],
  [(31,48),  0.000000575603073],
  [(32,48),  0.001069717680456],
  [(33,48),  0.000000575603073],
  [(34,48), -0.000000114274140],
  [(35,48),  0.000000575603073],
  [(36,48),  0.001069717680456],
  [(37,48), -0.000000114274140],
  [(38,48),  0.000000022686778],
  [(39,48), -0.000000114274140],
  [(40,48), -0.000211817505560],
  [(41,48),  0.000000575603073],
  [(42,48), -0.000000114274140],
  [(43,48),  0.000000575603073],
  [(44,48),  0.001069717680456],
  [(45,48),  0.001069717680456],
  [(46,48), -0.000211817505560],
  [(47,48),  0.001069717680456],
  [(48,48),  0.994214444628929],
  [(52,48),  0.000000575603073],
  [(56,48), -0.000000114274140],
  [(60,48),  0.000000575603073],
  [(61,48),  0.000000575603073],
  [(62,48), -0.000000114274140],
  [(63,48),  0.000000575603073],
  [(64,48),  0.001069717680456],
   [(1,49),  0.001069717680456],
   [(2,49),  0.000000575603073],
   [(3,49), -0.000000114274140],
   [(4,49),  0.000000575603073],
   [(5,49),  0.000000575603073],
   [(9,49), -0.000000114274140],
  [(13,49),  0.000000575603073],
  [(17,49), -0.000211817505560],
  [(18,49), -0.000000114274140],
  [(19,49),  0.000000022686778],
  [(20,49), -0.000000114274140],
  [(21,49), -0.000000114274140],
  [(25,49),  0.000000022686778],
  [(29,49), -0.000000114274140],
  [(33,49),  0.001069717680456],
  [(34,49),  0.000000575603073],
  [(35,49), -0.000000114274140],
  [(36,49),  0.000000575603073],
  [(37,49),  0.000000575603073],
  [(41,49), -0.000000114274140],
  [(45,49),  0.000000575603073],
  [(49,49),  0.994204425324595],
  [(50,49),  0.001069717680456],
  [(51,49), -0.000211817505560],
  [(52,49),  0.001069717680456],
  [(53,49),  0.001069717680456],
  [(54,49),  0.000000575603073],
  [(55,49), -0.000000114274140],
  [(56,49),  0.000000575603073],
  [(57,49), -0.000211817505560],
  [(58,49), -0.000000114274140],
  [(59,49),  0.000000022686778],
  [(60,49), -0.000000114274140],
  [(61,49),  0.001069717680456],
  [(62,49),  0.000000575603073],
  [(63,49), -0.000000114274140],
  [(64,49),  0.000000575603073],
   [(1,50),  0.000000575603073],
   [(2,50),  0.001069717680456],
   [(3,50),  0.000000575603073],
   [(4,50), -0.000000114274140],
   [(6,50),  0.000000575603073],
  [(10,50), -0.000000114274140],
  [(14,50),  0.000000575603073],
  [(17,50), -0.000000114274140],
  [(18,50), -0.000211817505560],
  [(19,50), -0.000000114274140],
  [(20,50),  0.000000022686778],
  [(22,50), -0.000000114274140],
  [(26,50),  0.000000022686778],
  [(30,50), -0.000000114274140],
  [(33,50),  0.000000575603073],
  [(34,50),  0.001069717680456],
  [(35,50),  0.000000575603073],
  [(36,50), -0.000000114274140],
  [(38,50),  0.000000575603073],
  [(42,50), -0.000000114274140],
  [(46,50),  0.000000575603073],
  [(49,50),  0.001069717680456],
  [(50,50),  0.994208664156144],
  [(51,50),  0.001069717680456],
  [(52,50), -0.000211817505560],
  [(53,50),  0.000000575603073],
  [(54,50),  0.001069717680456],
  [(55,50),  0.000000575603073],
  [(56,50), -0.000000114274140],
  [(57,50), -0.000000114274140],
  [(58,50), -0.000211817505560],
  [(59,50), -0.000000114274140],
  [(60,50),  0.000000022686778],
  [(61,50),  0.000000575603073],
  [(62,50),  0.001069717680456],
  [(63,50),  0.000000575603073],
  [(64,50), -0.000000114274140],
   [(1,51), -0.000000114274140],
   [(2,51),  0.000000575603073],
   [(3,51),  0.001069717680456],
   [(4,51),  0.000000575603073],
   [(7,51),  0.000000575603073],
  [(11,51), -0.000000114274140],
  [(15,51),  0.000000575603073],
  [(17,51),  0.000000022686778],
  [(18,51), -0.000000114274140],
  [(19,51), -0.000211817505560],
  [(20,51), -0.000000114274140],
  [(23,51), -0.000000114274140],
  [(27,51),  0.000000022686778],
  [(31,51), -0.000000114274140],
  [(33,51), -0.000000114274140],
  [(34,51),  0.000000575603073],
  [(35,51),  0.001069717680456],
  [(36,51),  0.000000575603073],
  [(39,51),  0.000000575603073],
  [(43,51), -0.000000114274140],
  [(47,51),  0.000000575603073],
  [(49,51), -0.000211817505560],
  [(50,51),  0.001069717680456],
  [(51,51),  0.994212837645899],
  [(52,51),  0.001069717680456],
  [(53,51), -0.000000114274140],
  [(54,51),  0.000000575603073],
  [(55,51),  0.001069717680456],
  [(56,51),  0.000000575603073],
  [(57,51),  0.000000022686778],
  [(58,51), -0.000000114274140],
  [(59,51), -0.000211817505560],
  [(60,51), -0.000000114274140],
  [(61,51), -0.000000114274140],
  [(62,51),  0.000000575603073],
  [(63,51),  0.001069717680456],
  [(64,51),  0.000000575603073],
   [(1,52),  0.000000575603073],
   [(2,52), -0.000000114274140],
   [(3,52),  0.000000575603073],
   [(4,52),  0.001069717680456],
   [(8,52),  0.000000575603073],
  [(12,52), -0.000000114274140],
  [(16,52),  0.000000575603073],
  [(17,52), -0.000000114274140],
  [(18,52),  0.000000022686778],
  [(19,52), -0.000000114274140],
  [(20,52), -0.000211817505560],
  [(24,52), -0.000000114274140],
  [(28,52),  0.000000022686778],
  [(32,52), -0.000000114274140],
  [(33,52),  0.000000575603073],
  [(34,52), -0.000000114274140],
  [(35,52),  0.000000575603073],
  [(36,52),  0.001069717680456],
  [(40,52),  0.000000575603073],
  [(44,52), -0.000000114274140],
  [(48,52),  0.000000575603073],
  [(49,52),  0.001069717680456],
  [(50,52), -0.000211817505560],
  [(51,52),  0.001069717680456],
  [(52,52),  0.994208664156144],
  [(53,52),  0.000000575603073],
  [(54,52), -0.000000114274140],
  [(55,52),  0.000000575603073],
  [(56,52),  0.001069717680456],
  [(57,52), -0.000000114274140],
  [(58,52),  0.000000022686778],
  [(59,52), -0.000000114274140],
  [(60,52), -0.000211817505560],
  [(61,52),  0.000000575603073],
  [(62,52), -0.000000114274140],
  [(63,52),  0.000000575603073],
  [(64,52),  0.001069717680456],
   [(1,53),  0.000000575603073],
   [(5,53),  0.001069717680456],
   [(6,53),  0.000000575603073],
   [(7,53), -0.000000114274140],
   [(8,53),  0.000000575603073],
   [(9,53),  0.000000575603073],
  [(13,53), -0.000000114274140],
  [(17,53), -0.000000114274140],
  [(21,53), -0.000211817505560],
  [(22,53), -0.000000114274140],
  [(23,53),  0.000000022686778],
  [(24,53), -0.000000114274140],
  [(25,53), -0.000000114274140],
  [(29,53),  0.000000022686778],
  [(33,53),  0.000000575603073],
  [(37,53),  0.001069717680456],
  [(38,53),  0.000000575603073],
  [(39,53), -0.000000114274140],
  [(40,53),  0.000000575603073],
  [(41,53),  0.000000575603073],
  [(45,53), -0.000000114274140],
  [(49,53),  0.001069717680456],
  [(50,53),  0.000000575603073],
  [(51,53), -0.000000114274140],
  [(52,53),  0.000000575603073],
  [(53,53),  0.994208664156144],
  [(54,53),  0.001069717680456],
  [(55,53), -0.000211817505560],
  [(56,53),  0.001069717680456],
  [(57,53),  0.001069717680456],
  [(58,53),  0.000000575603073],
  [(59,53), -0.000000114274140],
  [(60,53),  0.000000575603073],
  [(61,53), -0.000211817505560],
  [(62,53), -0.000000114274140],
  [(63,53),  0.000000022686778],
  [(64,53), -0.000000114274140],
   [(2,54),  0.000000575603073],
   [(5,54),  0.000000575603073],
   [(6,54),  0.001069717680456],
   [(7,54),  0.000000575603073],
   [(8,54), -0.000000114274140],
  [(10,54),  0.000000575603073],
  [(14,54), -0.000000114274140],
  [(18,54), -0.000000114274140],
  [(21,54), -0.000000114274140],
  [(22,54), -0.000211817505560],
  [(23,54), -0.000000114274140],
  [(24,54),  0.000000022686778],
  [(26,54), -0.000000114274140],
  [(30,54),  0.000000022686778],
  [(34,54),  0.000000575603073],
  [(37,54),  0.000000575603073],
  [(38,54),  0.001069717680456],
  [(39,54),  0.000000575603073],
  [(40,54), -0.000000114274140],
  [(42,54),  0.000000575603073],
  [(46,54), -0.000000114274140],
  [(49,54),  0.000000575603073],
  [(50,54),  0.001069717680456],
  [(51,54),  0.000000575603073],
  [(52,54), -0.000000114274140],
  [(53,54),  0.001069717680456],
  [(54,54),  0.994211578247139],
  [(55,54),  0.001069717680456],
  [(56,54), -0.000211817505560],
  [(57,54),  0.000000575603073],
  [(58,54),  0.001069717680456],
  [(59,54),  0.000000575603073],
  [(60,54), -0.000000114274140],
  [(61,54), -0.000000114274140],
  [(62,54), -0.000211817505560],
  [(63,54), -0.000000114274140],
  [(64,54),  0.000000022686778],
   [(3,55),  0.000000575603073],
   [(5,55), -0.000000114274140],
   [(6,55),  0.000000575603073],
   [(7,55),  0.001069717680456],
   [(8,55),  0.000000575603073],
  [(11,55),  0.000000575603073],
  [(15,55), -0.000000114274140],
  [(19,55), -0.000000114274140],
  [(21,55),  0.000000022686778],
  [(22,55), -0.000000114274140],
  [(23,55), -0.000211817505560],
  [(24,55), -0.000000114274140],
  [(27,55), -0.000000114274140],
  [(31,55),  0.000000022686778],
  [(35,55),  0.000000575603073],
  [(37,55), -0.000000114274140],
  [(38,55),  0.000000575603073],
  [(39,55),  0.001069717680456],
  [(40,55),  0.000000575603073],
  [(43,55),  0.000000575603073],
  [(47,55), -0.000000114274140],
  [(49,55), -0.000000114274140],
  [(50,55),  0.000000575603073],
  [(51,55),  0.001069717680456],
  [(52,55),  0.000000575603073],
  [(53,55), -0.000211817505560],
  [(54,55),  0.001069717680456],
  [(55,55),  0.994214444628929],
  [(56,55),  0.001069717680456],
  [(57,55), -0.000000114274140],
  [(58,55),  0.000000575603073],
  [(59,55),  0.001069717680456],
  [(60,55),  0.000000575603073],
  [(61,55),  0.000000022686778],
  [(62,55), -0.000000114274140],
  [(63,55), -0.000211817505560],
  [(64,55), -0.000000114274140],
   [(4,56),  0.000000575603073],
   [(5,56),  0.000000575603073],
   [(6,56), -0.000000114274140],
   [(7,56),  0.000000575603073],
   [(8,56),  0.001069717680456],
  [(12,56),  0.000000575603073],
  [(16,56), -0.000000114274140],
  [(20,56), -0.000000114274140],
  [(21,56), -0.000000114274140],
  [(22,56),  0.000000022686778],
  [(23,56), -0.000000114274140],
  [(24,56), -0.000211817505560],
  [(28,56), -0.000000114274140],
  [(32,56),  0.000000022686778],
  [(36,56),  0.000000575603073],
  [(37,56),  0.000000575603073],
  [(38,56), -0.000000114274140],
  [(39,56),  0.000000575603073],
  [(40,56),  0.001069717680456],
  [(44,56),  0.000000575603073],
  [(48,56), -0.000000114274140],
  [(49,56),  0.000000575603073],
  [(50,56), -0.000000114274140],
  [(51,56),  0.000000575603073],
  [(52,56),  0.001069717680456],
  [(53,56),  0.001069717680456],
  [(54,56), -0.000211817505560],
  [(55,56),  0.001069717680456],
  [(56,56),  0.994211578247139],
  [(57,56),  0.000000575603073],
  [(58,56), -0.000000114274140],
  [(59,56),  0.000000575603073],
  [(60,56),  0.001069717680456],
  [(61,56), -0.000000114274140],
  [(62,56),  0.000000022686778],
  [(63,56), -0.000000114274140],
  [(64,56), -0.000211817505560],
   [(1,57), -0.000000114274140],
   [(5,57),  0.000000575603073],
   [(9,57),  0.001069717680456],
  [(10,57),  0.000000575603073],
  [(11,57), -0.000000114274140],
  [(12,57),  0.000000575603073],
  [(13,57),  0.000000575603073],
  [(17,57),  0.000000022686778],
  [(21,57), -0.000000114274140],
  [(25,57), -0.000211817505560],
  [(26,57), -0.000000114274140],
  [(27,57),  0.000000022686778],
  [(28,57), -0.000000114274140],
  [(29,57), -0.000000114274140],
  [(33,57), -0.000000114274140],
  [(37,57),  0.000000575603073],
  [(41,57),  0.001069717680456],
  [(42,57),  0.000000575603073],
  [(43,57), -0.000000114274140],
  [(44,57),  0.000000575603073],
  [(45,57),  0.000000575603073],
  [(49,57), -0.000211817505560],
  [(50,57), -0.000000114274140],
  [(51,57),  0.000000022686778],
  [(52,57), -0.000000114274140],
  [(53,57),  0.001069717680456],
  [(54,57),  0.000000575603073],
  [(55,57), -0.000000114274140],
  [(56,57),  0.000000575603073],
  [(57,57),  0.994212837645899],
  [(58,57),  0.001069717680456],
  [(59,57), -0.000211817505560],
  [(60,57),  0.001069717680456],
  [(61,57),  0.001069717680456],
  [(62,57),  0.000000575603073],
  [(63,57), -0.000000114274140],
  [(64,57),  0.000000575603073],
   [(2,58), -0.000000114274140],
   [(6,58),  0.000000575603073],
   [(9,58),  0.000000575603073],
  [(10,58),  0.001069717680456],
  [(11,58),  0.000000575603073],
  [(12,58), -0.000000114274140],
  [(14,58),  0.000000575603073],
  [(18,58),  0.000000022686778],
  [(22,58), -0.000000114274140],
  [(25,58), -0.000000114274140],
  [(26,58), -0.000211817505560],
  [(27,58), -0.000000114274140],
  [(28,58),  0.000000022686778],
  [(30,58), -0.000000114274140],
  [(34,58), -0.000000114274140],
  [(38,58),  0.000000575603073],
  [(41,58),  0.000000575603073],
  [(42,58),  0.001069717680456],
  [(43,58),  0.000000575603073],
  [(44,58), -0.000000114274140],
  [(46,58),  0.000000575603073],
  [(49,58), -0.000000114274140],
  [(50,58), -0.000211817505560],
  [(51,58), -0.000000114274140],
  [(52,58),  0.000000022686778],
  [(53,58),  0.000000575603073],
  [(54,58),  0.001069717680456],
  [(55,58),  0.000000575603073],
  [(56,58), -0.000000114274140],
  [(57,58),  0.001069717680456],
  [(58,58),  0.994214444628929],
  [(59,58),  0.001069717680456],
  [(60,58), -0.000211817505560],
  [(61,58),  0.000000575603073],
  [(62,58),  0.001069717680456],
  [(63,58),  0.000000575603073],
  [(64,58), -0.000000114274140],
   [(3,59), -0.000000114274140],
   [(7,59),  0.000000575603073],
   [(9,59), -0.000000114274140],
  [(10,59),  0.000000575603073],
  [(11,59),  0.001069717680456],
  [(12,59),  0.000000575603073],
  [(15,59),  0.000000575603073],
  [(19,59),  0.000000022686778],
  [(23,59), -0.000000114274140],
  [(25,59),  0.000000022686778],
  [(26,59), -0.000000114274140],
  [(27,59), -0.000211817505560],
  [(28,59), -0.000000114274140],
  [(31,59), -0.000000114274140],
  [(35,59), -0.000000114274140],
  [(39,59),  0.000000575603073],
  [(41,59), -0.000000114274140],
  [(42,59),  0.000000575603073],
  [(43,59),  0.001069717680456],
  [(44,59),  0.000000575603073],
  [(47,59),  0.000000575603073],
  [(49,59),  0.000000022686778],
  [(50,59), -0.000000114274140],
  [(51,59), -0.000211817505560],
  [(52,59), -0.000000114274140],
  [(53,59), -0.000000114274140],
  [(54,59),  0.000000575603073],
  [(55,59),  0.001069717680456],
  [(56,59),  0.000000575603073],
  [(57,59), -0.000211817505560],
  [(58,59),  0.001069717680456],
  [(59,59),  0.994216021328346],
  [(60,59),  0.001069717680456],
  [(61,59), -0.000000114274140],
  [(62,59),  0.000000575603073],
  [(63,59),  0.001069717680456],
  [(64,59),  0.000000575603073],
   [(4,60), -0.000000114274140],
   [(8,60),  0.000000575603073],
   [(9,60),  0.000000575603073],
  [(10,60), -0.000000114274140],
  [(11,60),  0.000000575603073],
  [(12,60),  0.001069717680456],
  [(16,60),  0.000000575603073],
  [(20,60),  0.000000022686778],
  [(24,60), -0.000000114274140],
  [(25,60), -0.000000114274140],
  [(26,60),  0.000000022686778],
  [(27,60), -0.000000114274140],
  [(28,60), -0.000211817505560],
  [(32,60), -0.000000114274140],
  [(36,60), -0.000000114274140],
  [(40,60),  0.000000575603073],
  [(41,60),  0.000000575603073],
  [(42,60), -0.000000114274140],
  [(43,60),  0.000000575603073],
  [(44,60),  0.001069717680456],
  [(48,60),  0.000000575603073],
  [(49,60), -0.000000114274140],
  [(50,60),  0.000000022686778],
  [(51,60), -0.000000114274140],
  [(52,60), -0.000211817505560],
  [(53,60),  0.000000575603073],
  [(54,60), -0.000000114274140],
  [(55,60),  0.000000575603073],
  [(56,60),  0.001069717680456],
  [(57,60),  0.001069717680456],
  [(58,60), -0.000211817505560],
  [(59,60),  0.001069717680456],
  [(60,60),  0.994214444628929],
  [(61,60),  0.000000575603073],
  [(62,60), -0.000000114274140],
  [(63,60),  0.000000575603073],
  [(64,60),  0.001069717680456],
   [(1,61),  0.000000575603073],
   [(5,61), -0.000000114274140],
   [(9,61),  0.000000575603073],
  [(13,61),  0.001069717680456],
  [(14,61),  0.000000575603073],
  [(15,61), -0.000000114274140],
  [(16,61),  0.000000575603073],
  [(17,61), -0.000000114274140],
  [(21,61),  0.000000022686778],
  [(25,61), -0.000000114274140],
  [(29,61), -0.000211817505560],
  [(30,61), -0.000000114274140],
  [(31,61),  0.000000022686778],
  [(32,61), -0.000000114274140],
  [(33,61),  0.000000575603073],
  [(37,61), -0.000000114274140],
  [(41,61),  0.000000575603073],
  [(45,61),  0.001069717680456],
  [(46,61),  0.000000575603073],
  [(47,61), -0.000000114274140],
  [(48,61),  0.000000575603073],
  [(49,61),  0.001069717680456],
  [(50,61),  0.000000575603073],
  [(51,61), -0.000000114274140],
  [(52,61),  0.000000575603073],
  [(53,61), -0.000211817505560],
  [(54,61), -0.000000114274140],
  [(55,61),  0.000000022686778],
  [(56,61), -0.000000114274140],
  [(57,61),  0.001069717680456],
  [(58,61),  0.000000575603073],
  [(59,61), -0.000000114274140],
  [(60,61),  0.000000575603073],
  [(61,61),  0.994208664156144],
  [(62,61),  0.001069717680456],
  [(63,61), -0.000211817505560],
  [(64,61),  0.001069717680456],
   [(2,62),  0.000000575603073],
   [(6,62), -0.000000114274140],
  [(10,62),  0.000000575603073],
  [(13,62),  0.000000575603073],
  [(14,62),  0.001069717680456],
  [(15,62),  0.000000575603073],
  [(16,62), -0.000000114274140],
  [(18,62), -0.000000114274140],
  [(22,62),  0.000000022686778],
  [(26,62), -0.000000114274140],
  [(29,62), -0.000000114274140],
  [(30,62), -0.000211817505560],
  [(31,62), -0.000000114274140],
  [(32,62),  0.000000022686778],
  [(34,62),  0.000000575603073],
  [(38,62), -0.000000114274140],
  [(42,62),  0.000000575603073],
  [(45,62),  0.000000575603073],
  [(46,62),  0.001069717680456],
  [(47,62),  0.000000575603073],
  [(48,62), -0.000000114274140],
  [(49,62),  0.000000575603073],
  [(50,62),  0.001069717680456],
  [(51,62),  0.000000575603073],
  [(52,62), -0.000000114274140],
  [(53,62), -0.000000114274140],
  [(54,62), -0.000211817505560],
  [(55,62), -0.000000114274140],
  [(56,62),  0.000000022686778],
  [(57,62),  0.000000575603073],
  [(58,62),  0.001069717680456],
  [(59,62),  0.000000575603073],
  [(60,62), -0.000000114274140],
  [(61,62),  0.001069717680456],
  [(62,62),  0.994211578247139],
  [(63,62),  0.001069717680456],
  [(64,62), -0.000211817505560],
   [(3,63),  0.000000575603073],
   [(7,63), -0.000000114274140],
  [(11,63),  0.000000575603073],
  [(13,63), -0.000000114274140],
  [(14,63),  0.000000575603073],
  [(15,63),  0.001069717680456],
  [(16,63),  0.000000575603073],
  [(19,63), -0.000000114274140],
  [(23,63),  0.000000022686778],
  [(27,63), -0.000000114274140],
  [(29,63),  0.000000022686778],
  [(30,63), -0.000000114274140],
  [(31,63), -0.000211817505560],
  [(32,63), -0.000000114274140],
  [(35,63),  0.000000575603073],
  [(39,63), -0.000000114274140],
  [(43,63),  0.000000575603073],
  [(45,63), -0.000000114274140],
  [(46,63),  0.000000575603073],
  [(47,63),  0.001069717680456],
  [(48,63),  0.000000575603073],
  [(49,63), -0.000000114274140],
  [(50,63),  0.000000575603073],
  [(51,63),  0.001069717680456],
  [(52,63),  0.000000575603073],
  [(53,63),  0.000000022686778],
  [(54,63), -0.000000114274140],
  [(55,63), -0.000211817505560],
  [(56,63), -0.000000114274140],
  [(57,63), -0.000000114274140],
  [(58,63),  0.000000575603073],
  [(59,63),  0.001069717680456],
  [(60,63),  0.000000575603073],
  [(61,63), -0.000211817505560],
  [(62,63),  0.001069717680456],
  [(63,63),  0.994214444628929],
  [(64,63),  0.001069717680456],
   [(4,64),  0.000000575603073],
   [(8,64), -0.000000114274140],
  [(12,64),  0.000000575603073],
  [(13,64),  0.000000575603073],
  [(14,64), -0.000000114274140],
  [(15,64),  0.000000575603073],
  [(16,64),  0.001069717680456],
  [(20,64), -0.000000114274140],
  [(24,64),  0.000000022686778],
  [(28,64), -0.000000114274140],
  [(29,64), -0.000000114274140],
  [(30,64),  0.000000022686778],
  [(31,64), -0.000000114274140],
  [(32,64), -0.000211817505560],
  [(36,64),  0.000000575603073],
  [(40,64), -0.000000114274140],
  [(44,64),  0.000000575603073],
  [(45,64),  0.000000575603073],
  [(46,64), -0.000000114274140],
  [(47,64),  0.000000575603073],
  [(48,64),  0.001069717680456],
  [(49,64),  0.000000575603073],
  [(50,64), -0.000000114274140],
  [(51,64),  0.000000575603073],
  [(52,64),  0.001069717680456],
  [(53,64), -0.000000114274140],
  [(54,64),  0.000000022686778],
  [(55,64), -0.000000114274140],
  [(56,64), -0.000211817505560],
  [(57,64),  0.000000575603073],
  [(58,64), -0.000000114274140],
  [(59,64),  0.000000575603073],
  [(60,64),  0.001069717680456],
  [(61,64),  0.001069717680456],
  [(62,64), -0.000211817505560],
  [(63,64),  0.001069717680456],
  [(64,64),  0.994211578247139]]

In [47]:
testL4Full = [[(1,1),   0.995609629058400],
   [(2,1),   0.001317465291762],
   [(3,1),  -0.000184461157144],
   [(4,1),   0.001317465291762],
   [(5,1),   0.001317465291762],
   [(6,1),   0.000027931951489],
   [(7,1),   0.000001195135140],
   [(8,1),   0.000027931951489],
   [(9,1),  -0.000184461157144],
  [(10,1),   0.000001195135140],
  [(11,1),   0.000000038205702],
  [(12,1),   0.000001195135140],
  [(13,1),   0.001317465291762],
  [(14,1),   0.000027931951489],
  [(15,1),   0.000001195135140],
  [(16,1),   0.000027931951489],
  [(17,1),   0.001317465291762],
  [(18,1),   0.000027931951489],
  [(19,1),   0.000001195135140],
  [(20,1),   0.000027931951489],
  [(21,1),   0.000027931951489],
  [(22,1),   0.000001309409280],
  [(23,1),   0.000000015518925],
  [(24,1),   0.000001309409280],
  [(25,1),   0.000001195135140],
  [(26,1),   0.000000015518925],
  [(28,1),   0.000000015518925],
  [(29,1),   0.000027931951489],
  [(30,1),   0.000001309409280],
  [(31,1),   0.000000015518925],
  [(32,1),   0.000001309409280],
  [(33,1),  -0.000184461157144],
  [(34,1),   0.000001195135140],
  [(35,1),   0.000000038205702],
  [(36,1),   0.000001195135140],
  [(37,1),   0.000001195135140],
  [(38,1),   0.000000015518925],
  [(40,1),   0.000000015518925],
  [(41,1),   0.000000038205702],
  [(45,1),   0.000001195135140],
  [(46,1),   0.000000015518925],
  [(48,1),   0.000000015518925],
  [(49,1),   0.001317465291762],
  [(50,1),   0.000027931951489],
  [(51,1),   0.000001195135140],
  [(52,1),   0.000027931951489],
  [(53,1),   0.000027931951489],
  [(54,1),   0.000001309409280],
  [(55,1),   0.000000015518925],
  [(56,1),   0.000001309409280],
  [(57,1),   0.000001195135140],
  [(58,1),   0.000000015518925],
  [(60,1),   0.000000015518925],
  [(61,1),   0.000027931951489],
  [(62,1),   0.000001309409280],
  [(63,1),   0.000000015518925],
  [(64,1),   0.000001309409280],
   [(1,2),   0.001317465291762],
   [(2,2),   0.994452172935901],
   [(3,2),   0.001108282612722],
   [(4,2),  -0.000173252573293],
   [(5,2),   0.000039140535340],
   [(6,2),   0.001108282612722],
   [(7,2),   0.000004071402540],
   [(8,2),   0.000003381525327],
   [(9,2),   0.000003381525327],
  [(10,2),  -0.000208321706093],
  [(11,2),  -0.000000017280860],
  [(12,2),   0.000000119680058],
  [(13,2),   0.000039140535340],
  [(14,2),   0.001108282612722],
  [(15,2),   0.000004071402540],
  [(16,2),   0.000003381525327],
  [(17,2),   0.000039140535340],
  [(18,2),   0.001108282612722],
  [(19,2),   0.000004071402540],
  [(20,2),   0.000003381525327],
  [(21,2),   0.000003495799467],
  [(22,2),   0.000004071402540],
  [(23,2),   0.000000096993280],
  [(24,2),   0.000000096993280],
  [(25,2),   0.000000096993280],
  [(26,2),  -0.000000017280860],
  [(29,2),   0.000003495799467],
  [(30,2),   0.000004071402540],
  [(31,2),   0.000000096993280],
  [(32,2),   0.000000096993280],
  [(33,2),   0.000003381525327],
  [(34,2),  -0.000208321706093],
  [(35,2),  -0.000000017280860],
  [(36,2),   0.000000119680058],
  [(37,2),   0.000000096993280],
  [(38,2),  -0.000000017280860],
  [(42,2),   0.000000022686778],
  [(45,2),   0.000000096993280],
  [(46,2),  -0.000000017280860],
  [(49,2),   0.000039140535340],
  [(50,2),   0.001108282612722],
  [(51,2),   0.000004071402540],
  [(52,2),   0.000003381525327],
  [(53,2),   0.000003495799467],
  [(54,2),   0.000004071402540],
  [(55,2),   0.000000096993280],
  [(56,2),   0.000000096993280],
  [(57,2),   0.000000096993280],
  [(58,2),  -0.000000017280860],
  [(61,2),   0.000003495799467],
  [(62,2),   0.000004071402540],
  [(63,2),   0.000000096993280],
  [(64,2),   0.000000096993280],
   [(1,3),  -0.000184461157144],
   [(2,3),   0.001108282612722],
   [(3,3),   0.994237776751864],
   [(4,3),   0.001108282612722],
   [(5,3),   0.000003381525327],
   [(6,3),   0.000006267168743],
   [(7,3),   0.001073213479922],
   [(8,3),   0.000006267168743],
   [(9,3),   0.000000242861523],
  [(10,3),   0.000000354526714],
  [(11,3),  -0.000211597330814],
  [(12,3),   0.000000354526714],
  [(13,3),   0.000003381525327],
  [(14,3),   0.000006267168743],
  [(15,3),   0.001073213479922],
  [(16,3),   0.000006267168743],
  [(17,3),   0.000003381525327],
  [(18,3),   0.000006267168743],
  [(19,3),   0.001073213479922],
  [(20,3),   0.000006267168743],
  [(21,3),   0.000000220174746],
  [(22,3),   0.000000468800853],
  [(23,3),   0.000000795777819],
  [(24,3),   0.000000468800853],
  [(26,3),   0.000000007759462],
  [(27,3),  -0.000000114274140],
  [(28,3),   0.000000007759462],
  [(29,3),   0.000000220174746],
  [(30,3),   0.000000468800853],
  [(31,3),   0.000000795777819],
  [(32,3),   0.000000468800853],
  [(33,3),   0.000000242861523],
  [(34,3),   0.000000354526714],
  [(35,3),  -0.000211597330814],
  [(36,3),   0.000000354526714],
  [(38,3),   0.000000007759462],
  [(39,3),  -0.000000114274140],
  [(40,3),   0.000000007759462],
  [(43,3),   0.000000022686778],
  [(46,3),   0.000000007759462],
  [(47,3),  -0.000000114274140],
  [(48,3),   0.000000007759462],
  [(49,3),   0.000003381525327],
  [(50,3),   0.000006267168743],
  [(51,3),   0.001073213479922],
  [(52,3),   0.000006267168743],
  [(53,3),   0.000000220174746],
  [(54,3),   0.000000468800853],
  [(55,3),   0.000000795777819],
  [(56,3),   0.000000468800853],
  [(58,3),   0.000000007759462],
  [(59,3),  -0.000000114274140],
  [(60,3),   0.000000007759462],
  [(61,3),   0.000000220174746],
  [(62,3),   0.000000468800853],
  [(63,3),   0.000000795777819],
  [(64,3),   0.000000468800853],
   [(1,4),   0.001317465291762],
   [(2,4),  -0.000173252573293],
   [(3,4),   0.001108282612722],
   [(4,4),   0.994452172935901],
   [(5,4),   0.000039140535340],
   [(6,4),   0.000003381525327],
   [(7,4),   0.000004071402540],
   [(8,4),   0.001108282612722],
   [(9,4),   0.000003381525327],
  [(10,4),   0.000000119680058],
  [(11,4),  -0.000000017280860],
  [(12,4),  -0.000208321706093],
  [(13,4),   0.000039140535340],
  [(14,4),   0.000003381525327],
  [(15,4),   0.000004071402540],
  [(16,4),   0.001108282612722],
  [(17,4),   0.000039140535340],
  [(18,4),   0.000003381525327],
  [(19,4),   0.000004071402540],
  [(20,4),   0.001108282612722],
  [(21,4),   0.000003495799467],
  [(22,4),   0.000000096993280],
  [(23,4),   0.000000096993280],
  [(24,4),   0.000004071402540],
  [(25,4),   0.000000096993280],
  [(28,4),  -0.000000017280860],
  [(29,4),   0.000003495799467],
  [(30,4),   0.000000096993280],
  [(31,4),   0.000000096993280],
  [(32,4),   0.000004071402540],
  [(33,4),   0.000003381525327],
  [(34,4),   0.000000119680058],
  [(35,4),  -0.000000017280860],
  [(36,4),  -0.000208321706093],
  [(37,4),   0.000000096993280],
  [(40,4),  -0.000000017280860],
  [(44,4),   0.000000022686778],
  [(45,4),   0.000000096993280],
  [(48,4),  -0.000000017280860],
  [(49,4),   0.000039140535340],
  [(50,4),   0.000003381525327],
  [(51,4),   0.000004071402540],
  [(52,4),   0.001108282612722],
  [(53,4),   0.000003495799467],
  [(54,4),   0.000000096993280],
  [(55,4),   0.000000096993280],
  [(56,4),   0.000004071402540],
  [(57,4),   0.000000096993280],
  [(60,4),  -0.000000017280860],
  [(61,4),   0.000003495799467],
  [(62,4),   0.000000096993280],
  [(63,4),   0.000000096993280],
  [(64,4),   0.000004071402540],
   [(1,5),   0.001317465291762],
   [(2,5),   0.000039140535340],
   [(3,5),   0.000003381525327],
   [(4,5),   0.000039140535340],
   [(5,5),   0.994452172935901],
   [(6,5),   0.001108282612722],
   [(7,5),  -0.000208321706093],
   [(8,5),   0.001108282612722],
   [(9,5),   0.001108282612722],
  [(10,5),   0.000004071402540],
  [(11,5),  -0.000000017280860],
  [(12,5),   0.000004071402540],
  [(13,5),  -0.000173252573293],
  [(14,5),   0.000003381525327],
  [(15,5),   0.000000119680058],
  [(16,5),   0.000003381525327],
  [(17,5),   0.000039140535340],
  [(18,5),   0.000003495799467],
  [(19,5),   0.000000096993280],
  [(20,5),   0.000003495799467],
  [(21,5),   0.001108282612722],
  [(22,5),   0.000004071402540],
  [(23,5),  -0.000000017280860],
  [(24,5),   0.000004071402540],
  [(25,5),   0.000004071402540],
  [(26,5),   0.000000096993280],
  [(28,5),   0.000000096993280],
  [(29,5),   0.000003381525327],
  [(30,5),   0.000000096993280],
  [(32,5),   0.000000096993280],
  [(33,5),   0.000003381525327],
  [(34,5),   0.000000096993280],
  [(36,5),   0.000000096993280],
  [(37,5),  -0.000208321706093],
  [(38,5),  -0.000000017280860],
  [(39,5),   0.000000022686778],
  [(40,5),  -0.000000017280860],
  [(41,5),  -0.000000017280860],
  [(45,5),   0.000000119680058],
  [(49,5),   0.000039140535340],
  [(50,5),   0.000003495799467],
  [(51,5),   0.000000096993280],
  [(52,5),   0.000003495799467],
  [(53,5),   0.001108282612722],
  [(54,5),   0.000004071402540],
  [(55,5),  -0.000000017280860],
  [(56,5),   0.000004071402540],
  [(57,5),   0.000004071402540],
  [(58,5),   0.000000096993280],
  [(60,5),   0.000000096993280],
  [(61,5),   0.000003381525327],
  [(62,5),   0.000000096993280],
  [(64,5),   0.000000096993280],
   [(1,6),   0.000027931951489],
   [(2,6),   0.001108282612722],
   [(3,6),   0.000006267168743],
   [(4,6),   0.000003381525327],
   [(5,6),   0.001108282612722],
   [(6,6),   0.994236020504560],
   [(7,6),   0.001073213479922],
   [(8,6),  -0.000206125939889],
   [(9,6),   0.000006267168743],
  [(10,6),   0.001073213479922],
  [(11,6),   0.000000795777819],
  [(12,6),   0.000000354526714],
  [(13,6),   0.000003381525327],
  [(14,6),  -0.000206125939889],
  [(15,6),   0.000000354526714],
  [(16,6),   0.000000242861523],
  [(17,6),   0.000003495799467],
  [(18,6),   0.000006267168743],
  [(19,6),   0.000000468800853],
  [(20,6),   0.000000220174746],
  [(21,6),   0.000006267168743],
  [(22,6),   0.001073213479922],
  [(23,6),   0.000000795777819],
  [(24,6),   0.000000354526714],
  [(25,6),   0.000000468800853],
  [(26,6),   0.000000795777819],
  [(28,6),   0.000000007759462],
  [(29,6),   0.000000220174746],
  [(30,6),   0.000000354526714],
  [(31,6),   0.000000007759462],
  [(33,6),   0.000000220174746],
  [(34,6),   0.000000354526714],
  [(35,6),   0.000000007759462],
  [(37,6),   0.000000354526714],
  [(38,6),  -0.000211597330814],
  [(39,6),  -0.000000114274140],
  [(40,6),   0.000000030446240],
  [(41,6),   0.000000007759462],
  [(42,6),  -0.000000114274140],
  [(46,6),   0.000000030446240],
  [(49,6),   0.000003495799467],
  [(50,6),   0.000006267168743],
  [(51,6),   0.000000468800853],
  [(52,6),   0.000000220174746],
  [(53,6),   0.000006267168743],
  [(54,6),   0.001073213479922],
  [(55,6),   0.000000795777819],
  [(56,6),   0.000000354526714],
  [(57,6),   0.000000468800853],
  [(58,6),   0.000000795777819],
  [(60,6),   0.000000007759462],
  [(61,6),   0.000000220174746],
  [(62,6),   0.000000354526714],
  [(63,6),   0.000000007759462],
   [(1,7),   0.000001195135140],
   [(2,7),   0.000004071402540],
   [(3,7),   0.001073213479922],
   [(4,7),   0.000004071402540],
   [(5,7),  -0.000208321706093],
   [(6,7),   0.001073213479922],
   [(7,7),   0.994214147055179],
   [(8,7),   0.001073213479922],
   [(9,7),   0.000000354526714],
  [(10,7),   0.000001044403926],
  [(11,7),   0.001069814673736],
  [(12,7),   0.000001044403926],
  [(13,7),   0.000000119680058],
  [(14,7),   0.000000354526714],
  [(15,7),  -0.000211348704706],
  [(16,7),   0.000000354526714],
  [(17,7),   0.000000096993280],
  [(18,7),   0.000000468800853],
  [(19,7),   0.000001044403926],
  [(20,7),   0.000000468800853],
  [(21,7),   0.000000354526714],
  [(22,7),   0.000001044403926],
  [(23,7),   0.001069814673736],
  [(24,7),   0.000001044403926],
  [(25,7),   0.000000032331093],
  [(26,7),   0.000000032331093],
  [(27,7),   0.000000575603073],
  [(28,7),   0.000000032331093],
  [(30,7),   0.000000032331093],
  [(31,7),  -0.000000081943046],
  [(32,7),   0.000000032331093],
  [(34,7),   0.000000032331093],
  [(35,7),  -0.000000081943046],
  [(36,7),   0.000000032331093],
  [(37,7),   0.000000055017871],
  [(38,7),  -0.000000081943046],
  [(39,7),  -0.000211817505560],
  [(40,7),  -0.000000081943046],
  [(43,7),  -0.000000114274140],
  [(47,7),   0.000000022686778],
  [(49,7),   0.000000096993280],
  [(50,7),   0.000000468800853],
  [(51,7),   0.000001044403926],
  [(52,7),   0.000000468800853],
  [(53,7),   0.000000354526714],
  [(54,7),   0.000001044403926],
  [(55,7),   0.001069814673736],
  [(56,7),   0.000001044403926],
  [(57,7),   0.000000032331093],
  [(58,7),   0.000000032331093],
  [(59,7),   0.000000575603073],
  [(60,7),   0.000000032331093],
  [(62,7),   0.000000032331093],
  [(63,7),  -0.000000081943046],
  [(64,7),   0.000000032331093],
   [(1,8),   0.000027931951489],
   [(2,8),   0.000003381525327],
   [(3,8),   0.000006267168743],
   [(4,8),   0.001108282612722],
   [(5,8),   0.001108282612722],
   [(6,8),  -0.000206125939889],
   [(7,8),   0.001073213479922],
   [(8,8),   0.994236020504560],
   [(9,8),   0.000006267168743],
  [(10,8),   0.000000354526714],
  [(11,8),   0.000000795777819],
  [(12,8),   0.001073213479922],
  [(13,8),   0.000003381525327],
  [(14,8),   0.000000242861523],
  [(15,8),   0.000000354526714],
  [(16,8),  -0.000206125939889],
  [(17,8),   0.000003495799467],
  [(18,8),   0.000000220174746],
  [(19,8),   0.000000468800853],
  [(20,8),   0.000006267168743],
  [(21,8),   0.000006267168743],
  [(22,8),   0.000000354526714],
  [(23,8),   0.000000795777819],
  [(24,8),   0.001073213479922],
  [(25,8),   0.000000468800853],
  [(26,8),   0.000000007759462],
  [(28,8),   0.000000795777819],
  [(29,8),   0.000000220174746],
  [(31,8),   0.000000007759462],
  [(32,8),   0.000000354526714],
  [(33,8),   0.000000220174746],
  [(35,8),   0.000000007759462],
  [(36,8),   0.000000354526714],
  [(37,8),   0.000000354526714],
  [(38,8),   0.000000030446240],
  [(39,8),  -0.000000114274140],
  [(40,8),  -0.000211597330814],
  [(41,8),   0.000000007759462],
  [(44,8),  -0.000000114274140],
  [(48,8),   0.000000030446240],
  [(49,8),   0.000003495799467],
  [(50,8),   0.000000220174746],
  [(51,8),   0.000000468800853],
  [(52,8),   0.000006267168743],
  [(53,8),   0.000006267168743],
  [(54,8),   0.000000354526714],
  [(55,8),   0.000000795777819],
  [(56,8),   0.001073213479922],
  [(57,8),   0.000000468800853],
  [(58,8),   0.000000007759462],
  [(60,8),   0.000000795777819],
  [(61,8),   0.000000220174746],
  [(63,8),   0.000000007759462],
  [(64,8),   0.000000354526714],
   [(1,9),  -0.000184461157144],
   [(2,9),   0.000003381525327],
   [(3,9),   0.000000242861523],
   [(4,9),   0.000003381525327],
   [(5,9),   0.001108282612722],
   [(6,9),   0.000006267168743],
   [(7,9),   0.000000354526714],
   [(8,9),   0.000006267168743],
   [(9,9),   0.994237776751864],
  [(10,9),   0.001073213479922],
  [(11,9),  -0.000211597330814],
  [(12,9),   0.001073213479922],
  [(13,9),   0.001108282612722],
  [(14,9),   0.000006267168743],
  [(15,9),   0.000000354526714],
  [(16,9),   0.000006267168743],
  [(17,9),   0.000003381525327],
  [(18,9),   0.000000220174746],
  [(20,9),   0.000000220174746],
  [(21,9),   0.000006267168743],
  [(22,9),   0.000000468800853],
  [(23,9),   0.000000007759462],
  [(24,9),   0.000000468800853],
  [(25,9),   0.001073213479922],
  [(26,9),   0.000000795777819],
  [(27,9),  -0.000000114274140],
  [(28,9),   0.000000795777819],
  [(29,9),   0.000006267168743],
  [(30,9),   0.000000468800853],
  [(31,9),   0.000000007759462],
  [(32,9),   0.000000468800853],
  [(33,9),   0.000000242861523],
  [(37,9),   0.000000354526714],
  [(38,9),   0.000000007759462],
  [(40,9),   0.000000007759462],
  [(41,9),  -0.000211597330814],
  [(42,9),  -0.000000114274140],
  [(43,9),   0.000000022686778],
  [(44,9),  -0.000000114274140],
  [(45,9),   0.000000354526714],
  [(46,9),   0.000000007759462],
  [(48,9),   0.000000007759462],
  [(49,9),   0.000003381525327],
  [(50,9),   0.000000220174746],
  [(52,9),   0.000000220174746],
  [(53,9),   0.000006267168743],
  [(54,9),   0.000000468800853],
  [(55,9),   0.000000007759462],
  [(56,9),   0.000000468800853],
  [(57,9),   0.001073213479922],
  [(58,9),   0.000000795777819],
  [(59,9),  -0.000000114274140],
  [(60,9),   0.000000795777819],
  [(61,9),   0.000006267168743],
  [(62,9),   0.000000468800853],
  [(63,9),   0.000000007759462],
  [(64,9),   0.000000468800853],
   [(1,10),  0.000001195135140],
   [(2,10), -0.000208321706093],
   [(3,10),  0.000000354526714],
   [(4,10),  0.000000119680058],
   [(5,10),  0.000004071402540],
   [(6,10),  0.001073213479922],
   [(7,10),  0.000001044403926],
   [(8,10),  0.000000354526714],
   [(9,10),  0.001073213479922],
  [(10,10),  0.994214147055179],
  [(11,10),  0.001069814673736],
  [(12,10), -0.000211348704706],
  [(13,10),  0.000004071402540],
  [(14,10),  0.001073213479922],
  [(15,10),  0.000001044403926],
  [(16,10),  0.000000354526714],
  [(17,10),  0.000000096993280],
  [(18,10),  0.000000354526714],
  [(19,10),  0.000000032331093],
  [(21,10),  0.000000468800853],
  [(22,10),  0.000001044403926],
  [(23,10),  0.000000032331093],
  [(24,10),  0.000000032331093],
  [(25,10),  0.000001044403926],
  [(26,10),  0.001069814673736],
  [(27,10),  0.000000575603073],
  [(28,10), -0.000000081943046],
  [(29,10),  0.000000468800853],
  [(30,10),  0.000001044403926],
  [(31,10),  0.000000032331093],
  [(32,10),  0.000000032331093],
  [(34,10),  0.000000055017871],
  [(37,10),  0.000000032331093],
  [(38,10), -0.000000081943046],
  [(41,10), -0.000000081943046],
  [(42,10), -0.000211817505560],
  [(43,10), -0.000000114274140],
  [(44,10),  0.000000022686778],
  [(45,10),  0.000000032331093],
  [(46,10), -0.000000081943046],
  [(49,10),  0.000000096993280],
  [(50,10),  0.000000354526714],
  [(51,10),  0.000000032331093],
  [(53,10),  0.000000468800853],
  [(54,10),  0.000001044403926],
  [(55,10),  0.000000032331093],
  [(56,10),  0.000000032331093],
  [(57,10),  0.000001044403926],
  [(58,10),  0.001069814673736],
  [(59,10),  0.000000575603073],
  [(60,10), -0.000000081943046],
  [(61,10),  0.000000468800853],
  [(62,10),  0.000001044403926],
  [(63,10),  0.000000032331093],
  [(64,10),  0.000000032331093],
   [(1,11),  0.000000038205702],
   [(2,11), -0.000000017280860],
   [(3,11), -0.000211597330814],
   [(4,11), -0.000000017280860],
   [(5,11), -0.000000017280860],
   [(6,11),  0.000000795777819],
   [(7,11),  0.001069814673736],
   [(8,11),  0.000000795777819],
   [(9,11), -0.000211597330814],
  [(10,11),  0.001069814673736],
  [(11,11),  0.994215228867032],
  [(12,11),  0.001069814673736],
  [(13,11), -0.000000017280860],
  [(14,11),  0.000000795777819],
  [(15,11),  0.001069814673736],
  [(16,11),  0.000000795777819],
  [(18,11),  0.000000007759462],
  [(19,11), -0.000000081943046],
  [(20,11),  0.000000007759462],
  [(21,11),  0.000000007759462],
  [(22,11),  0.000000032331093],
  [(23,11),  0.000000583362535],
  [(24,11),  0.000000032331093],
  [(25,11), -0.000000081943046],
  [(26,11),  0.000000583362535],
  [(27,11),  0.001069717680456],
  [(28,11),  0.000000583362535],
  [(29,11),  0.000000007759462],
  [(30,11),  0.000000032331093],
  [(31,11),  0.000000583362535],
  [(32,11),  0.000000032331093],
  [(35,11),  0.000000025273265],
  [(38,11),  0.000000002586487],
  [(39,11), -0.000000114274140],
  [(40,11),  0.000000002586487],
  [(41,11),  0.000000025273265],
  [(42,11), -0.000000114274140],
  [(43,11), -0.000211817505560],
  [(44,11), -0.000000114274140],
  [(46,11),  0.000000002586487],
  [(47,11), -0.000000114274140],
  [(48,11),  0.000000002586487],
  [(50,11),  0.000000007759462],
  [(51,11), -0.000000081943046],
  [(52,11),  0.000000007759462],
  [(53,11),  0.000000007759462],
  [(54,11),  0.000000032331093],
  [(55,11),  0.000000583362535],
  [(56,11),  0.000000032331093],
  [(57,11), -0.000000081943046],
  [(58,11),  0.000000583362535],
  [(59,11),  0.001069717680456],
  [(60,11),  0.000000583362535],
  [(61,11),  0.000000007759462],
  [(62,11),  0.000000032331093],
  [(63,11),  0.000000583362535],
  [(64,11),  0.000000032331093],
   [(1,12),  0.000001195135140],
   [(2,12),  0.000000119680058],
   [(3,12),  0.000000354526714],
   [(4,12), -0.000208321706093],
   [(5,12),  0.000004071402540],
   [(6,12),  0.000000354526714],
   [(7,12),  0.000001044403926],
   [(8,12),  0.001073213479922],
   [(9,12),  0.001073213479922],
  [(10,12), -0.000211348704706],
  [(11,12),  0.001069814673736],
  [(12,12),  0.994214147055179],
  [(13,12),  0.000004071402540],
  [(14,12),  0.000000354526714],
  [(15,12),  0.000001044403926],
  [(16,12),  0.001073213479922],
  [(17,12),  0.000000096993280],
  [(19,12),  0.000000032331093],
  [(20,12),  0.000000354526714],
  [(21,12),  0.000000468800853],
  [(22,12),  0.000000032331093],
  [(23,12),  0.000000032331093],
  [(24,12),  0.000001044403926],
  [(25,12),  0.000001044403926],
  [(26,12), -0.000000081943046],
  [(27,12),  0.000000575603073],
  [(28,12),  0.001069814673736],
  [(29,12),  0.000000468800853],
  [(30,12),  0.000000032331093],
  [(31,12),  0.000000032331093],
  [(32,12),  0.000001044403926],
  [(36,12),  0.000000055017871],
  [(37,12),  0.000000032331093],
  [(40,12), -0.000000081943046],
  [(41,12), -0.000000081943046],
  [(42,12),  0.000000022686778],
  [(43,12), -0.000000114274140],
  [(44,12), -0.000211817505560],
  [(45,12),  0.000000032331093],
  [(48,12), -0.000000081943046],
  [(49,12),  0.000000096993280],
  [(51,12),  0.000000032331093],
  [(52,12),  0.000000354526714],
  [(53,12),  0.000000468800853],
  [(54,12),  0.000000032331093],
  [(55,12),  0.000000032331093],
  [(56,12),  0.000001044403926],
  [(57,12),  0.000001044403926],
  [(58,12), -0.000000081943046],
  [(59,12),  0.000000575603073],
  [(60,12),  0.001069814673736],
  [(61,12),  0.000000468800853],
  [(62,12),  0.000000032331093],
  [(63,12),  0.000000032331093],
  [(64,12),  0.000001044403926],
   [(1,13),  0.001317465291762],
   [(2,13),  0.000039140535340],
   [(3,13),  0.000003381525327],
   [(4,13),  0.000039140535340],
   [(5,13), -0.000173252573293],
   [(6,13),  0.000003381525327],
   [(7,13),  0.000000119680058],
   [(8,13),  0.000003381525327],
   [(9,13),  0.001108282612722],
  [(10,13),  0.000004071402540],
  [(11,13), -0.000000017280860],
  [(12,13),  0.000004071402540],
  [(13,13),  0.994452172935901],
  [(14,13),  0.001108282612722],
  [(15,13), -0.000208321706093],
  [(16,13),  0.001108282612722],
  [(17,13),  0.000039140535340],
  [(18,13),  0.000003495799467],
  [(19,13),  0.000000096993280],
  [(20,13),  0.000003495799467],
  [(21,13),  0.000003381525327],
  [(22,13),  0.000000096993280],
  [(24,13),  0.000000096993280],
  [(25,13),  0.000004071402540],
  [(26,13),  0.000000096993280],
  [(28,13),  0.000000096993280],
  [(29,13),  0.001108282612722],
  [(30,13),  0.000004071402540],
  [(31,13), -0.000000017280860],
  [(32,13),  0.000004071402540],
  [(33,13),  0.000003381525327],
  [(34,13),  0.000000096993280],
  [(36,13),  0.000000096993280],
  [(37,13),  0.000000119680058],
  [(41,13), -0.000000017280860],
  [(45,13), -0.000208321706093],
  [(46,13), -0.000000017280860],
  [(47,13),  0.000000022686778],
  [(48,13), -0.000000017280860],
  [(49,13),  0.000039140535340],
  [(50,13),  0.000003495799467],
  [(51,13),  0.000000096993280],
  [(52,13),  0.000003495799467],
  [(53,13),  0.000003381525327],
  [(54,13),  0.000000096993280],
  [(56,13),  0.000000096993280],
  [(57,13),  0.000004071402540],
  [(58,13),  0.000000096993280],
  [(60,13),  0.000000096993280],
  [(61,13),  0.001108282612722],
  [(62,13),  0.000004071402540],
  [(63,13), -0.000000017280860],
  [(64,13),  0.000004071402540],
   [(1,14),  0.000027931951489],
   [(2,14),  0.001108282612722],
   [(3,14),  0.000006267168743],
   [(4,14),  0.000003381525327],
   [(5,14),  0.000003381525327],
   [(6,14), -0.000206125939889],
   [(7,14),  0.000000354526714],
   [(8,14),  0.000000242861523],
   [(9,14),  0.000006267168743],
  [(10,14),  0.001073213479922],
  [(11,14),  0.000000795777819],
  [(12,14),  0.000000354526714],
  [(13,14),  0.001108282612722],
  [(14,14),  0.994236020504560],
  [(15,14),  0.001073213479922],
  [(16,14), -0.000206125939889],
  [(17,14),  0.000003495799467],
  [(18,14),  0.000006267168743],
  [(19,14),  0.000000468800853],
  [(20,14),  0.000000220174746],
  [(21,14),  0.000000220174746],
  [(22,14),  0.000000354526714],
  [(23,14),  0.000000007759462],
  [(25,14),  0.000000468800853],
  [(26,14),  0.000000795777819],
  [(28,14),  0.000000007759462],
  [(29,14),  0.000006267168743],
  [(30,14),  0.001073213479922],
  [(31,14),  0.000000795777819],
  [(32,14),  0.000000354526714],
  [(33,14),  0.000000220174746],
  [(34,14),  0.000000354526714],
  [(35,14),  0.000000007759462],
  [(38,14),  0.000000030446240],
  [(41,14),  0.000000007759462],
  [(42,14), -0.000000114274140],
  [(45,14),  0.000000354526714],
  [(46,14), -0.000211597330814],
  [(47,14), -0.000000114274140],
  [(48,14),  0.000000030446240],
  [(49,14),  0.000003495799467],
  [(50,14),  0.000006267168743],
  [(51,14),  0.000000468800853],
  [(52,14),  0.000000220174746],
  [(53,14),  0.000000220174746],
  [(54,14),  0.000000354526714],
  [(55,14),  0.000000007759462],
  [(57,14),  0.000000468800853],
  [(58,14),  0.000000795777819],
  [(60,14),  0.000000007759462],
  [(61,14),  0.000006267168743],
  [(62,14),  0.001073213479922],
  [(63,14),  0.000000795777819],
  [(64,14),  0.000000354526714],
   [(1,15),  0.000001195135140],
   [(2,15),  0.000004071402540],
   [(3,15),  0.001073213479922],
   [(4,15),  0.000004071402540],
   [(5,15),  0.000000119680058],
   [(6,15),  0.000000354526714],
   [(7,15), -0.000211348704706],
   [(8,15),  0.000000354526714],
   [(9,15),  0.000000354526714],
  [(10,15),  0.000001044403926],
  [(11,15),  0.001069814673736],
  [(12,15),  0.000001044403926],
  [(13,15), -0.000208321706093],
  [(14,15),  0.001073213479922],
  [(15,15),  0.994214147055179],
  [(16,15),  0.001073213479922],
  [(17,15),  0.000000096993280],
  [(18,15),  0.000000468800853],
  [(19,15),  0.000001044403926],
  [(20,15),  0.000000468800853],
  [(22,15),  0.000000032331093],
  [(23,15), -0.000000081943046],
  [(24,15),  0.000000032331093],
  [(25,15),  0.000000032331093],
  [(26,15),  0.000000032331093],
  [(27,15),  0.000000575603073],
  [(28,15),  0.000000032331093],
  [(29,15),  0.000000354526714],
  [(30,15),  0.000001044403926],
  [(31,15),  0.001069814673736],
  [(32,15),  0.000001044403926],
  [(34,15),  0.000000032331093],
  [(35,15), -0.000000081943046],
  [(36,15),  0.000000032331093],
  [(39,15),  0.000000022686778],
  [(43,15), -0.000000114274140],
  [(45,15),  0.000000055017871],
  [(46,15), -0.000000081943046],
  [(47,15), -0.000211817505560],
  [(48,15), -0.000000081943046],
  [(49,15),  0.000000096993280],
  [(50,15),  0.000000468800853],
  [(51,15),  0.000001044403926],
  [(52,15),  0.000000468800853],
  [(54,15),  0.000000032331093],
  [(55,15), -0.000000081943046],
  [(56,15),  0.000000032331093],
  [(57,15),  0.000000032331093],
  [(58,15),  0.000000032331093],
  [(59,15),  0.000000575603073],
  [(60,15),  0.000000032331093],
  [(61,15),  0.000000354526714],
  [(62,15),  0.000001044403926],
  [(63,15),  0.001069814673736],
  [(64,15),  0.000001044403926],
   [(1,16),  0.000027931951489],
   [(2,16),  0.000003381525327],
   [(3,16),  0.000006267168743],
   [(4,16),  0.001108282612722],
   [(5,16),  0.000003381525327],
   [(6,16),  0.000000242861523],
   [(7,16),  0.000000354526714],
   [(8,16), -0.000206125939889],
   [(9,16),  0.000006267168743],
  [(10,16),  0.000000354526714],
  [(11,16),  0.000000795777819],
  [(12,16),  0.001073213479922],
  [(13,16),  0.001108282612722],
  [(14,16), -0.000206125939889],
  [(15,16),  0.001073213479922],
  [(16,16),  0.994236020504560],
  [(17,16),  0.000003495799467],
  [(18,16),  0.000000220174746],
  [(19,16),  0.000000468800853],
  [(20,16),  0.000006267168743],
  [(21,16),  0.000000220174746],
  [(23,16),  0.000000007759462],
  [(24,16),  0.000000354526714],
  [(25,16),  0.000000468800853],
  [(26,16),  0.000000007759462],
  [(28,16),  0.000000795777819],
  [(29,16),  0.000006267168743],
  [(30,16),  0.000000354526714],
  [(31,16),  0.000000795777819],
  [(32,16),  0.001073213479922],
  [(33,16),  0.000000220174746],
  [(35,16),  0.000000007759462],
  [(36,16),  0.000000354526714],
  [(40,16),  0.000000030446240],
  [(41,16),  0.000000007759462],
  [(44,16), -0.000000114274140],
  [(45,16),  0.000000354526714],
  [(46,16),  0.000000030446240],
  [(47,16), -0.000000114274140],
  [(48,16), -0.000211597330814],
  [(49,16),  0.000003495799467],
  [(50,16),  0.000000220174746],
  [(51,16),  0.000000468800853],
  [(52,16),  0.000006267168743],
  [(53,16),  0.000000220174746],
  [(55,16),  0.000000007759462],
  [(56,16),  0.000000354526714],
  [(57,16),  0.000000468800853],
  [(58,16),  0.000000007759462],
  [(60,16),  0.000000795777819],
  [(61,16),  0.000006267168743],
  [(62,16),  0.000000354526714],
  [(63,16),  0.000000795777819],
  [(64,16),  0.001073213479922],
   [(1,17),  0.001317465291762],
   [(2,17),  0.000039140535340],
   [(3,17),  0.000003381525327],
   [(4,17),  0.000039140535340],
   [(5,17),  0.000039140535340],
   [(6,17),  0.000003495799467],
   [(7,17),  0.000000096993280],
   [(8,17),  0.000003495799467],
   [(9,17),  0.000003381525327],
  [(10,17),  0.000000096993280],
  [(12,17),  0.000000096993280],
  [(13,17),  0.000039140535340],
  [(14,17),  0.000003495799467],
  [(15,17),  0.000000096993280],
  [(16,17),  0.000003495799467],
  [(17,17),  0.994452172935901],
  [(18,17),  0.001108282612722],
  [(19,17), -0.000208321706093],
  [(20,17),  0.001108282612722],
  [(21,17),  0.001108282612722],
  [(22,17),  0.000004071402540],
  [(23,17), -0.000000017280860],
  [(24,17),  0.000004071402540],
  [(25,17), -0.000208321706093],
  [(26,17), -0.000000017280860],
  [(27,17),  0.000000022686778],
  [(28,17), -0.000000017280860],
  [(29,17),  0.001108282612722],
  [(30,17),  0.000004071402540],
  [(31,17), -0.000000017280860],
  [(32,17),  0.000004071402540],
  [(33,17),  0.001108282612722],
  [(34,17),  0.000004071402540],
  [(35,17), -0.000000017280860],
  [(36,17),  0.000004071402540],
  [(37,17),  0.000004071402540],
  [(38,17),  0.000000096993280],
  [(40,17),  0.000000096993280],
  [(41,17), -0.000000017280860],
  [(45,17),  0.000004071402540],
  [(46,17),  0.000000096993280],
  [(48,17),  0.000000096993280],
  [(49,17), -0.000173252573293],
  [(50,17),  0.000003381525327],
  [(51,17),  0.000000119680058],
  [(52,17),  0.000003381525327],
  [(53,17),  0.000003381525327],
  [(54,17),  0.000000096993280],
  [(56,17),  0.000000096993280],
  [(57,17),  0.000000119680058],
  [(61,17),  0.000003381525327],
  [(62,17),  0.000000096993280],
  [(64,17),  0.000000096993280],
   [(1,18),  0.000027931951489],
   [(2,18),  0.001108282612722],
   [(3,18),  0.000006267168743],
   [(4,18),  0.000003381525327],
   [(5,18),  0.000003495799467],
   [(6,18),  0.000006267168743],
   [(7,18),  0.000000468800853],
   [(8,18),  0.000000220174746],
   [(9,18),  0.000000220174746],
  [(10,18),  0.000000354526714],
  [(11,18),  0.000000007759462],
  [(13,18),  0.000003495799467],
  [(14,18),  0.000006267168743],
  [(15,18),  0.000000468800853],
  [(16,18),  0.000000220174746],
  [(17,18),  0.001108282612722],
  [(18,18),  0.994236020504560],
  [(19,18),  0.001073213479922],
  [(20,18), -0.000206125939889],
  [(21,18),  0.000006267168743],
  [(22,18),  0.001073213479922],
  [(23,18),  0.000000795777819],
  [(24,18),  0.000000354526714],
  [(25,18),  0.000000354526714],
  [(26,18), -0.000211597330814],
  [(27,18), -0.000000114274140],
  [(28,18),  0.000000030446240],
  [(29,18),  0.000006267168743],
  [(30,18),  0.001073213479922],
  [(31,18),  0.000000795777819],
  [(32,18),  0.000000354526714],
  [(33,18),  0.000006267168743],
  [(34,18),  0.001073213479922],
  [(35,18),  0.000000795777819],
  [(36,18),  0.000000354526714],
  [(37,18),  0.000000468800853],
  [(38,18),  0.000000795777819],
  [(40,18),  0.000000007759462],
  [(41,18),  0.000000007759462],
  [(42,18), -0.000000114274140],
  [(45,18),  0.000000468800853],
  [(46,18),  0.000000795777819],
  [(48,18),  0.000000007759462],
  [(49,18),  0.000003381525327],
  [(50,18), -0.000206125939889],
  [(51,18),  0.000000354526714],
  [(52,18),  0.000000242861523],
  [(53,18),  0.000000220174746],
  [(54,18),  0.000000354526714],
  [(55,18),  0.000000007759462],
  [(58,18),  0.000000030446240],
  [(61,18),  0.000000220174746],
  [(62,18),  0.000000354526714],
  [(63,18),  0.000000007759462],
   [(1,19),  0.000001195135140],
   [(2,19),  0.000004071402540],
   [(3,19),  0.001073213479922],
   [(4,19),  0.000004071402540],
   [(5,19),  0.000000096993280],
   [(6,19),  0.000000468800853],
   [(7,19),  0.000001044403926],
   [(8,19),  0.000000468800853],
  [(10,19),  0.000000032331093],
  [(11,19), -0.000000081943046],
  [(12,19),  0.000000032331093],
  [(13,19),  0.000000096993280],
  [(14,19),  0.000000468800853],
  [(15,19),  0.000001044403926],
  [(16,19),  0.000000468800853],
  [(17,19), -0.000208321706093],
  [(18,19),  0.001073213479922],
  [(19,19),  0.994214147055179],
  [(20,19),  0.001073213479922],
  [(21,19),  0.000000354526714],
  [(22,19),  0.000001044403926],
  [(23,19),  0.001069814673736],
  [(24,19),  0.000001044403926],
  [(25,19),  0.000000055017871],
  [(26,19), -0.000000081943046],
  [(27,19), -0.000211817505560],
  [(28,19), -0.000000081943046],
  [(29,19),  0.000000354526714],
  [(30,19),  0.000001044403926],
  [(31,19),  0.001069814673736],
  [(32,19),  0.000001044403926],
  [(33,19),  0.000000354526714],
  [(34,19),  0.000001044403926],
  [(35,19),  0.001069814673736],
  [(36,19),  0.000001044403926],
  [(37,19),  0.000000032331093],
  [(38,19),  0.000000032331093],
  [(39,19),  0.000000575603073],
  [(40,19),  0.000000032331093],
  [(43,19), -0.000000114274140],
  [(45,19),  0.000000032331093],
  [(46,19),  0.000000032331093],
  [(47,19),  0.000000575603073],
  [(48,19),  0.000000032331093],
  [(49,19),  0.000000119680058],
  [(50,19),  0.000000354526714],
  [(51,19), -0.000211348704706],
  [(52,19),  0.000000354526714],
  [(54,19),  0.000000032331093],
  [(55,19), -0.000000081943046],
  [(56,19),  0.000000032331093],
  [(59,19),  0.000000022686778],
  [(62,19),  0.000000032331093],
  [(63,19), -0.000000081943046],
  [(64,19),  0.000000032331093],
   [(1,20),  0.000027931951489],
   [(2,20),  0.000003381525327],
   [(3,20),  0.000006267168743],
   [(4,20),  0.001108282612722],
   [(5,20),  0.000003495799467],
   [(6,20),  0.000000220174746],
   [(7,20),  0.000000468800853],
   [(8,20),  0.000006267168743],
   [(9,20),  0.000000220174746],
  [(11,20),  0.000000007759462],
  [(12,20),  0.000000354526714],
  [(13,20),  0.000003495799467],
  [(14,20),  0.000000220174746],
  [(15,20),  0.000000468800853],
  [(16,20),  0.000006267168743],
  [(17,20),  0.001108282612722],
  [(18,20), -0.000206125939889],
  [(19,20),  0.001073213479922],
  [(20,20),  0.994236020504560],
  [(21,20),  0.000006267168743],
  [(22,20),  0.000000354526714],
  [(23,20),  0.000000795777819],
  [(24,20),  0.001073213479922],
  [(25,20),  0.000000354526714],
  [(26,20),  0.000000030446240],
  [(27,20), -0.000000114274140],
  [(28,20), -0.000211597330814],
  [(29,20),  0.000006267168743],
  [(30,20),  0.000000354526714],
  [(31,20),  0.000000795777819],
  [(32,20),  0.001073213479922],
  [(33,20),  0.000006267168743],
  [(34,20),  0.000000354526714],
  [(35,20),  0.000000795777819],
  [(36,20),  0.001073213479922],
  [(37,20),  0.000000468800853],
  [(38,20),  0.000000007759462],
  [(40,20),  0.000000795777819],
  [(41,20),  0.000000007759462],
  [(44,20), -0.000000114274140],
  [(45,20),  0.000000468800853],
  [(46,20),  0.000000007759462],
  [(48,20),  0.000000795777819],
  [(49,20),  0.000003381525327],
  [(50,20),  0.000000242861523],
  [(51,20),  0.000000354526714],
  [(52,20), -0.000206125939889],
  [(53,20),  0.000000220174746],
  [(55,20),  0.000000007759462],
  [(56,20),  0.000000354526714],
  [(60,20),  0.000000030446240],
  [(61,20),  0.000000220174746],
  [(63,20),  0.000000007759462],
  [(64,20),  0.000000354526714],
   [(1,21),  0.000027931951489],
   [(2,21),  0.000003495799467],
   [(3,21),  0.000000220174746],
   [(4,21),  0.000003495799467],
   [(5,21),  0.001108282612722],
   [(6,21),  0.000006267168743],
   [(7,21),  0.000000354526714],
   [(8,21),  0.000006267168743],
   [(9,21),  0.000006267168743],
  [(10,21),  0.000000468800853],
  [(11,21),  0.000000007759462],
  [(12,21),  0.000000468800853],
  [(13,21),  0.000003381525327],
  [(14,21),  0.000000220174746],
  [(16,21),  0.000000220174746],
  [(17,21),  0.001108282612722],
  [(18,21),  0.000006267168743],
  [(19,21),  0.000000354526714],
  [(20,21),  0.000006267168743],
  [(21,21),  0.994236020504560],
  [(22,21),  0.001073213479922],
  [(23,21), -0.000211597330814],
  [(24,21),  0.001073213479922],
  [(25,21),  0.001073213479922],
  [(26,21),  0.000000795777819],
  [(27,21), -0.000000114274140],
  [(28,21),  0.000000795777819],
  [(29,21), -0.000206125939889],
  [(30,21),  0.000000354526714],
  [(31,21),  0.000000030446240],
  [(32,21),  0.000000354526714],
  [(33,21),  0.000006267168743],
  [(34,21),  0.000000468800853],
  [(35,21),  0.000000007759462],
  [(36,21),  0.000000468800853],
  [(37,21),  0.001073213479922],
  [(38,21),  0.000000795777819],
  [(39,21), -0.000000114274140],
  [(40,21),  0.000000795777819],
  [(41,21),  0.000000795777819],
  [(45,21),  0.000000354526714],
  [(46,21),  0.000000007759462],
  [(48,21),  0.000000007759462],
  [(49,21),  0.000003381525327],
  [(50,21),  0.000000220174746],
  [(52,21),  0.000000220174746],
  [(53,21), -0.000206125939889],
  [(54,21),  0.000000354526714],
  [(55,21),  0.000000030446240],
  [(56,21),  0.000000354526714],
  [(57,21),  0.000000354526714],
  [(58,21),  0.000000007759462],
  [(60,21),  0.000000007759462],
  [(61,21),  0.000000242861523],
   [(1,22),  0.000001309409280],
   [(2,22),  0.000004071402540],
   [(3,22),  0.000000468800853],
   [(4,22),  0.000000096993280],
   [(5,22),  0.000004071402540],
   [(6,22),  0.001073213479922],
   [(7,22),  0.000001044403926],
   [(8,22),  0.000000354526714],
   [(9,22),  0.000000468800853],
  [(10,22),  0.000001044403926],
  [(11,22),  0.000000032331093],
  [(12,22),  0.000000032331093],
  [(13,22),  0.000000096993280],
  [(14,22),  0.000000354526714],
  [(15,22),  0.000000032331093],
  [(17,22),  0.000004071402540],
  [(18,22),  0.001073213479922],
  [(19,22),  0.000001044403926],
  [(20,22),  0.000000354526714],
  [(21,22),  0.001073213479922],
  [(22,22),  0.994212887656419],
  [(23,22),  0.001069814673736],
  [(24,22), -0.000211348704706],
  [(25,22),  0.000001044403926],
  [(26,22),  0.001069814673736],
  [(27,22),  0.000000575603073],
  [(28,22), -0.000000081943046],
  [(29,22),  0.000000354526714],
  [(30,22), -0.000211348704706],
  [(31,22), -0.000000081943046],
  [(32,22),  0.000000055017871],
  [(33,22),  0.000000468800853],
  [(34,22),  0.000001044403926],
  [(35,22),  0.000000032331093],
  [(36,22),  0.000000032331093],
  [(37,22),  0.000001044403926],
  [(38,22),  0.001069814673736],
  [(39,22),  0.000000575603073],
  [(40,22), -0.000000081943046],
  [(41,22),  0.000000032331093],
  [(42,22),  0.000000575603073],
  [(45,22),  0.000000032331093],
  [(46,22), -0.000000081943046],
  [(49,22),  0.000000096993280],
  [(50,22),  0.000000354526714],
  [(51,22),  0.000000032331093],
  [(53,22),  0.000000354526714],
  [(54,22), -0.000211348704706],
  [(55,22), -0.000000081943046],
  [(56,22),  0.000000055017871],
  [(57,22),  0.000000032331093],
  [(58,22), -0.000000081943046],
  [(62,22),  0.000000055017871],
   [(1,23),  0.000000015518925],
   [(2,23),  0.000000096993280],
   [(3,23),  0.000000795777819],
   [(4,23),  0.000000096993280],
   [(5,23), -0.000000017280860],
   [(6,23),  0.000000795777819],
   [(7,23),  0.001069814673736],
   [(8,23),  0.000000795777819],
   [(9,23),  0.000000007759462],
  [(10,23),  0.000000032331093],
  [(11,23),  0.000000583362535],
  [(12,23),  0.000000032331093],
  [(14,23),  0.000000007759462],
  [(15,23), -0.000000081943046],
  [(16,23),  0.000000007759462],
  [(17,23), -0.000000017280860],
  [(18,23),  0.000000795777819],
  [(19,23),  0.001069814673736],
  [(20,23),  0.000000795777819],
  [(21,23), -0.000211597330814],
  [(22,23),  0.001069814673736],
  [(23,23),  0.994214460147854],
  [(24,23),  0.001069814673736],
  [(25,23), -0.000000081943046],
  [(26,23),  0.000000583362535],
  [(27,23),  0.001069717680456],
  [(28,23),  0.000000583362535],
  [(29,23),  0.000000030446240],
  [(30,23), -0.000000081943046],
  [(31,23), -0.000211809746097],
  [(32,23), -0.000000081943046],
  [(33,23),  0.000000007759462],
  [(34,23),  0.000000032331093],
  [(35,23),  0.000000583362535],
  [(36,23),  0.000000032331093],
  [(37,23), -0.000000081943046],
  [(38,23),  0.000000583362535],
  [(39,23),  0.001069717680456],
  [(40,23),  0.000000583362535],
  [(41,23),  0.000000002586487],
  [(43,23),  0.000000575603073],
  [(46,23),  0.000000002586487],
  [(47,23), -0.000000114274140],
  [(48,23),  0.000000002586487],
  [(50,23),  0.000000007759462],
  [(51,23), -0.000000081943046],
  [(52,23),  0.000000007759462],
  [(53,23),  0.000000030446240],
  [(54,23), -0.000000081943046],
  [(55,23), -0.000211809746097],
  [(56,23), -0.000000081943046],
  [(58,23),  0.000000002586487],
  [(59,23), -0.000000114274140],
  [(60,23),  0.000000002586487],
  [(63,23),  0.000000025273265],
   [(1,24),  0.000001309409280],
   [(2,24),  0.000000096993280],
   [(3,24),  0.000000468800853],
   [(4,24),  0.000004071402540],
   [(5,24),  0.000004071402540],
   [(6,24),  0.000000354526714],
   [(7,24),  0.000001044403926],
   [(8,24),  0.001073213479922],
   [(9,24),  0.000000468800853],
  [(10,24),  0.000000032331093],
  [(11,24),  0.000000032331093],
  [(12,24),  0.000001044403926],
  [(13,24),  0.000000096993280],
  [(15,24),  0.000000032331093],
  [(16,24),  0.000000354526714],
  [(17,24),  0.000004071402540],
  [(18,24),  0.000000354526714],
  [(19,24),  0.000001044403926],
  [(20,24),  0.001073213479922],
  [(21,24),  0.001073213479922],
  [(22,24), -0.000211348704706],
  [(23,24),  0.001069814673736],
  [(24,24),  0.994212887656419],
  [(25,24),  0.000001044403926],
  [(26,24), -0.000000081943046],
  [(27,24),  0.000000575603073],
  [(28,24),  0.001069814673736],
  [(29,24),  0.000000354526714],
  [(30,24),  0.000000055017871],
  [(31,24), -0.000000081943046],
  [(32,24), -0.000211348704706],
  [(33,24),  0.000000468800853],
  [(34,24),  0.000000032331093],
  [(35,24),  0.000000032331093],
  [(36,24),  0.000001044403926],
  [(37,24),  0.000001044403926],
  [(38,24), -0.000000081943046],
  [(39,24),  0.000000575603073],
  [(40,24),  0.001069814673736],
  [(41,24),  0.000000032331093],
  [(44,24),  0.000000575603073],
  [(45,24),  0.000000032331093],
  [(48,24), -0.000000081943046],
  [(49,24),  0.000000096993280],
  [(51,24),  0.000000032331093],
  [(52,24),  0.000000354526714],
  [(53,24),  0.000000354526714],
  [(54,24),  0.000000055017871],
  [(55,24), -0.000000081943046],
  [(56,24), -0.000211348704706],
  [(57,24),  0.000000032331093],
  [(60,24), -0.000000081943046],
  [(64,24),  0.000000055017871],
   [(1,25),  0.000001195135140],
   [(2,25),  0.000000096993280],
   [(4,25),  0.000000096993280],
   [(5,25),  0.000004071402540],
   [(6,25),  0.000000468800853],
   [(7,25),  0.000000032331093],
   [(8,25),  0.000000468800853],
   [(9,25),  0.001073213479922],
  [(10,25),  0.000001044403926],
  [(11,25), -0.000000081943046],
  [(12,25),  0.000001044403926],
  [(13,25),  0.000004071402540],
  [(14,25),  0.000000468800853],
  [(15,25),  0.000000032331093],
  [(16,25),  0.000000468800853],
  [(17,25), -0.000208321706093],
  [(18,25),  0.000000354526714],
  [(19,25),  0.000000055017871],
  [(20,25),  0.000000354526714],
  [(21,25),  0.001073213479922],
  [(22,25),  0.000001044403926],
  [(23,25), -0.000000081943046],
  [(24,25),  0.000001044403926],
  [(25,25),  0.994214147055179],
  [(26,25),  0.001069814673736],
  [(27,25), -0.000211817505560],
  [(28,25),  0.001069814673736],
  [(29,25),  0.001073213479922],
  [(30,25),  0.000001044403926],
  [(31,25), -0.000000081943046],
  [(32,25),  0.000001044403926],
  [(33,25),  0.000000354526714],
  [(34,25),  0.000000032331093],
  [(36,25),  0.000000032331093],
  [(37,25),  0.000001044403926],
  [(38,25),  0.000000032331093],
  [(40,25),  0.000000032331093],
  [(41,25),  0.001069814673736],
  [(42,25),  0.000000575603073],
  [(43,25), -0.000000114274140],
  [(44,25),  0.000000575603073],
  [(45,25),  0.000001044403926],
  [(46,25),  0.000000032331093],
  [(48,25),  0.000000032331093],
  [(49,25),  0.000000119680058],
  [(53,25),  0.000000354526714],
  [(54,25),  0.000000032331093],
  [(56,25),  0.000000032331093],
  [(57,25), -0.000211348704706],
  [(58,25), -0.000000081943046],
  [(59,25),  0.000000022686778],
  [(60,25), -0.000000081943046],
  [(61,25),  0.000000354526714],
  [(62,25),  0.000000032331093],
  [(64,25),  0.000000032331093],
   [(1,26),  0.000000015518925],
   [(2,26), -0.000000017280860],
   [(3,26),  0.000000007759462],
   [(5,26),  0.000000096993280],
   [(6,26),  0.000000795777819],
   [(7,26),  0.000000032331093],
   [(8,26),  0.000000007759462],
   [(9,26),  0.000000795777819],
  [(10,26),  0.001069814673736],
  [(11,26),  0.000000583362535],
  [(12,26), -0.000000081943046],
  [(13,26),  0.000000096993280],
  [(14,26),  0.000000795777819],
  [(15,26),  0.000000032331093],
  [(16,26),  0.000000007759462],
  [(17,26), -0.000000017280860],
  [(18,26), -0.000211597330814],
  [(19,26), -0.000000081943046],
  [(20,26),  0.000000030446240],
  [(21,26),  0.000000795777819],
  [(22,26),  0.001069814673736],
  [(23,26),  0.000000583362535],
  [(24,26), -0.000000081943046],
  [(25,26),  0.001069814673736],
  [(26,26),  0.994214460147854],
  [(27,26),  0.001069717680456],
  [(28,26), -0.000211809746097],
  [(29,26),  0.000000795777819],
  [(30,26),  0.001069814673736],
  [(31,26),  0.000000583362535],
  [(32,26), -0.000000081943046],
  [(33,26),  0.000000007759462],
  [(34,26), -0.000000081943046],
  [(35,26),  0.000000002586487],
  [(37,26),  0.000000032331093],
  [(38,26),  0.000000583362535],
  [(40,26),  0.000000002586487],
  [(41,26),  0.000000583362535],
  [(42,26),  0.001069717680456],
  [(43,26),  0.000000575603073],
  [(44,26), -0.000000114274140],
  [(45,26),  0.000000032331093],
  [(46,26),  0.000000583362535],
  [(48,26),  0.000000002586487],
  [(50,26),  0.000000030446240],
  [(53,26),  0.000000007759462],
  [(54,26), -0.000000081943046],
  [(55,26),  0.000000002586487],
  [(57,26), -0.000000081943046],
  [(58,26), -0.000211809746097],
  [(59,26), -0.000000114274140],
  [(60,26),  0.000000025273265],
  [(61,26),  0.000000007759462],
  [(62,26), -0.000000081943046],
  [(63,26),  0.000000002586487],
   [(3,27), -0.000000114274140],
   [(7,27),  0.000000575603073],
   [(9,27), -0.000000114274140],
  [(10,27),  0.000000575603073],
  [(11,27),  0.001069717680456],
  [(12,27),  0.000000575603073],
  [(15,27),  0.000000575603073],
  [(17,27),  0.000000022686778],
  [(18,27), -0.000000114274140],
  [(19,27), -0.000211817505560],
  [(20,27), -0.000000114274140],
  [(21,27), -0.000000114274140],
  [(22,27),  0.000000575603073],
  [(23,27),  0.001069717680456],
  [(24,27),  0.000000575603073],
  [(25,27), -0.000211817505560],
  [(26,27),  0.001069717680456],
  [(27,27),  0.994216021328346],
  [(28,27),  0.001069717680456],
  [(29,27), -0.000000114274140],
  [(30,27),  0.000000575603073],
  [(31,27),  0.001069717680456],
  [(32,27),  0.000000575603073],
  [(35,27), -0.000000114274140],
  [(39,27),  0.000000575603073],
  [(41,27), -0.000000114274140],
  [(42,27),  0.000000575603073],
  [(43,27),  0.001069717680456],
  [(44,27),  0.000000575603073],
  [(47,27),  0.000000575603073],
  [(51,27),  0.000000022686778],
  [(55,27), -0.000000114274140],
  [(57,27),  0.000000022686778],
  [(58,27), -0.000000114274140],
  [(59,27), -0.000211817505560],
  [(60,27), -0.000000114274140],
  [(63,27), -0.000000114274140],
   [(1,28),  0.000000015518925],
   [(3,28),  0.000000007759462],
   [(4,28), -0.000000017280860],
   [(5,28),  0.000000096993280],
   [(6,28),  0.000000007759462],
   [(7,28),  0.000000032331093],
   [(8,28),  0.000000795777819],
   [(9,28),  0.000000795777819],
  [(10,28), -0.000000081943046],
  [(11,28),  0.000000583362535],
  [(12,28),  0.001069814673736],
  [(13,28),  0.000000096993280],
  [(14,28),  0.000000007759462],
  [(15,28),  0.000000032331093],
  [(16,28),  0.000000795777819],
  [(17,28), -0.000000017280860],
  [(18,28),  0.000000030446240],
  [(19,28), -0.000000081943046],
  [(20,28), -0.000211597330814],
  [(21,28),  0.000000795777819],
  [(22,28), -0.000000081943046],
  [(23,28),  0.000000583362535],
  [(24,28),  0.001069814673736],
  [(25,28),  0.001069814673736],
  [(26,28), -0.000211809746097],
  [(27,28),  0.001069717680456],
  [(28,28),  0.994214460147854],
  [(29,28),  0.000000795777819],
  [(30,28), -0.000000081943046],
  [(31,28),  0.000000583362535],
  [(32,28),  0.001069814673736],
  [(33,28),  0.000000007759462],
  [(35,28),  0.000000002586487],
  [(36,28), -0.000000081943046],
  [(37,28),  0.000000032331093],
  [(38,28),  0.000000002586487],
  [(40,28),  0.000000583362535],
  [(41,28),  0.000000583362535],
  [(42,28), -0.000000114274140],
  [(43,28),  0.000000575603073],
  [(44,28),  0.001069717680456],
  [(45,28),  0.000000032331093],
  [(46,28),  0.000000002586487],
  [(48,28),  0.000000583362535],
  [(52,28),  0.000000030446240],
  [(53,28),  0.000000007759462],
  [(55,28),  0.000000002586487],
  [(56,28), -0.000000081943046],
  [(57,28), -0.000000081943046],
  [(58,28),  0.000000025273265],
  [(59,28), -0.000000114274140],
  [(60,28), -0.000211809746097],
  [(61,28),  0.000000007759462],
  [(63,28),  0.000000002586487],
  [(64,28), -0.000000081943046],
   [(1,29),  0.000027931951489],
   [(2,29),  0.000003495799467],
   [(3,29),  0.000000220174746],
   [(4,29),  0.000003495799467],
   [(5,29),  0.000003381525327],
   [(6,29),  0.000000220174746],
   [(8,29),  0.000000220174746],
   [(9,29),  0.000006267168743],
  [(10,29),  0.000000468800853],
  [(11,29),  0.000000007759462],
  [(12,29),  0.000000468800853],
  [(13,29),  0.001108282612722],
  [(14,29),  0.000006267168743],
  [(15,29),  0.000000354526714],
  [(16,29),  0.000006267168743],
  [(17,29),  0.001108282612722],
  [(18,29),  0.000006267168743],
  [(19,29),  0.000000354526714],
  [(20,29),  0.000006267168743],
  [(21,29), -0.000206125939889],
  [(22,29),  0.000000354526714],
  [(23,29),  0.000000030446240],
  [(24,29),  0.000000354526714],
  [(25,29),  0.001073213479922],
  [(26,29),  0.000000795777819],
  [(27,29), -0.000000114274140],
  [(28,29),  0.000000795777819],
  [(29,29),  0.994236020504560],
  [(30,29),  0.001073213479922],
  [(31,29), -0.000211597330814],
  [(32,29),  0.001073213479922],
  [(33,29),  0.000006267168743],
  [(34,29),  0.000000468800853],
  [(35,29),  0.000000007759462],
  [(36,29),  0.000000468800853],
  [(37,29),  0.000000354526714],
  [(38,29),  0.000000007759462],
  [(40,29),  0.000000007759462],
  [(41,29),  0.000000795777819],
  [(45,29),  0.001073213479922],
  [(46,29),  0.000000795777819],
  [(47,29), -0.000000114274140],
  [(48,29),  0.000000795777819],
  [(49,29),  0.000003381525327],
  [(50,29),  0.000000220174746],
  [(52,29),  0.000000220174746],
  [(53,29),  0.000000242861523],
  [(57,29),  0.000000354526714],
  [(58,29),  0.000000007759462],
  [(60,29),  0.000000007759462],
  [(61,29), -0.000206125939889],
  [(62,29),  0.000000354526714],
  [(63,29),  0.000000030446240],
  [(64,29),  0.000000354526714],
   [(1,30),  0.000001309409280],
   [(2,30),  0.000004071402540],
   [(3,30),  0.000000468800853],
   [(4,30),  0.000000096993280],
   [(5,30),  0.000000096993280],
   [(6,30),  0.000000354526714],
   [(7,30),  0.000000032331093],
   [(9,30),  0.000000468800853],
  [(10,30),  0.000001044403926],
  [(11,30),  0.000000032331093],
  [(12,30),  0.000000032331093],
  [(13,30),  0.000004071402540],
  [(14,30),  0.001073213479922],
  [(15,30),  0.000001044403926],
  [(16,30),  0.000000354526714],
  [(17,30),  0.000004071402540],
  [(18,30),  0.001073213479922],
  [(19,30),  0.000001044403926],
  [(20,30),  0.000000354526714],
  [(21,30),  0.000000354526714],
  [(22,30), -0.000211348704706],
  [(23,30), -0.000000081943046],
  [(24,30),  0.000000055017871],
  [(25,30),  0.000001044403926],
  [(26,30),  0.001069814673736],
  [(27,30),  0.000000575603073],
  [(28,30), -0.000000081943046],
  [(29,30),  0.001073213479922],
  [(30,30),  0.994212887656419],
  [(31,30),  0.001069814673736],
  [(32,30), -0.000211348704706],
  [(33,30),  0.000000468800853],
  [(34,30),  0.000001044403926],
  [(35,30),  0.000000032331093],
  [(36,30),  0.000000032331093],
  [(37,30),  0.000000032331093],
  [(38,30), -0.000000081943046],
  [(41,30),  0.000000032331093],
  [(42,30),  0.000000575603073],
  [(45,30),  0.000001044403926],
  [(46,30),  0.001069814673736],
  [(47,30),  0.000000575603073],
  [(48,30), -0.000000081943046],
  [(49,30),  0.000000096993280],
  [(50,30),  0.000000354526714],
  [(51,30),  0.000000032331093],
  [(54,30),  0.000000055017871],
  [(57,30),  0.000000032331093],
  [(58,30), -0.000000081943046],
  [(61,30),  0.000000354526714],
  [(62,30), -0.000211348704706],
  [(63,30), -0.000000081943046],
  [(64,30),  0.000000055017871],
   [(1,31),  0.000000015518925],
   [(2,31),  0.000000096993280],
   [(3,31),  0.000000795777819],
   [(4,31),  0.000000096993280],
   [(6,31),  0.000000007759462],
   [(7,31), -0.000000081943046],
   [(8,31),  0.000000007759462],
   [(9,31),  0.000000007759462],
  [(10,31),  0.000000032331093],
  [(11,31),  0.000000583362535],
  [(12,31),  0.000000032331093],
  [(13,31), -0.000000017280860],
  [(14,31),  0.000000795777819],
  [(15,31),  0.001069814673736],
  [(16,31),  0.000000795777819],
  [(17,31), -0.000000017280860],
  [(18,31),  0.000000795777819],
  [(19,31),  0.001069814673736],
  [(20,31),  0.000000795777819],
  [(21,31),  0.000000030446240],
  [(22,31), -0.000000081943046],
  [(23,31), -0.000211809746097],
  [(24,31), -0.000000081943046],
  [(25,31), -0.000000081943046],
  [(26,31),  0.000000583362535],
  [(27,31),  0.001069717680456],
  [(28,31),  0.000000583362535],
  [(29,31), -0.000211597330814],
  [(30,31),  0.001069814673736],
  [(31,31),  0.994214460147854],
  [(32,31),  0.001069814673736],
  [(33,31),  0.000000007759462],
  [(34,31),  0.000000032331093],
  [(35,31),  0.000000583362535],
  [(36,31),  0.000000032331093],
  [(38,31),  0.000000002586487],
  [(39,31), -0.000000114274140],
  [(40,31),  0.000000002586487],
  [(41,31),  0.000000002586487],
  [(43,31),  0.000000575603073],
  [(45,31), -0.000000081943046],
  [(46,31),  0.000000583362535],
  [(47,31),  0.001069717680456],
  [(48,31),  0.000000583362535],
  [(50,31),  0.000000007759462],
  [(51,31), -0.000000081943046],
  [(52,31),  0.000000007759462],
  [(55,31),  0.000000025273265],
  [(58,31),  0.000000002586487],
  [(59,31), -0.000000114274140],
  [(60,31),  0.000000002586487],
  [(61,31),  0.000000030446240],
  [(62,31), -0.000000081943046],
  [(63,31), -0.000211809746097],
  [(64,31), -0.000000081943046],
   [(1,32),  0.000001309409280],
   [(2,32),  0.000000096993280],
   [(3,32),  0.000000468800853],
   [(4,32),  0.000004071402540],
   [(5,32),  0.000000096993280],
   [(7,32),  0.000000032331093],
   [(8,32),  0.000000354526714],
   [(9,32),  0.000000468800853],
  [(10,32),  0.000000032331093],
  [(11,32),  0.000000032331093],
  [(12,32),  0.000001044403926],
  [(13,32),  0.000004071402540],
  [(14,32),  0.000000354526714],
  [(15,32),  0.000001044403926],
  [(16,32),  0.001073213479922],
  [(17,32),  0.000004071402540],
  [(18,32),  0.000000354526714],
  [(19,32),  0.000001044403926],
  [(20,32),  0.001073213479922],
  [(21,32),  0.000000354526714],
  [(22,32),  0.000000055017871],
  [(23,32), -0.000000081943046],
  [(24,32), -0.000211348704706],
  [(25,32),  0.000001044403926],
  [(26,32), -0.000000081943046],
  [(27,32),  0.000000575603073],
  [(28,32),  0.001069814673736],
  [(29,32),  0.001073213479922],
  [(30,32), -0.000211348704706],
  [(31,32),  0.001069814673736],
  [(32,32),  0.994212887656419],
  [(33,32),  0.000000468800853],
  [(34,32),  0.000000032331093],
  [(35,32),  0.000000032331093],
  [(36,32),  0.000001044403926],
  [(37,32),  0.000000032331093],
  [(40,32), -0.000000081943046],
  [(41,32),  0.000000032331093],
  [(44,32),  0.000000575603073],
  [(45,32),  0.000001044403926],
  [(46,32), -0.000000081943046],
  [(47,32),  0.000000575603073],
  [(48,32),  0.001069814673736],
  [(49,32),  0.000000096993280],
  [(51,32),  0.000000032331093],
  [(52,32),  0.000000354526714],
  [(56,32),  0.000000055017871],
  [(57,32),  0.000000032331093],
  [(60,32), -0.000000081943046],
  [(61,32),  0.000000354526714],
  [(62,32),  0.000000055017871],
  [(63,32), -0.000000081943046],
  [(64,32), -0.000211348704706],
   [(1,33), -0.000184461157144],
   [(2,33),  0.000003381525327],
   [(3,33),  0.000000242861523],
   [(4,33),  0.000003381525327],
   [(5,33),  0.000003381525327],
   [(6,33),  0.000000220174746],
   [(8,33),  0.000000220174746],
   [(9,33),  0.000000242861523],
  [(13,33),  0.000003381525327],
  [(14,33),  0.000000220174746],
  [(16,33),  0.000000220174746],
  [(17,33),  0.001108282612722],
  [(18,33),  0.000006267168743],
  [(19,33),  0.000000354526714],
  [(20,33),  0.000006267168743],
  [(21,33),  0.000006267168743],
  [(22,33),  0.000000468800853],
  [(23,33),  0.000000007759462],
  [(24,33),  0.000000468800853],
  [(25,33),  0.000000354526714],
  [(26,33),  0.000000007759462],
  [(28,33),  0.000000007759462],
  [(29,33),  0.000006267168743],
  [(30,33),  0.000000468800853],
  [(31,33),  0.000000007759462],
  [(32,33),  0.000000468800853],
  [(33,33),  0.994237776751864],
  [(34,33),  0.001073213479922],
  [(35,33), -0.000211597330814],
  [(36,33),  0.001073213479922],
  [(37,33),  0.001073213479922],
  [(38,33),  0.000000795777819],
  [(39,33), -0.000000114274140],
  [(40,33),  0.000000795777819],
  [(41,33), -0.000211597330814],
  [(42,33), -0.000000114274140],
  [(43,33),  0.000000022686778],
  [(44,33), -0.000000114274140],
  [(45,33),  0.001073213479922],
  [(46,33),  0.000000795777819],
  [(47,33), -0.000000114274140],
  [(48,33),  0.000000795777819],
  [(49,33),  0.001108282612722],
  [(50,33),  0.000006267168743],
  [(51,33),  0.000000354526714],
  [(52,33),  0.000006267168743],
  [(53,33),  0.000006267168743],
  [(54,33),  0.000000468800853],
  [(55,33),  0.000000007759462],
  [(56,33),  0.000000468800853],
  [(57,33),  0.000000354526714],
  [(58,33),  0.000000007759462],
  [(60,33),  0.000000007759462],
  [(61,33),  0.000006267168743],
  [(62,33),  0.000000468800853],
  [(63,33),  0.000000007759462],
  [(64,33),  0.000000468800853],
   [(1,34),  0.000001195135140],
   [(2,34), -0.000208321706093],
   [(3,34),  0.000000354526714],
   [(4,34),  0.000000119680058],
   [(5,34),  0.000000096993280],
   [(6,34),  0.000000354526714],
   [(7,34),  0.000000032331093],
  [(10,34),  0.000000055017871],
  [(13,34),  0.000000096993280],
  [(14,34),  0.000000354526714],
  [(15,34),  0.000000032331093],
  [(17,34),  0.000004071402540],
  [(18,34),  0.001073213479922],
  [(19,34),  0.000001044403926],
  [(20,34),  0.000000354526714],
  [(21,34),  0.000000468800853],
  [(22,34),  0.000001044403926],
  [(23,34),  0.000000032331093],
  [(24,34),  0.000000032331093],
  [(25,34),  0.000000032331093],
  [(26,34), -0.000000081943046],
  [(29,34),  0.000000468800853],
  [(30,34),  0.000001044403926],
  [(31,34),  0.000000032331093],
  [(32,34),  0.000000032331093],
  [(33,34),  0.001073213479922],
  [(34,34),  0.994214147055179],
  [(35,34),  0.001069814673736],
  [(36,34), -0.000211348704706],
  [(37,34),  0.000001044403926],
  [(38,34),  0.001069814673736],
  [(39,34),  0.000000575603073],
  [(40,34), -0.000000081943046],
  [(41,34), -0.000000081943046],
  [(42,34), -0.000211817505560],
  [(43,34), -0.000000114274140],
  [(44,34),  0.000000022686778],
  [(45,34),  0.000001044403926],
  [(46,34),  0.001069814673736],
  [(47,34),  0.000000575603073],
  [(48,34), -0.000000081943046],
  [(49,34),  0.000004071402540],
  [(50,34),  0.001073213479922],
  [(51,34),  0.000001044403926],
  [(52,34),  0.000000354526714],
  [(53,34),  0.000000468800853],
  [(54,34),  0.000001044403926],
  [(55,34),  0.000000032331093],
  [(56,34),  0.000000032331093],
  [(57,34),  0.000000032331093],
  [(58,34), -0.000000081943046],
  [(61,34),  0.000000468800853],
  [(62,34),  0.000001044403926],
  [(63,34),  0.000000032331093],
  [(64,34),  0.000000032331093],
   [(1,35),  0.000000038205702],
   [(2,35), -0.000000017280860],
   [(3,35), -0.000211597330814],
   [(4,35), -0.000000017280860],
   [(6,35),  0.000000007759462],
   [(7,35), -0.000000081943046],
   [(8,35),  0.000000007759462],
  [(11,35),  0.000000025273265],
  [(14,35),  0.000000007759462],
  [(15,35), -0.000000081943046],
  [(16,35),  0.000000007759462],
  [(17,35), -0.000000017280860],
  [(18,35),  0.000000795777819],
  [(19,35),  0.001069814673736],
  [(20,35),  0.000000795777819],
  [(21,35),  0.000000007759462],
  [(22,35),  0.000000032331093],
  [(23,35),  0.000000583362535],
  [(24,35),  0.000000032331093],
  [(26,35),  0.000000002586487],
  [(27,35), -0.000000114274140],
  [(28,35),  0.000000002586487],
  [(29,35),  0.000000007759462],
  [(30,35),  0.000000032331093],
  [(31,35),  0.000000583362535],
  [(32,35),  0.000000032331093],
  [(33,35), -0.000211597330814],
  [(34,35),  0.001069814673736],
  [(35,35),  0.994215228867032],
  [(36,35),  0.001069814673736],
  [(37,35), -0.000000081943046],
  [(38,35),  0.000000583362535],
  [(39,35),  0.001069717680456],
  [(40,35),  0.000000583362535],
  [(41,35),  0.000000025273265],
  [(42,35), -0.000000114274140],
  [(43,35), -0.000211817505560],
  [(44,35), -0.000000114274140],
  [(45,35), -0.000000081943046],
  [(46,35),  0.000000583362535],
  [(47,35),  0.001069717680456],
  [(48,35),  0.000000583362535],
  [(49,35), -0.000000017280860],
  [(50,35),  0.000000795777819],
  [(51,35),  0.001069814673736],
  [(52,35),  0.000000795777819],
  [(53,35),  0.000000007759462],
  [(54,35),  0.000000032331093],
  [(55,35),  0.000000583362535],
  [(56,35),  0.000000032331093],
  [(58,35),  0.000000002586487],
  [(59,35), -0.000000114274140],
  [(60,35),  0.000000002586487],
  [(61,35),  0.000000007759462],
  [(62,35),  0.000000032331093],
  [(63,35),  0.000000583362535],
  [(64,35),  0.000000032331093],
   [(1,36),  0.000001195135140],
   [(2,36),  0.000000119680058],
   [(3,36),  0.000000354526714],
   [(4,36), -0.000208321706093],
   [(5,36),  0.000000096993280],
   [(7,36),  0.000000032331093],
   [(8,36),  0.000000354526714],
  [(12,36),  0.000000055017871],
  [(13,36),  0.000000096993280],
  [(15,36),  0.000000032331093],
  [(16,36),  0.000000354526714],
  [(17,36),  0.000004071402540],
  [(18,36),  0.000000354526714],
  [(19,36),  0.000001044403926],
  [(20,36),  0.001073213479922],
  [(21,36),  0.000000468800853],
  [(22,36),  0.000000032331093],
  [(23,36),  0.000000032331093],
  [(24,36),  0.000001044403926],
  [(25,36),  0.000000032331093],
  [(28,36), -0.000000081943046],
  [(29,36),  0.000000468800853],
  [(30,36),  0.000000032331093],
  [(31,36),  0.000000032331093],
  [(32,36),  0.000001044403926],
  [(33,36),  0.001073213479922],
  [(34,36), -0.000211348704706],
  [(35,36),  0.001069814673736],
  [(36,36),  0.994214147055179],
  [(37,36),  0.000001044403926],
  [(38,36), -0.000000081943046],
  [(39,36),  0.000000575603073],
  [(40,36),  0.001069814673736],
  [(41,36), -0.000000081943046],
  [(42,36),  0.000000022686778],
  [(43,36), -0.000000114274140],
  [(44,36), -0.000211817505560],
  [(45,36),  0.000001044403926],
  [(46,36), -0.000000081943046],
  [(47,36),  0.000000575603073],
  [(48,36),  0.001069814673736],
  [(49,36),  0.000004071402540],
  [(50,36),  0.000000354526714],
  [(51,36),  0.000001044403926],
  [(52,36),  0.001073213479922],
  [(53,36),  0.000000468800853],
  [(54,36),  0.000000032331093],
  [(55,36),  0.000000032331093],
  [(56,36),  0.000001044403926],
  [(57,36),  0.000000032331093],
  [(60,36), -0.000000081943046],
  [(61,36),  0.000000468800853],
  [(62,36),  0.000000032331093],
  [(63,36),  0.000000032331093],
  [(64,36),  0.000001044403926],
   [(1,37),  0.000001195135140],
   [(2,37),  0.000000096993280],
   [(4,37),  0.000000096993280],
   [(5,37), -0.000208321706093],
   [(6,37),  0.000000354526714],
   [(7,37),  0.000000055017871],
   [(8,37),  0.000000354526714],
   [(9,37),  0.000000354526714],
  [(10,37),  0.000000032331093],
  [(12,37),  0.000000032331093],
  [(13,37),  0.000000119680058],
  [(17,37),  0.000004071402540],
  [(18,37),  0.000000468800853],
  [(19,37),  0.000000032331093],
  [(20,37),  0.000000468800853],
  [(21,37),  0.001073213479922],
  [(22,37),  0.000001044403926],
  [(23,37), -0.000000081943046],
  [(24,37),  0.000001044403926],
  [(25,37),  0.000001044403926],
  [(26,37),  0.000000032331093],
  [(28,37),  0.000000032331093],
  [(29,37),  0.000000354526714],
  [(30,37),  0.000000032331093],
  [(32,37),  0.000000032331093],
  [(33,37),  0.001073213479922],
  [(34,37),  0.000001044403926],
  [(35,37), -0.000000081943046],
  [(36,37),  0.000001044403926],
  [(37,37),  0.994214147055179],
  [(38,37),  0.001069814673736],
  [(39,37), -0.000211817505560],
  [(40,37),  0.001069814673736],
  [(41,37),  0.001069814673736],
  [(42,37),  0.000000575603073],
  [(43,37), -0.000000114274140],
  [(44,37),  0.000000575603073],
  [(45,37), -0.000211348704706],
  [(46,37), -0.000000081943046],
  [(47,37),  0.000000022686778],
  [(48,37), -0.000000081943046],
  [(49,37),  0.000004071402540],
  [(50,37),  0.000000468800853],
  [(51,37),  0.000000032331093],
  [(52,37),  0.000000468800853],
  [(53,37),  0.001073213479922],
  [(54,37),  0.000001044403926],
  [(55,37), -0.000000081943046],
  [(56,37),  0.000001044403926],
  [(57,37),  0.000001044403926],
  [(58,37),  0.000000032331093],
  [(60,37),  0.000000032331093],
  [(61,37),  0.000000354526714],
  [(62,37),  0.000000032331093],
  [(64,37),  0.000000032331093],
   [(1,38),  0.000000015518925],
   [(2,38), -0.000000017280860],
   [(3,38),  0.000000007759462],
   [(5,38), -0.000000017280860],
   [(6,38), -0.000211597330814],
   [(7,38), -0.000000081943046],
   [(8,38),  0.000000030446240],
   [(9,38),  0.000000007759462],
  [(10,38), -0.000000081943046],
  [(11,38),  0.000000002586487],
  [(14,38),  0.000000030446240],
  [(17,38),  0.000000096993280],
  [(18,38),  0.000000795777819],
  [(19,38),  0.000000032331093],
  [(20,38),  0.000000007759462],
  [(21,38),  0.000000795777819],
  [(22,38),  0.001069814673736],
  [(23,38),  0.000000583362535],
  [(24,38), -0.000000081943046],
  [(25,38),  0.000000032331093],
  [(26,38),  0.000000583362535],
  [(28,38),  0.000000002586487],
  [(29,38),  0.000000007759462],
  [(30,38), -0.000000081943046],
  [(31,38),  0.000000002586487],
  [(33,38),  0.000000795777819],
  [(34,38),  0.001069814673736],
  [(35,38),  0.000000583362535],
  [(36,38), -0.000000081943046],
  [(37,38),  0.001069814673736],
  [(38,38),  0.994214460147854],
  [(39,38),  0.001069717680456],
  [(40,38), -0.000211809746097],
  [(41,38),  0.000000583362535],
  [(42,38),  0.001069717680456],
  [(43,38),  0.000000575603073],
  [(44,38), -0.000000114274140],
  [(45,38), -0.000000081943046],
  [(46,38), -0.000211809746097],
  [(47,38), -0.000000114274140],
  [(48,38),  0.000000025273265],
  [(49,38),  0.000000096993280],
  [(50,38),  0.000000795777819],
  [(51,38),  0.000000032331093],
  [(52,38),  0.000000007759462],
  [(53,38),  0.000000795777819],
  [(54,38),  0.001069814673736],
  [(55,38),  0.000000583362535],
  [(56,38), -0.000000081943046],
  [(57,38),  0.000000032331093],
  [(58,38),  0.000000583362535],
  [(60,38),  0.000000002586487],
  [(61,38),  0.000000007759462],
  [(62,38), -0.000000081943046],
  [(63,38),  0.000000002586487],
   [(3,39), -0.000000114274140],
   [(5,39),  0.000000022686778],
   [(6,39), -0.000000114274140],
   [(7,39), -0.000211817505560],
   [(8,39), -0.000000114274140],
  [(11,39), -0.000000114274140],
  [(15,39),  0.000000022686778],
  [(19,39),  0.000000575603073],
  [(21,39), -0.000000114274140],
  [(22,39),  0.000000575603073],
  [(23,39),  0.001069717680456],
  [(24,39),  0.000000575603073],
  [(27,39),  0.000000575603073],
  [(31,39), -0.000000114274140],
  [(33,39), -0.000000114274140],
  [(34,39),  0.000000575603073],
  [(35,39),  0.001069717680456],
  [(36,39),  0.000000575603073],
  [(37,39), -0.000211817505560],
  [(38,39),  0.001069717680456],
  [(39,39),  0.994216021328346],
  [(40,39),  0.001069717680456],
  [(41,39), -0.000000114274140],
  [(42,39),  0.000000575603073],
  [(43,39),  0.001069717680456],
  [(44,39),  0.000000575603073],
  [(45,39),  0.000000022686778],
  [(46,39), -0.000000114274140],
  [(47,39), -0.000211817505560],
  [(48,39), -0.000000114274140],
  [(51,39),  0.000000575603073],
  [(53,39), -0.000000114274140],
  [(54,39),  0.000000575603073],
  [(55,39),  0.001069717680456],
  [(56,39),  0.000000575603073],
  [(59,39),  0.000000575603073],
  [(63,39), -0.000000114274140],
   [(1,40),  0.000000015518925],
   [(3,40),  0.000000007759462],
   [(4,40), -0.000000017280860],
   [(5,40), -0.000000017280860],
   [(6,40),  0.000000030446240],
   [(7,40), -0.000000081943046],
   [(8,40), -0.000211597330814],
   [(9,40),  0.000000007759462],
  [(11,40),  0.000000002586487],
  [(12,40), -0.000000081943046],
  [(16,40),  0.000000030446240],
  [(17,40),  0.000000096993280],
  [(18,40),  0.000000007759462],
  [(19,40),  0.000000032331093],
  [(20,40),  0.000000795777819],
  [(21,40),  0.000000795777819],
  [(22,40), -0.000000081943046],
  [(23,40),  0.000000583362535],
  [(24,40),  0.001069814673736],
  [(25,40),  0.000000032331093],
  [(26,40),  0.000000002586487],
  [(28,40),  0.000000583362535],
  [(29,40),  0.000000007759462],
  [(31,40),  0.000000002586487],
  [(32,40), -0.000000081943046],
  [(33,40),  0.000000795777819],
  [(34,40), -0.000000081943046],
  [(35,40),  0.000000583362535],
  [(36,40),  0.001069814673736],
  [(37,40),  0.001069814673736],
  [(38,40), -0.000211809746097],
  [(39,40),  0.001069717680456],
  [(40,40),  0.994214460147854],
  [(41,40),  0.000000583362535],
  [(42,40), -0.000000114274140],
  [(43,40),  0.000000575603073],
  [(44,40),  0.001069717680456],
  [(45,40), -0.000000081943046],
  [(46,40),  0.000000025273265],
  [(47,40), -0.000000114274140],
  [(48,40), -0.000211809746097],
  [(49,40),  0.000000096993280],
  [(50,40),  0.000000007759462],
  [(51,40),  0.000000032331093],
  [(52,40),  0.000000795777819],
  [(53,40),  0.000000795777819],
  [(54,40), -0.000000081943046],
  [(55,40),  0.000000583362535],
  [(56,40),  0.001069814673736],
  [(57,40),  0.000000032331093],
  [(58,40),  0.000000002586487],
  [(60,40),  0.000000583362535],
  [(61,40),  0.000000007759462],
  [(63,40),  0.000000002586487],
  [(64,40), -0.000000081943046],
   [(1,41),  0.000000038205702],
   [(5,41), -0.000000017280860],
   [(6,41),  0.000000007759462],
   [(8,41),  0.000000007759462],
   [(9,41), -0.000211597330814],
  [(10,41), -0.000000081943046],
  [(11,41),  0.000000025273265],
  [(12,41), -0.000000081943046],
  [(13,41), -0.000000017280860],
  [(14,41),  0.000000007759462],
  [(16,41),  0.000000007759462],
  [(17,41), -0.000000017280860],
  [(18,41),  0.000000007759462],
  [(20,41),  0.000000007759462],
  [(21,41),  0.000000795777819],
  [(22,41),  0.000000032331093],
  [(23,41),  0.000000002586487],
  [(24,41),  0.000000032331093],
  [(25,41),  0.001069814673736],
  [(26,41),  0.000000583362535],
  [(27,41), -0.000000114274140],
  [(28,41),  0.000000583362535],
  [(29,41),  0.000000795777819],
  [(30,41),  0.000000032331093],
  [(31,41),  0.000000002586487],
  [(32,41),  0.000000032331093],
  [(33,41), -0.000211597330814],
  [(34,41), -0.000000081943046],
  [(35,41),  0.000000025273265],
  [(36,41), -0.000000081943046],
  [(37,41),  0.001069814673736],
  [(38,41),  0.000000583362535],
  [(39,41), -0.000000114274140],
  [(40,41),  0.000000583362535],
  [(41,41),  0.994215228867032],
  [(42,41),  0.001069717680456],
  [(43,41), -0.000211817505560],
  [(44,41),  0.001069717680456],
  [(45,41),  0.001069814673736],
  [(46,41),  0.000000583362535],
  [(47,41), -0.000000114274140],
  [(48,41),  0.000000583362535],
  [(49,41), -0.000000017280860],
  [(50,41),  0.000000007759462],
  [(52,41),  0.000000007759462],
  [(53,41),  0.000000795777819],
  [(54,41),  0.000000032331093],
  [(55,41),  0.000000002586487],
  [(56,41),  0.000000032331093],
  [(57,41),  0.001069814673736],
  [(58,41),  0.000000583362535],
  [(59,41), -0.000000114274140],
  [(60,41),  0.000000583362535],
  [(61,41),  0.000000795777819],
  [(62,41),  0.000000032331093],
  [(63,41),  0.000000002586487],
  [(64,41),  0.000000032331093],
   [(2,42),  0.000000022686778],
   [(6,42), -0.000000114274140],
   [(9,42), -0.000000114274140],
  [(10,42), -0.000211817505560],
  [(11,42), -0.000000114274140],
  [(12,42),  0.000000022686778],
  [(14,42), -0.000000114274140],
  [(18,42), -0.000000114274140],
  [(22,42),  0.000000575603073],
  [(25,42),  0.000000575603073],
  [(26,42),  0.001069717680456],
  [(27,42),  0.000000575603073],
  [(28,42), -0.000000114274140],
  [(30,42),  0.000000575603073],
  [(33,42), -0.000000114274140],
  [(34,42), -0.000211817505560],
  [(35,42), -0.000000114274140],
  [(36,42),  0.000000022686778],
  [(37,42),  0.000000575603073],
  [(38,42),  0.001069717680456],
  [(39,42),  0.000000575603073],
  [(40,42), -0.000000114274140],
  [(41,42),  0.001069717680456],
  [(42,42),  0.994216021328346],
  [(43,42),  0.001069717680456],
  [(44,42), -0.000211817505560],
  [(45,42),  0.000000575603073],
  [(46,42),  0.001069717680456],
  [(47,42),  0.000000575603073],
  [(48,42), -0.000000114274140],
  [(50,42), -0.000000114274140],
  [(54,42),  0.000000575603073],
  [(57,42),  0.000000575603073],
  [(58,42),  0.001069717680456],
  [(59,42),  0.000000575603073],
  [(60,42), -0.000000114274140],
  [(62,42),  0.000000575603073],
   [(3,43),  0.000000022686778],
   [(7,43), -0.000000114274140],
   [(9,43),  0.000000022686778],
  [(10,43), -0.000000114274140],
  [(11,43), -0.000211817505560],
  [(12,43), -0.000000114274140],
  [(15,43), -0.000000114274140],
  [(19,43), -0.000000114274140],
  [(23,43),  0.000000575603073],
  [(25,43), -0.000000114274140],
  [(26,43),  0.000000575603073],
  [(27,43),  0.001069717680456],
  [(28,43),  0.000000575603073],
  [(31,43),  0.000000575603073],
  [(33,43),  0.000000022686778],
  [(34,43), -0.000000114274140],
  [(35,43), -0.000211817505560],
  [(36,43), -0.000000114274140],
  [(37,43), -0.000000114274140],
  [(38,43),  0.000000575603073],
  [(39,43),  0.001069717680456],
  [(40,43),  0.000000575603073],
  [(41,43), -0.000211817505560],
  [(42,43),  0.001069717680456],
  [(43,43),  0.994216810148551],
  [(44,43),  0.001069717680456],
  [(45,43), -0.000000114274140],
  [(46,43),  0.000000575603073],
  [(47,43),  0.001069717680456],
  [(48,43),  0.000000575603073],
  [(51,43), -0.000000114274140],
  [(55,43),  0.000000575603073],
  [(57,43), -0.000000114274140],
  [(58,43),  0.000000575603073],
  [(59,43),  0.001069717680456],
  [(60,43),  0.000000575603073],
  [(63,43),  0.000000575603073],
   [(4,44),  0.000000022686778],
   [(8,44), -0.000000114274140],
   [(9,44), -0.000000114274140],
  [(10,44),  0.000000022686778],
  [(11,44), -0.000000114274140],
  [(12,44), -0.000211817505560],
  [(16,44), -0.000000114274140],
  [(20,44), -0.000000114274140],
  [(24,44),  0.000000575603073],
  [(25,44),  0.000000575603073],
  [(26,44), -0.000000114274140],
  [(27,44),  0.000000575603073],
  [(28,44),  0.001069717680456],
  [(32,44),  0.000000575603073],
  [(33,44), -0.000000114274140],
  [(34,44),  0.000000022686778],
  [(35,44), -0.000000114274140],
  [(36,44), -0.000211817505560],
  [(37,44),  0.000000575603073],
  [(38,44), -0.000000114274140],
  [(39,44),  0.000000575603073],
  [(40,44),  0.001069717680456],
  [(41,44),  0.001069717680456],
  [(42,44), -0.000211817505560],
  [(43,44),  0.001069717680456],
  [(44,44),  0.994216021328346],
  [(45,44),  0.000000575603073],
  [(46,44), -0.000000114274140],
  [(47,44),  0.000000575603073],
  [(48,44),  0.001069717680456],
  [(52,44), -0.000000114274140],
  [(56,44),  0.000000575603073],
  [(57,44),  0.000000575603073],
  [(58,44), -0.000000114274140],
  [(59,44),  0.000000575603073],
  [(60,44),  0.001069717680456],
  [(64,44),  0.000000575603073],
   [(1,45),  0.000001195135140],
   [(2,45),  0.000000096993280],
   [(4,45),  0.000000096993280],
   [(5,45),  0.000000119680058],
   [(9,45),  0.000000354526714],
  [(10,45),  0.000000032331093],
  [(12,45),  0.000000032331093],
  [(13,45), -0.000208321706093],
  [(14,45),  0.000000354526714],
  [(15,45),  0.000000055017871],
  [(16,45),  0.000000354526714],
  [(17,45),  0.000004071402540],
  [(18,45),  0.000000468800853],
  [(19,45),  0.000000032331093],
  [(20,45),  0.000000468800853],
  [(21,45),  0.000000354526714],
  [(22,45),  0.000000032331093],
  [(24,45),  0.000000032331093],
  [(25,45),  0.000001044403926],
  [(26,45),  0.000000032331093],
  [(28,45),  0.000000032331093],
  [(29,45),  0.001073213479922],
  [(30,45),  0.000001044403926],
  [(31,45), -0.000000081943046],
  [(32,45),  0.000001044403926],
  [(33,45),  0.001073213479922],
  [(34,45),  0.000001044403926],
  [(35,45), -0.000000081943046],
  [(36,45),  0.000001044403926],
  [(37,45), -0.000211348704706],
  [(38,45), -0.000000081943046],
  [(39,45),  0.000000022686778],
  [(40,45), -0.000000081943046],
  [(41,45),  0.001069814673736],
  [(42,45),  0.000000575603073],
  [(43,45), -0.000000114274140],
  [(44,45),  0.000000575603073],
  [(45,45),  0.994214147055179],
  [(46,45),  0.001069814673736],
  [(47,45), -0.000211817505560],
  [(48,45),  0.001069814673736],
  [(49,45),  0.000004071402540],
  [(50,45),  0.000000468800853],
  [(51,45),  0.000000032331093],
  [(52,45),  0.000000468800853],
  [(53,45),  0.000000354526714],
  [(54,45),  0.000000032331093],
  [(56,45),  0.000000032331093],
  [(57,45),  0.000001044403926],
  [(58,45),  0.000000032331093],
  [(60,45),  0.000000032331093],
  [(61,45),  0.001073213479922],
  [(62,45),  0.000001044403926],
  [(63,45), -0.000000081943046],
  [(64,45),  0.000001044403926],
   [(1,46),  0.000000015518925],
   [(2,46), -0.000000017280860],
   [(3,46),  0.000000007759462],
   [(6,46),  0.000000030446240],
   [(9,46),  0.000000007759462],
  [(10,46), -0.000000081943046],
  [(11,46),  0.000000002586487],
  [(13,46), -0.000000017280860],
  [(14,46), -0.000211597330814],
  [(15,46), -0.000000081943046],
  [(16,46),  0.000000030446240],
  [(17,46),  0.000000096993280],
  [(18,46),  0.000000795777819],
  [(19,46),  0.000000032331093],
  [(20,46),  0.000000007759462],
  [(21,46),  0.000000007759462],
  [(22,46), -0.000000081943046],
  [(23,46),  0.000000002586487],
  [(25,46),  0.000000032331093],
  [(26,46),  0.000000583362535],
  [(28,46),  0.000000002586487],
  [(29,46),  0.000000795777819],
  [(30,46),  0.001069814673736],
  [(31,46),  0.000000583362535],
  [(32,46), -0.000000081943046],
  [(33,46),  0.000000795777819],
  [(34,46),  0.001069814673736],
  [(35,46),  0.000000583362535],
  [(36,46), -0.000000081943046],
  [(37,46), -0.000000081943046],
  [(38,46), -0.000211809746097],
  [(39,46), -0.000000114274140],
  [(40,46),  0.000000025273265],
  [(41,46),  0.000000583362535],
  [(42,46),  0.001069717680456],
  [(43,46),  0.000000575603073],
  [(44,46), -0.000000114274140],
  [(45,46),  0.001069814673736],
  [(46,46),  0.994214460147854],
  [(47,46),  0.001069717680456],
  [(48,46), -0.000211809746097],
  [(49,46),  0.000000096993280],
  [(50,46),  0.000000795777819],
  [(51,46),  0.000000032331093],
  [(52,46),  0.000000007759462],
  [(53,46),  0.000000007759462],
  [(54,46), -0.000000081943046],
  [(55,46),  0.000000002586487],
  [(57,46),  0.000000032331093],
  [(58,46),  0.000000583362535],
  [(60,46),  0.000000002586487],
  [(61,46),  0.000000795777819],
  [(62,46),  0.001069814673736],
  [(63,46),  0.000000583362535],
  [(64,46), -0.000000081943046],
   [(3,47), -0.000000114274140],
   [(7,47),  0.000000022686778],
  [(11,47), -0.000000114274140],
  [(13,47),  0.000000022686778],
  [(14,47), -0.000000114274140],
  [(15,47), -0.000211817505560],
  [(16,47), -0.000000114274140],
  [(19,47),  0.000000575603073],
  [(23,47), -0.000000114274140],
  [(27,47),  0.000000575603073],
  [(29,47), -0.000000114274140],
  [(30,47),  0.000000575603073],
  [(31,47),  0.001069717680456],
  [(32,47),  0.000000575603073],
  [(33,47), -0.000000114274140],
  [(34,47),  0.000000575603073],
  [(35,47),  0.001069717680456],
  [(36,47),  0.000000575603073],
  [(37,47),  0.000000022686778],
  [(38,47), -0.000000114274140],
  [(39,47), -0.000211817505560],
  [(40,47), -0.000000114274140],
  [(41,47), -0.000000114274140],
  [(42,47),  0.000000575603073],
  [(43,47),  0.001069717680456],
  [(44,47),  0.000000575603073],
  [(45,47), -0.000211817505560],
  [(46,47),  0.001069717680456],
  [(47,47),  0.994216021328346],
  [(48,47),  0.001069717680456],
  [(51,47),  0.000000575603073],
  [(55,47), -0.000000114274140],
  [(59,47),  0.000000575603073],
  [(61,47), -0.000000114274140],
  [(62,47),  0.000000575603073],
  [(63,47),  0.001069717680456],
  [(64,47),  0.000000575603073],
   [(1,48),  0.000000015518925],
   [(3,48),  0.000000007759462],
   [(4,48), -0.000000017280860],
   [(8,48),  0.000000030446240],
   [(9,48),  0.000000007759462],
  [(11,48),  0.000000002586487],
  [(12,48), -0.000000081943046],
  [(13,48), -0.000000017280860],
  [(14,48),  0.000000030446240],
  [(15,48), -0.000000081943046],
  [(16,48), -0.000211597330814],
  [(17,48),  0.000000096993280],
  [(18,48),  0.000000007759462],
  [(19,48),  0.000000032331093],
  [(20,48),  0.000000795777819],
  [(21,48),  0.000000007759462],
  [(23,48),  0.000000002586487],
  [(24,48), -0.000000081943046],
  [(25,48),  0.000000032331093],
  [(26,48),  0.000000002586487],
  [(28,48),  0.000000583362535],
  [(29,48),  0.000000795777819],
  [(30,48), -0.000000081943046],
  [(31,48),  0.000000583362535],
  [(32,48),  0.001069814673736],
  [(33,48),  0.000000795777819],
  [(34,48), -0.000000081943046],
  [(35,48),  0.000000583362535],
  [(36,48),  0.001069814673736],
  [(37,48), -0.000000081943046],
  [(38,48),  0.000000025273265],
  [(39,48), -0.000000114274140],
  [(40,48), -0.000211809746097],
  [(41,48),  0.000000583362535],
  [(42,48), -0.000000114274140],
  [(43,48),  0.000000575603073],
  [(44,48),  0.001069717680456],
  [(45,48),  0.001069814673736],
  [(46,48), -0.000211809746097],
  [(47,48),  0.001069717680456],
  [(48,48),  0.994214460147854],
  [(49,48),  0.000000096993280],
  [(50,48),  0.000000007759462],
  [(51,48),  0.000000032331093],
  [(52,48),  0.000000795777819],
  [(53,48),  0.000000007759462],
  [(55,48),  0.000000002586487],
  [(56,48), -0.000000081943046],
  [(57,48),  0.000000032331093],
  [(58,48),  0.000000002586487],
  [(60,48),  0.000000583362535],
  [(61,48),  0.000000795777819],
  [(62,48), -0.000000081943046],
  [(63,48),  0.000000583362535],
  [(64,48),  0.001069814673736],
   [(1,49),  0.001317465291762],
   [(2,49),  0.000039140535340],
   [(3,49),  0.000003381525327],
   [(4,49),  0.000039140535340],
   [(5,49),  0.000039140535340],
   [(6,49),  0.000003495799467],
   [(7,49),  0.000000096993280],
   [(8,49),  0.000003495799467],
   [(9,49),  0.000003381525327],
  [(10,49),  0.000000096993280],
  [(12,49),  0.000000096993280],
  [(13,49),  0.000039140535340],
  [(14,49),  0.000003495799467],
  [(15,49),  0.000000096993280],
  [(16,49),  0.000003495799467],
  [(17,49), -0.000173252573293],
  [(18,49),  0.000003381525327],
  [(19,49),  0.000000119680058],
  [(20,49),  0.000003381525327],
  [(21,49),  0.000003381525327],
  [(22,49),  0.000000096993280],
  [(24,49),  0.000000096993280],
  [(25,49),  0.000000119680058],
  [(29,49),  0.000003381525327],
  [(30,49),  0.000000096993280],
  [(32,49),  0.000000096993280],
  [(33,49),  0.001108282612722],
  [(34,49),  0.000004071402540],
  [(35,49), -0.000000017280860],
  [(36,49),  0.000004071402540],
  [(37,49),  0.000004071402540],
  [(38,49),  0.000000096993280],
  [(40,49),  0.000000096993280],
  [(41,49), -0.000000017280860],
  [(45,49),  0.000004071402540],
  [(46,49),  0.000000096993280],
  [(48,49),  0.000000096993280],
  [(49,49),  0.994452172935901],
  [(50,49),  0.001108282612722],
  [(51,49), -0.000208321706093],
  [(52,49),  0.001108282612722],
  [(53,49),  0.001108282612722],
  [(54,49),  0.000004071402540],
  [(55,49), -0.000000017280860],
  [(56,49),  0.000004071402540],
  [(57,49), -0.000208321706093],
  [(58,49), -0.000000017280860],
  [(59,49),  0.000000022686778],
  [(60,49), -0.000000017280860],
  [(61,49),  0.001108282612722],
  [(62,49),  0.000004071402540],
  [(63,49), -0.000000017280860],
  [(64,49),  0.000004071402540],
   [(1,50),  0.000027931951489],
   [(2,50),  0.001108282612722],
   [(3,50),  0.000006267168743],
   [(4,50),  0.000003381525327],
   [(5,50),  0.000003495799467],
   [(6,50),  0.000006267168743],
   [(7,50),  0.000000468800853],
   [(8,50),  0.000000220174746],
   [(9,50),  0.000000220174746],
  [(10,50),  0.000000354526714],
  [(11,50),  0.000000007759462],
  [(13,50),  0.000003495799467],
  [(14,50),  0.000006267168743],
  [(15,50),  0.000000468800853],
  [(16,50),  0.000000220174746],
  [(17,50),  0.000003381525327],
  [(18,50), -0.000206125939889],
  [(19,50),  0.000000354526714],
  [(20,50),  0.000000242861523],
  [(21,50),  0.000000220174746],
  [(22,50),  0.000000354526714],
  [(23,50),  0.000000007759462],
  [(26,50),  0.000000030446240],
  [(29,50),  0.000000220174746],
  [(30,50),  0.000000354526714],
  [(31,50),  0.000000007759462],
  [(33,50),  0.000006267168743],
  [(34,50),  0.001073213479922],
  [(35,50),  0.000000795777819],
  [(36,50),  0.000000354526714],
  [(37,50),  0.000000468800853],
  [(38,50),  0.000000795777819],
  [(40,50),  0.000000007759462],
  [(41,50),  0.000000007759462],
  [(42,50), -0.000000114274140],
  [(45,50),  0.000000468800853],
  [(46,50),  0.000000795777819],
  [(48,50),  0.000000007759462],
  [(49,50),  0.001108282612722],
  [(50,50),  0.994236020504560],
  [(51,50),  0.001073213479922],
  [(52,50), -0.000206125939889],
  [(53,50),  0.000006267168743],
  [(54,50),  0.001073213479922],
  [(55,50),  0.000000795777819],
  [(56,50),  0.000000354526714],
  [(57,50),  0.000000354526714],
  [(58,50), -0.000211597330814],
  [(59,50), -0.000000114274140],
  [(60,50),  0.000000030446240],
  [(61,50),  0.000006267168743],
  [(62,50),  0.001073213479922],
  [(63,50),  0.000000795777819],
  [(64,50),  0.000000354526714],
   [(1,51),  0.000001195135140],
   [(2,51),  0.000004071402540],
   [(3,51),  0.001073213479922],
   [(4,51),  0.000004071402540],
   [(5,51),  0.000000096993280],
   [(6,51),  0.000000468800853],
   [(7,51),  0.000001044403926],
   [(8,51),  0.000000468800853],
  [(10,51),  0.000000032331093],
  [(11,51), -0.000000081943046],
  [(12,51),  0.000000032331093],
  [(13,51),  0.000000096993280],
  [(14,51),  0.000000468800853],
  [(15,51),  0.000001044403926],
  [(16,51),  0.000000468800853],
  [(17,51),  0.000000119680058],
  [(18,51),  0.000000354526714],
  [(19,51), -0.000211348704706],
  [(20,51),  0.000000354526714],
  [(22,51),  0.000000032331093],
  [(23,51), -0.000000081943046],
  [(24,51),  0.000000032331093],
  [(27,51),  0.000000022686778],
  [(30,51),  0.000000032331093],
  [(31,51), -0.000000081943046],
  [(32,51),  0.000000032331093],
  [(33,51),  0.000000354526714],
  [(34,51),  0.000001044403926],
  [(35,51),  0.001069814673736],
  [(36,51),  0.000001044403926],
  [(37,51),  0.000000032331093],
  [(38,51),  0.000000032331093],
  [(39,51),  0.000000575603073],
  [(40,51),  0.000000032331093],
  [(43,51), -0.000000114274140],
  [(45,51),  0.000000032331093],
  [(46,51),  0.000000032331093],
  [(47,51),  0.000000575603073],
  [(48,51),  0.000000032331093],
  [(49,51), -0.000208321706093],
  [(50,51),  0.001073213479922],
  [(51,51),  0.994214147055179],
  [(52,51),  0.001073213479922],
  [(53,51),  0.000000354526714],
  [(54,51),  0.000001044403926],
  [(55,51),  0.001069814673736],
  [(56,51),  0.000001044403926],
  [(57,51),  0.000000055017871],
  [(58,51), -0.000000081943046],
  [(59,51), -0.000211817505560],
  [(60,51), -0.000000081943046],
  [(61,51),  0.000000354526714],
  [(62,51),  0.000001044403926],
  [(63,51),  0.001069814673736],
  [(64,51),  0.000001044403926],
   [(1,52),  0.000027931951489],
   [(2,52),  0.000003381525327],
   [(3,52),  0.000006267168743],
   [(4,52),  0.001108282612722],
   [(5,52),  0.000003495799467],
   [(6,52),  0.000000220174746],
   [(7,52),  0.000000468800853],
   [(8,52),  0.000006267168743],
   [(9,52),  0.000000220174746],
  [(11,52),  0.000000007759462],
  [(12,52),  0.000000354526714],
  [(13,52),  0.000003495799467],
  [(14,52),  0.000000220174746],
  [(15,52),  0.000000468800853],
  [(16,52),  0.000006267168743],
  [(17,52),  0.000003381525327],
  [(18,52),  0.000000242861523],
  [(19,52),  0.000000354526714],
  [(20,52), -0.000206125939889],
  [(21,52),  0.000000220174746],
  [(23,52),  0.000000007759462],
  [(24,52),  0.000000354526714],
  [(28,52),  0.000000030446240],
  [(29,52),  0.000000220174746],
  [(31,52),  0.000000007759462],
  [(32,52),  0.000000354526714],
  [(33,52),  0.000006267168743],
  [(34,52),  0.000000354526714],
  [(35,52),  0.000000795777819],
  [(36,52),  0.001073213479922],
  [(37,52),  0.000000468800853],
  [(38,52),  0.000000007759462],
  [(40,52),  0.000000795777819],
  [(41,52),  0.000000007759462],
  [(44,52), -0.000000114274140],
  [(45,52),  0.000000468800853],
  [(46,52),  0.000000007759462],
  [(48,52),  0.000000795777819],
  [(49,52),  0.001108282612722],
  [(50,52), -0.000206125939889],
  [(51,52),  0.001073213479922],
  [(52,52),  0.994236020504560],
  [(53,52),  0.000006267168743],
  [(54,52),  0.000000354526714],
  [(55,52),  0.000000795777819],
  [(56,52),  0.001073213479922],
  [(57,52),  0.000000354526714],
  [(58,52),  0.000000030446240],
  [(59,52), -0.000000114274140],
  [(60,52), -0.000211597330814],
  [(61,52),  0.000006267168743],
  [(62,52),  0.000000354526714],
  [(63,52),  0.000000795777819],
  [(64,52),  0.001073213479922],
   [(1,53),  0.000027931951489],
   [(2,53),  0.000003495799467],
   [(3,53),  0.000000220174746],
   [(4,53),  0.000003495799467],
   [(5,53),  0.001108282612722],
   [(6,53),  0.000006267168743],
   [(7,53),  0.000000354526714],
   [(8,53),  0.000006267168743],
   [(9,53),  0.000006267168743],
  [(10,53),  0.000000468800853],
  [(11,53),  0.000000007759462],
  [(12,53),  0.000000468800853],
  [(13,53),  0.000003381525327],
  [(14,53),  0.000000220174746],
  [(16,53),  0.000000220174746],
  [(17,53),  0.000003381525327],
  [(18,53),  0.000000220174746],
  [(20,53),  0.000000220174746],
  [(21,53), -0.000206125939889],
  [(22,53),  0.000000354526714],
  [(23,53),  0.000000030446240],
  [(24,53),  0.000000354526714],
  [(25,53),  0.000000354526714],
  [(26,53),  0.000000007759462],
  [(28,53),  0.000000007759462],
  [(29,53),  0.000000242861523],
  [(33,53),  0.000006267168743],
  [(34,53),  0.000000468800853],
  [(35,53),  0.000000007759462],
  [(36,53),  0.000000468800853],
  [(37,53),  0.001073213479922],
  [(38,53),  0.000000795777819],
  [(39,53), -0.000000114274140],
  [(40,53),  0.000000795777819],
  [(41,53),  0.000000795777819],
  [(45,53),  0.000000354526714],
  [(46,53),  0.000000007759462],
  [(48,53),  0.000000007759462],
  [(49,53),  0.001108282612722],
  [(50,53),  0.000006267168743],
  [(51,53),  0.000000354526714],
  [(52,53),  0.000006267168743],
  [(53,53),  0.994236020504560],
  [(54,53),  0.001073213479922],
  [(55,53), -0.000211597330814],
  [(56,53),  0.001073213479922],
  [(57,53),  0.001073213479922],
  [(58,53),  0.000000795777819],
  [(59,53), -0.000000114274140],
  [(60,53),  0.000000795777819],
  [(61,53), -0.000206125939889],
  [(62,53),  0.000000354526714],
  [(63,53),  0.000000030446240],
  [(64,53),  0.000000354526714],
   [(1,54),  0.000001309409280],
   [(2,54),  0.000004071402540],
   [(3,54),  0.000000468800853],
   [(4,54),  0.000000096993280],
   [(5,54),  0.000004071402540],
   [(6,54),  0.001073213479922],
   [(7,54),  0.000001044403926],
   [(8,54),  0.000000354526714],
   [(9,54),  0.000000468800853],
  [(10,54),  0.000001044403926],
  [(11,54),  0.000000032331093],
  [(12,54),  0.000000032331093],
  [(13,54),  0.000000096993280],
  [(14,54),  0.000000354526714],
  [(15,54),  0.000000032331093],
  [(17,54),  0.000000096993280],
  [(18,54),  0.000000354526714],
  [(19,54),  0.000000032331093],
  [(21,54),  0.000000354526714],
  [(22,54), -0.000211348704706],
  [(23,54), -0.000000081943046],
  [(24,54),  0.000000055017871],
  [(25,54),  0.000000032331093],
  [(26,54), -0.000000081943046],
  [(30,54),  0.000000055017871],
  [(33,54),  0.000000468800853],
  [(34,54),  0.000001044403926],
  [(35,54),  0.000000032331093],
  [(36,54),  0.000000032331093],
  [(37,54),  0.000001044403926],
  [(38,54),  0.001069814673736],
  [(39,54),  0.000000575603073],
  [(40,54), -0.000000081943046],
  [(41,54),  0.000000032331093],
  [(42,54),  0.000000575603073],
  [(45,54),  0.000000032331093],
  [(46,54), -0.000000081943046],
  [(49,54),  0.000004071402540],
  [(50,54),  0.001073213479922],
  [(51,54),  0.000001044403926],
  [(52,54),  0.000000354526714],
  [(53,54),  0.001073213479922],
  [(54,54),  0.994212887656419],
  [(55,54),  0.001069814673736],
  [(56,54), -0.000211348704706],
  [(57,54),  0.000001044403926],
  [(58,54),  0.001069814673736],
  [(59,54),  0.000000575603073],
  [(60,54), -0.000000081943046],
  [(61,54),  0.000000354526714],
  [(62,54), -0.000211348704706],
  [(63,54), -0.000000081943046],
  [(64,54),  0.000000055017871],
   [(1,55),  0.000000015518925],
   [(2,55),  0.000000096993280],
   [(3,55),  0.000000795777819],
   [(4,55),  0.000000096993280],
   [(5,55), -0.000000017280860],
   [(6,55),  0.000000795777819],
   [(7,55),  0.001069814673736],
   [(8,55),  0.000000795777819],
   [(9,55),  0.000000007759462],
  [(10,55),  0.000000032331093],
  [(11,55),  0.000000583362535],
  [(12,55),  0.000000032331093],
  [(14,55),  0.000000007759462],
  [(15,55), -0.000000081943046],
  [(16,55),  0.000000007759462],
  [(18,55),  0.000000007759462],
  [(19,55), -0.000000081943046],
  [(20,55),  0.000000007759462],
  [(21,55),  0.000000030446240],
  [(22,55), -0.000000081943046],
  [(23,55), -0.000211809746097],
  [(24,55), -0.000000081943046],
  [(26,55),  0.000000002586487],
  [(27,55), -0.000000114274140],
  [(28,55),  0.000000002586487],
  [(31,55),  0.000000025273265],
  [(33,55),  0.000000007759462],
  [(34,55),  0.000000032331093],
  [(35,55),  0.000000583362535],
  [(36,55),  0.000000032331093],
  [(37,55), -0.000000081943046],
  [(38,55),  0.000000583362535],
  [(39,55),  0.001069717680456],
  [(40,55),  0.000000583362535],
  [(41,55),  0.000000002586487],
  [(43,55),  0.000000575603073],
  [(46,55),  0.000000002586487],
  [(47,55), -0.000000114274140],
  [(48,55),  0.000000002586487],
  [(49,55), -0.000000017280860],
  [(50,55),  0.000000795777819],
  [(51,55),  0.001069814673736],
  [(52,55),  0.000000795777819],
  [(53,55), -0.000211597330814],
  [(54,55),  0.001069814673736],
  [(55,55),  0.994214460147854],
  [(56,55),  0.001069814673736],
  [(57,55), -0.000000081943046],
  [(58,55),  0.000000583362535],
  [(59,55),  0.001069717680456],
  [(60,55),  0.000000583362535],
  [(61,55),  0.000000030446240],
  [(62,55), -0.000000081943046],
  [(63,55), -0.000211809746097],
  [(64,55), -0.000000081943046],
   [(1,56),  0.000001309409280],
   [(2,56),  0.000000096993280],
   [(3,56),  0.000000468800853],
   [(4,56),  0.000004071402540],
   [(5,56),  0.000004071402540],
   [(6,56),  0.000000354526714],
   [(7,56),  0.000001044403926],
   [(8,56),  0.001073213479922],
   [(9,56),  0.000000468800853],
  [(10,56),  0.000000032331093],
  [(11,56),  0.000000032331093],
  [(12,56),  0.000001044403926],
  [(13,56),  0.000000096993280],
  [(15,56),  0.000000032331093],
  [(16,56),  0.000000354526714],
  [(17,56),  0.000000096993280],
  [(19,56),  0.000000032331093],
  [(20,56),  0.000000354526714],
  [(21,56),  0.000000354526714],
  [(22,56),  0.000000055017871],
  [(23,56), -0.000000081943046],
  [(24,56), -0.000211348704706],
  [(25,56),  0.000000032331093],
  [(28,56), -0.000000081943046],
  [(32,56),  0.000000055017871],
  [(33,56),  0.000000468800853],
  [(34,56),  0.000000032331093],
  [(35,56),  0.000000032331093],
  [(36,56),  0.000001044403926],
  [(37,56),  0.000001044403926],
  [(38,56), -0.000000081943046],
  [(39,56),  0.000000575603073],
  [(40,56),  0.001069814673736],
  [(41,56),  0.000000032331093],
  [(44,56),  0.000000575603073],
  [(45,56),  0.000000032331093],
  [(48,56), -0.000000081943046],
  [(49,56),  0.000004071402540],
  [(50,56),  0.000000354526714],
  [(51,56),  0.000001044403926],
  [(52,56),  0.001073213479922],
  [(53,56),  0.001073213479922],
  [(54,56), -0.000211348704706],
  [(55,56),  0.001069814673736],
  [(56,56),  0.994212887656419],
  [(57,56),  0.000001044403926],
  [(58,56), -0.000000081943046],
  [(59,56),  0.000000575603073],
  [(60,56),  0.001069814673736],
  [(61,56),  0.000000354526714],
  [(62,56),  0.000000055017871],
  [(63,56), -0.000000081943046],
  [(64,56), -0.000211348704706],
   [(1,57),  0.000001195135140],
   [(2,57),  0.000000096993280],
   [(4,57),  0.000000096993280],
   [(5,57),  0.000004071402540],
   [(6,57),  0.000000468800853],
   [(7,57),  0.000000032331093],
   [(8,57),  0.000000468800853],
   [(9,57),  0.001073213479922],
  [(10,57),  0.000001044403926],
  [(11,57), -0.000000081943046],
  [(12,57),  0.000001044403926],
  [(13,57),  0.000004071402540],
  [(14,57),  0.000000468800853],
  [(15,57),  0.000000032331093],
  [(16,57),  0.000000468800853],
  [(17,57),  0.000000119680058],
  [(21,57),  0.000000354526714],
  [(22,57),  0.000000032331093],
  [(24,57),  0.000000032331093],
  [(25,57), -0.000211348704706],
  [(26,57), -0.000000081943046],
  [(27,57),  0.000000022686778],
  [(28,57), -0.000000081943046],
  [(29,57),  0.000000354526714],
  [(30,57),  0.000000032331093],
  [(32,57),  0.000000032331093],
  [(33,57),  0.000000354526714],
  [(34,57),  0.000000032331093],
  [(36,57),  0.000000032331093],
  [(37,57),  0.000001044403926],
  [(38,57),  0.000000032331093],
  [(40,57),  0.000000032331093],
  [(41,57),  0.001069814673736],
  [(42,57),  0.000000575603073],
  [(43,57), -0.000000114274140],
  [(44,57),  0.000000575603073],
  [(45,57),  0.000001044403926],
  [(46,57),  0.000000032331093],
  [(48,57),  0.000000032331093],
  [(49,57), -0.000208321706093],
  [(50,57),  0.000000354526714],
  [(51,57),  0.000000055017871],
  [(52,57),  0.000000354526714],
  [(53,57),  0.001073213479922],
  [(54,57),  0.000001044403926],
  [(55,57), -0.000000081943046],
  [(56,57),  0.000001044403926],
  [(57,57),  0.994214147055179],
  [(58,57),  0.001069814673736],
  [(59,57), -0.000211817505560],
  [(60,57),  0.001069814673736],
  [(61,57),  0.001073213479922],
  [(62,57),  0.000001044403926],
  [(63,57), -0.000000081943046],
  [(64,57),  0.000001044403926],
   [(1,58),  0.000000015518925],
   [(2,58), -0.000000017280860],
   [(3,58),  0.000000007759462],
   [(5,58),  0.000000096993280],
   [(6,58),  0.000000795777819],
   [(7,58),  0.000000032331093],
   [(8,58),  0.000000007759462],
   [(9,58),  0.000000795777819],
  [(10,58),  0.001069814673736],
  [(11,58),  0.000000583362535],
  [(12,58), -0.000000081943046],
  [(13,58),  0.000000096993280],
  [(14,58),  0.000000795777819],
  [(15,58),  0.000000032331093],
  [(16,58),  0.000000007759462],
  [(18,58),  0.000000030446240],
  [(21,58),  0.000000007759462],
  [(22,58), -0.000000081943046],
  [(23,58),  0.000000002586487],
  [(25,58), -0.000000081943046],
  [(26,58), -0.000211809746097],
  [(27,58), -0.000000114274140],
  [(28,58),  0.000000025273265],
  [(29,58),  0.000000007759462],
  [(30,58), -0.000000081943046],
  [(31,58),  0.000000002586487],
  [(33,58),  0.000000007759462],
  [(34,58), -0.000000081943046],
  [(35,58),  0.000000002586487],
  [(37,58),  0.000000032331093],
  [(38,58),  0.000000583362535],
  [(40,58),  0.000000002586487],
  [(41,58),  0.000000583362535],
  [(42,58),  0.001069717680456],
  [(43,58),  0.000000575603073],
  [(44,58), -0.000000114274140],
  [(45,58),  0.000000032331093],
  [(46,58),  0.000000583362535],
  [(48,58),  0.000000002586487],
  [(49,58), -0.000000017280860],
  [(50,58), -0.000211597330814],
  [(51,58), -0.000000081943046],
  [(52,58),  0.000000030446240],
  [(53,58),  0.000000795777819],
  [(54,58),  0.001069814673736],
  [(55,58),  0.000000583362535],
  [(56,58), -0.000000081943046],
  [(57,58),  0.001069814673736],
  [(58,58),  0.994214460147854],
  [(59,58),  0.001069717680456],
  [(60,58), -0.000211809746097],
  [(61,58),  0.000000795777819],
  [(62,58),  0.001069814673736],
  [(63,58),  0.000000583362535],
  [(64,58), -0.000000081943046],
   [(3,59), -0.000000114274140],
   [(7,59),  0.000000575603073],
   [(9,59), -0.000000114274140],
  [(10,59),  0.000000575603073],
  [(11,59),  0.001069717680456],
  [(12,59),  0.000000575603073],
  [(15,59),  0.000000575603073],
  [(19,59),  0.000000022686778],
  [(23,59), -0.000000114274140],
  [(25,59),  0.000000022686778],
  [(26,59), -0.000000114274140],
  [(27,59), -0.000211817505560],
  [(28,59), -0.000000114274140],
  [(31,59), -0.000000114274140],
  [(35,59), -0.000000114274140],
  [(39,59),  0.000000575603073],
  [(41,59), -0.000000114274140],
  [(42,59),  0.000000575603073],
  [(43,59),  0.001069717680456],
  [(44,59),  0.000000575603073],
  [(47,59),  0.000000575603073],
  [(49,59),  0.000000022686778],
  [(50,59), -0.000000114274140],
  [(51,59), -0.000211817505560],
  [(52,59), -0.000000114274140],
  [(53,59), -0.000000114274140],
  [(54,59),  0.000000575603073],
  [(55,59),  0.001069717680456],
  [(56,59),  0.000000575603073],
  [(57,59), -0.000211817505560],
  [(58,59),  0.001069717680456],
  [(59,59),  0.994216021328346],
  [(60,59),  0.001069717680456],
  [(61,59), -0.000000114274140],
  [(62,59),  0.000000575603073],
  [(63,59),  0.001069717680456],
  [(64,59),  0.000000575603073],
   [(1,60),  0.000000015518925],
   [(3,60),  0.000000007759462],
   [(4,60), -0.000000017280860],
   [(5,60),  0.000000096993280],
   [(6,60),  0.000000007759462],
   [(7,60),  0.000000032331093],
   [(8,60),  0.000000795777819],
   [(9,60),  0.000000795777819],
  [(10,60), -0.000000081943046],
  [(11,60),  0.000000583362535],
  [(12,60),  0.001069814673736],
  [(13,60),  0.000000096993280],
  [(14,60),  0.000000007759462],
  [(15,60),  0.000000032331093],
  [(16,60),  0.000000795777819],
  [(20,60),  0.000000030446240],
  [(21,60),  0.000000007759462],
  [(23,60),  0.000000002586487],
  [(24,60), -0.000000081943046],
  [(25,60), -0.000000081943046],
  [(26,60),  0.000000025273265],
  [(27,60), -0.000000114274140],
  [(28,60), -0.000211809746097],
  [(29,60),  0.000000007759462],
  [(31,60),  0.000000002586487],
  [(32,60), -0.000000081943046],
  [(33,60),  0.000000007759462],
  [(35,60),  0.000000002586487],
  [(36,60), -0.000000081943046],
  [(37,60),  0.000000032331093],
  [(38,60),  0.000000002586487],
  [(40,60),  0.000000583362535],
  [(41,60),  0.000000583362535],
  [(42,60), -0.000000114274140],
  [(43,60),  0.000000575603073],
  [(44,60),  0.001069717680456],
  [(45,60),  0.000000032331093],
  [(46,60),  0.000000002586487],
  [(48,60),  0.000000583362535],
  [(49,60), -0.000000017280860],
  [(50,60),  0.000000030446240],
  [(51,60), -0.000000081943046],
  [(52,60), -0.000211597330814],
  [(53,60),  0.000000795777819],
  [(54,60), -0.000000081943046],
  [(55,60),  0.000000583362535],
  [(56,60),  0.001069814673736],
  [(57,60),  0.001069814673736],
  [(58,60), -0.000211809746097],
  [(59,60),  0.001069717680456],
  [(60,60),  0.994214460147854],
  [(61,60),  0.000000795777819],
  [(62,60), -0.000000081943046],
  [(63,60),  0.000000583362535],
  [(64,60),  0.001069814673736],
   [(1,61),  0.000027931951489],
   [(2,61),  0.000003495799467],
   [(3,61),  0.000000220174746],
   [(4,61),  0.000003495799467],
   [(5,61),  0.000003381525327],
   [(6,61),  0.000000220174746],
   [(8,61),  0.000000220174746],
   [(9,61),  0.000006267168743],
  [(10,61),  0.000000468800853],
  [(11,61),  0.000000007759462],
  [(12,61),  0.000000468800853],
  [(13,61),  0.001108282612722],
  [(14,61),  0.000006267168743],
  [(15,61),  0.000000354526714],
  [(16,61),  0.000006267168743],
  [(17,61),  0.000003381525327],
  [(18,61),  0.000000220174746],
  [(20,61),  0.000000220174746],
  [(21,61),  0.000000242861523],
  [(25,61),  0.000000354526714],
  [(26,61),  0.000000007759462],
  [(28,61),  0.000000007759462],
  [(29,61), -0.000206125939889],
  [(30,61),  0.000000354526714],
  [(31,61),  0.000000030446240],
  [(32,61),  0.000000354526714],
  [(33,61),  0.000006267168743],
  [(34,61),  0.000000468800853],
  [(35,61),  0.000000007759462],
  [(36,61),  0.000000468800853],
  [(37,61),  0.000000354526714],
  [(38,61),  0.000000007759462],
  [(40,61),  0.000000007759462],
  [(41,61),  0.000000795777819],
  [(45,61),  0.001073213479922],
  [(46,61),  0.000000795777819],
  [(47,61), -0.000000114274140],
  [(48,61),  0.000000795777819],
  [(49,61),  0.001108282612722],
  [(50,61),  0.000006267168743],
  [(51,61),  0.000000354526714],
  [(52,61),  0.000006267168743],
  [(53,61), -0.000206125939889],
  [(54,61),  0.000000354526714],
  [(55,61),  0.000000030446240],
  [(56,61),  0.000000354526714],
  [(57,61),  0.001073213479922],
  [(58,61),  0.000000795777819],
  [(59,61), -0.000000114274140],
  [(60,61),  0.000000795777819],
  [(61,61),  0.994236020504560],
  [(62,61),  0.001073213479922],
  [(63,61), -0.000211597330814],
  [(64,61),  0.001073213479922],
   [(1,62),  0.000001309409280],
   [(2,62),  0.000004071402540],
   [(3,62),  0.000000468800853],
   [(4,62),  0.000000096993280],
   [(5,62),  0.000000096993280],
   [(6,62),  0.000000354526714],
   [(7,62),  0.000000032331093],
   [(9,62),  0.000000468800853],
  [(10,62),  0.000001044403926],
  [(11,62),  0.000000032331093],
  [(12,62),  0.000000032331093],
  [(13,62),  0.000004071402540],
  [(14,62),  0.001073213479922],
  [(15,62),  0.000001044403926],
  [(16,62),  0.000000354526714],
  [(17,62),  0.000000096993280],
  [(18,62),  0.000000354526714],
  [(19,62),  0.000000032331093],
  [(22,62),  0.000000055017871],
  [(25,62),  0.000000032331093],
  [(26,62), -0.000000081943046],
  [(29,62),  0.000000354526714],
  [(30,62), -0.000211348704706],
  [(31,62), -0.000000081943046],
  [(32,62),  0.000000055017871],
  [(33,62),  0.000000468800853],
  [(34,62),  0.000001044403926],
  [(35,62),  0.000000032331093],
  [(36,62),  0.000000032331093],
  [(37,62),  0.000000032331093],
  [(38,62), -0.000000081943046],
  [(41,62),  0.000000032331093],
  [(42,62),  0.000000575603073],
  [(45,62),  0.000001044403926],
  [(46,62),  0.001069814673736],
  [(47,62),  0.000000575603073],
  [(48,62), -0.000000081943046],
  [(49,62),  0.000004071402540],
  [(50,62),  0.001073213479922],
  [(51,62),  0.000001044403926],
  [(52,62),  0.000000354526714],
  [(53,62),  0.000000354526714],
  [(54,62), -0.000211348704706],
  [(55,62), -0.000000081943046],
  [(56,62),  0.000000055017871],
  [(57,62),  0.000001044403926],
  [(58,62),  0.001069814673736],
  [(59,62),  0.000000575603073],
  [(60,62), -0.000000081943046],
  [(61,62),  0.001073213479922],
  [(62,62),  0.994212887656419],
  [(63,62),  0.001069814673736],
  [(64,62), -0.000211348704706],
   [(1,63),  0.000000015518925],
   [(2,63),  0.000000096993280],
   [(3,63),  0.000000795777819],
   [(4,63),  0.000000096993280],
   [(6,63),  0.000000007759462],
   [(7,63), -0.000000081943046],
   [(8,63),  0.000000007759462],
   [(9,63),  0.000000007759462],
  [(10,63),  0.000000032331093],
  [(11,63),  0.000000583362535],
  [(12,63),  0.000000032331093],
  [(13,63), -0.000000017280860],
  [(14,63),  0.000000795777819],
  [(15,63),  0.001069814673736],
  [(16,63),  0.000000795777819],
  [(18,63),  0.000000007759462],
  [(19,63), -0.000000081943046],
  [(20,63),  0.000000007759462],
  [(23,63),  0.000000025273265],
  [(26,63),  0.000000002586487],
  [(27,63), -0.000000114274140],
  [(28,63),  0.000000002586487],
  [(29,63),  0.000000030446240],
  [(30,63), -0.000000081943046],
  [(31,63), -0.000211809746097],
  [(32,63), -0.000000081943046],
  [(33,63),  0.000000007759462],
  [(34,63),  0.000000032331093],
  [(35,63),  0.000000583362535],
  [(36,63),  0.000000032331093],
  [(38,63),  0.000000002586487],
  [(39,63), -0.000000114274140],
  [(40,63),  0.000000002586487],
  [(41,63),  0.000000002586487],
  [(43,63),  0.000000575603073],
  [(45,63), -0.000000081943046],
  [(46,63),  0.000000583362535],
  [(47,63),  0.001069717680456],
  [(48,63),  0.000000583362535],
  [(49,63), -0.000000017280860],
  [(50,63),  0.000000795777819],
  [(51,63),  0.001069814673736],
  [(52,63),  0.000000795777819],
  [(53,63),  0.000000030446240],
  [(54,63), -0.000000081943046],
  [(55,63), -0.000211809746097],
  [(56,63), -0.000000081943046],
  [(57,63), -0.000000081943046],
  [(58,63),  0.000000583362535],
  [(59,63),  0.001069717680456],
  [(60,63),  0.000000583362535],
  [(61,63), -0.000211597330814],
  [(62,63),  0.001069814673736],
  [(63,63),  0.994214460147854],
  [(64,63),  0.001069814673736],
   [(1,64),  0.000001309409280],
   [(2,64),  0.000000096993280],
   [(3,64),  0.000000468800853],
   [(4,64),  0.000004071402540],
   [(5,64),  0.000000096993280],
   [(7,64),  0.000000032331093],
   [(8,64),  0.000000354526714],
   [(9,64),  0.000000468800853],
  [(10,64),  0.000000032331093],
  [(11,64),  0.000000032331093],
  [(12,64),  0.000001044403926],
  [(13,64),  0.000004071402540],
  [(14,64),  0.000000354526714],
  [(15,64),  0.000001044403926],
  [(16,64),  0.001073213479922],
  [(17,64),  0.000000096993280],
  [(19,64),  0.000000032331093],
  [(20,64),  0.000000354526714],
  [(24,64),  0.000000055017871],
  [(25,64),  0.000000032331093],
  [(28,64), -0.000000081943046],
  [(29,64),  0.000000354526714],
  [(30,64),  0.000000055017871],
  [(31,64), -0.000000081943046],
  [(32,64), -0.000211348704706],
  [(33,64),  0.000000468800853],
  [(34,64),  0.000000032331093],
  [(35,64),  0.000000032331093],
  [(36,64),  0.000001044403926],
  [(37,64),  0.000000032331093],
  [(40,64), -0.000000081943046],
  [(41,64),  0.000000032331093],
  [(44,64),  0.000000575603073],
  [(45,64),  0.000001044403926],
  [(46,64), -0.000000081943046],
  [(47,64),  0.000000575603073],
  [(48,64),  0.001069814673736],
  [(49,64),  0.000004071402540],
  [(50,64),  0.000000354526714],
  [(51,64),  0.000001044403926],
  [(52,64),  0.001073213479922],
  [(53,64),  0.000000354526714],
  [(54,64),  0.000000055017871],
  [(55,64), -0.000000081943046],
  [(56,64), -0.000211348704706],
  [(57,64),  0.000001044403926],
  [(58,64), -0.000000081943046],
  [(59,64),  0.000000575603073],
  [(60,64),  0.001069814673736],
  [(61,64),  0.001073213479922],
  [(62,64), -0.000211348704706],
  [(63,64),  0.001069814673736],
  [(64,64),  0.994212887656419]]

In [51]:
testMat = np.zeros((64,64))
for i in testL4Full:
	testMat[i[0][0]-1, i[0][1]-1]= i[1]
print(np.max(np.abs(testMat - trMat)))

6.161768006940349e-08


In [71]:
#Choose whether or not to have v_pppp and v_ppph sparse or not
sparse = True
# make normal-ordered Hamiltonian
refEn, fock_mats, two_body_int = ccm.get_norm_ordered_ham(
    myL, holes, tKin, inter, sparse=sparse, NO2B=True)
corrEn, t1, t2 = ccm.ccsd_solver(fock_mats, two_body_int, eps = 1.e-8, maxSteps = 1000, 
                                 max_diis = 10, delta = -10, mixing = 0.5,
                                 sparse=sparse, verbose=True, ccs=False)

phys_unit = lat.phys_unit(a_lat)
#converting the energy from lattice units to MeV
gsEn = (corrEn + refEn) * phys_unit

print(f'The ground state energy on a L={myL} size lattice is {gsEn} MeV')

Step 0: -14989.018190312248
Step 1: -16490.71691308944 difference = 0.09106327703589552
Step 2: -17728.158340433736 difference = 0.06980090111909623
Step 3: -18755.88524835521 difference = 0.05479490273654777
Step 4: -19614.040884003178 difference = 0.043752108029297604
Step 5: -20333.373839395113 difference = 0.03537696011855427
Step 6: -20938.073666523695 difference = 0.02888039447943055
Step 7: -21447.52251830004 difference = 0.02375327273076225
Step 8: -21877.456444276624 difference = 0.01965191552645322
Step 9: -22240.775746495063 difference = 0.01633572975869308
Step 10: -22548.13353829568 difference = 0.013631185538199883
Step 11: -20931.603426110436 difference = 0.0772291581909466
Step 12: -21442.064452852723 difference = 0.023806524220869665
Step 13: -21872.844615706534 difference = 0.019694748004769103
Step 14: -22236.87325189398 difference = 0.01637049562066648
Step 15: -22544.826764599686 difference = 0.013659608739565057
Step 16: -22805.57081507161 difference = 0.011433349